In [5]:
from bs4 import BeautifulSoup
import math
import requests
import re

class RunReport(object):   
    resultsSystemText = ''
    currentEventVolunteers = []
    currentEventRunners = {}
    photos = []
    
    toc = []
    runCount = []
    
    runners = {}
    volunteers = {}
    
    VOLUNTEER_START_TEXT = 'We are very grateful to the volunteers who made this event happen:'
    PB_TEXT = 'New PB!'
    
    sectionHeadingTemplate = '''<div style="border: 1px solid #ccc; padding: 5px; text-align: center;">
<h3 style="padding: 0; margin: 0;"><a name="{}"></a>{}</h3>
</div>
'''
    sectionContentTemplate = '''<div style="border: 1px solid #ccc; padding: 5px;">
<p style="padding: 0; margin: 0;">{}</p>
</div>
'''
    separator = '<div style="height: 10px;"></div>'
    
    tableStart = '<table style="width: 100%; padding: 0; border-collapse: collapse;" cellspacing="0" cellpadding="0"><tbody>'
    
    tableHeader =  '''<tr>
<th style="width: 50%; text-align: center; border: 1px solid #ccc !important;">Runners ({} or more from last {} events)</th>
<th style="width: 50%; text-align: center; border: 1px solid #ccc !important;">Volunteers ({} or more from last {} events)</th>
</tr>
'''
    tableCell = '''<td style="padding-left: 10px; border-width: 0 0 1px 0 !important; border-style: solid  !important; border-color: #ccc !important;">
{}</td>
'''
    
    photoTemplate = '<img src="{}" alt="{}" width="{}" height="{}" />'
    
    tableEnd = '</tbody></table>' 
    
    def setResultsSystem(self, text):
        self.resultsSystemText = text
        
        
    def setCurrentEvent(self, text):
        # set currentEventRunners and currentEventVolunteers 
        self.setCurrentEventRunners(text)
        self.setCurrentEventVolunteers(text)
            
            
    def parseCurrentEvent(self, text, parseType):
        soup = BeautifulSoup(text,'html.parser')
        if parseType == 'runners':
            rows = soup.find(id="results").find("tbody").find_all("tr")
        elif parseType == 'volunteers':
            # <p class="paddedb">
            # We are very grateful to the volunteers who made this event happen:
            # Aaryan BHATIA, Naomi (Tullae) CROTTY, Darren JEFFREYS, Gregory MOORE, Sadia NAZIER, Jenny PATERSON, Elijah SUMMERS, Kaila SWYER, Rosemary WAGHORN, Ashley WILLIS, Nathan WRIGHT
            # </p>  
            start = soup.p.find(text=re.compile(self.VOLUNTEER_START_TEXT))
            pos = start.find(':')
            sub = start[pos+1:]
            rows = sub.split(', ')          
        return rows
    
            
    def setCurrentEventRunners(self, text):
        rows = self.parseCurrentEvent(text, 'runners')      
        for row in rows:
            details = self.getRunnerDetails(row.find_all("td"))
            if details:
                self.currentEventRunners[details['id']] = {"name":details['name'],"time":details['time']}
                
                
    def setCurrentEventVolunteers(self, text):              
        names = self.parseCurrentEvent(text, 'volunteers')          
        for n in names:
            self.currentEventVolunteers.append(n)
            
        
    def addTOC(self, anchor, heading):
        # Add to toc
        self.toc.append('<li><a href="#'+anchor+'">'+heading+'</a></li>')
        
        
    def getSectionHeading(self, anchor, heading):
        self.addTOC(anchor, heading)
        return self.sectionHeadingTemplate.format(anchor, heading)
    
    
    def getSectionContent(self, content):
        return self.sectionContentTemplate.format(content)
    
    def getSectionSeparator(self):
        return self.separator
    
    
    def getCurrentEventVolunteerHTML(self):
        html = '<ul style="padding-bottom: 0;padding-top: 0;padding-left: 10px;margin-bottom: 0;margin-top: 0;margin-left: 10px">'
        for vol in self.currentEventVolunteers:
            html += '<li>'+vol+'</li>'
        html += '</ul>'
        return html
    
           
    def getRunnerDetails(self, cells):
        # <tr>
        # <td class="pos">2</td>
        # <td><a href="athletehistory?athleteNumber=111111" target="_top">Firstname LASTNAME</a></td>
        # <td>18:14</td>
        # <td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td>
        # <td>71.12 %</td>
        # <td>M</td>
        # <td>2</td>
        # <td><a href="../clubhistory?clubNum=1187"/></td>
        # <td>New PB!</td>
        # <td>2</td>
        # <td style="min-width:72px"/>
        # </tr>
        cell = cells[1]
        pb = 0
        if cells[8] == self.PB_TEXT:
            pb = 1
        name = cell.get_text()
        time = cells[2].get_text()
        if name != 'Unknown':
            href = cell.a["href"]
            # format of href="athletehistory?athleteNumber=208507"
            pos = href.find('=')
            id = href[pos+1:]
            return {"id":id,"name":name,"time":time,"pb":pb}
        else:
            return False
        
        
    def addRunners(self, text):
        rows = self.parseCurrentEvent(text, 'runners') 
        eventCount = 0
        for row in rows:
            cells = row.find_all("td")
            details = self.getRunnerDetails(cells)
            if details:
                eventCount = eventCount + 1
                pbCount = 0
                if details['id'] in self.runners:
                    count = self.runners[details['id']]['count'] + 1
                    if details['pb'] == 1:
                        pbCount = self.runners[details['id']]['pbCount'] + 1
                else:
                    count = 1
                    if details['pb'] == 1:
                         pbCount = 1   
                
                self.runners[details['id']] = {"name":details['name'],"pbCount":pbCount,"count":count}
        self.runCount.append(eventCount);
        
        
    def addVolunteers(self, text):             
        names = self.parseCurrentEvent(text, 'volunteers') 
        
        for n in names:
            if n in self.volunteers:
                count = self.volunteers[n] + 1
            else:
                 count = 1
            self.volunteers[n] = count
            
    def addPhoto(self, text, size, photoType):
        startPos = text.find('[img]') + len('[img]')
        endPos = text.find('.jpg') + len('.jpg')
        flickrLink = text[startPos:endPos]
        
        self.photos.append({'link':flickrLink,'size':size,'type':photoType})
     
    def getPhotoLinks(self, photoType):
        html = ''
        for p in self.photos:
            if p['type'] == photoType:
                dims = p['size']
                if int(dims[0]) == 640:
                    dims[0] = 620
                    dims[1] = math.floor(620 * int(dims[1]) / 640)
                elif int(dims[1]) == 640:
                    dims[1] = 620
                    dims[0] = math.floor(620 * int(dims[0]) / 640)
                html += self.photoTemplate.format(p['link'],p['type'],dims[0],dims[1])

        #flickr = '1724/42682736792_3c5c710774_z.jpg'
        #href = 'https://farm1.staticflickr.com/' + flickr
        #req = urllib3.request.urlopen('http://answers.opencv.org/upfiles/logo_2.png')
        #http = urllib3.PoolManager()
        #req = http.request('GET', href)
        #image = req.data
        #print(req)
        #print(req.status)
        #print(req.data)
        
        #req = urllib.request.urlopen(href)
        #Image.fromarray(req)
        
        #imgdata = urllib.urlretrieve(href)
        #image_type,width,height = getimageinfo.getImageInfo(imgdata)
        #print('**'+width+'**'+height+'**')
        
        return html
        
    def getAestheticTimes(self):
        html = ''
        #print(self.currentEventRunners)
        for key, data in self.currentEventRunners.items():
            time = data['time']
            if time[-2:] == '00' or time[-2:] == time[0:2]:
                html += time + ' - ' + data['name'] + '<br/>'
            elif time[0] == time[1] and time[3] == time[4]:
                html += time + ' - ' + data['name'] + '<br/>'    
            elif time[0] == time[4] and time[1] == time[2]:
                html += time + ' - ' + data['name'] + '<br/>' 
        
        html = self.getSectionContent(html)
        return html
        
    def getRegularSummary(self, runnerLimit, volunteerLimit):
        events = len(self.runCount)
        regularRunners = {k:v for k,v in self.runners.items() if v['count'] >= runnerLimit}
        runnersList = sorted(regularRunners.items(), key=lambda x: x[1]['name'])  
        
        regularVolunteer = {k:v for k,v in self.volunteers.items() if v >= volunteerLimit}
        volunteerList = sorted(regularVolunteer.items(), key=lambda x: x[0])
        
        text = ''     
        text += self.tableStart + self.tableHeader.format(runnerLimit, events, volunteerLimit, events)
        
        # TODO only need single column from runners and from volunteers
        #volunteerList1 = sorted(list(regularVolunteer.keys()))
        #runnersList1 = [] #regularRunners.items()
        #for v in regularRunners.values():
        #    print(v['name'])
        #    name = v['name']
        #    runnersList1.append[name]
            
        #print(volunteerList1)
        #print("**\n")
        #print(runnersList1)
        #print("**\n")
        
        # work out a better way of transposing two arrays of diff lengths
        rows = {} 
        
        count = 1
        for l,v in runnersList: 
            rows[count] = []
            rows[count].append(self.tableCell.format(v['name']))
            count = count + 1
          
        count = 1
        for key,value in volunteerList: 
            rows[count].append(self.tableCell.format(key))
            count = count + 1 
            
        for key,value in rows.items():
            text += '<tr>'
            for r in value:
                text += r
                if len(value) == 1:
                    text += self.tableCell.format('')
            text += "</tr>\n"
            
        text += self.tableEnd
        
        return text;

class RunReportWeek(RunReport):
    
    runReportHTML = ''
    
    def createWeek(self, weekNumber):
        self.addSummarySection()
        self.addUpcomingSection()
        self.addVolunteerSection()
        
        if weekNumber == 1:
            self.addWeek1Section()
        elif weekNumber == 2:   
            self.addWeek2Section()
        elif weekNumber == 3:  
            self.addWeek3Section()
        elif weekNumber == 4:  
            self.addWeek4Section()
            
        self.addTimesSection()
        self.addPhotoSection()
    
        return self.getFullRunReportHTML()

    def addSummarySection(self):
        text = self.resultsSystemText
        html = self.getSectionHeading('summary','Summary')
        content = '<ul>';
        
        # get text until first .
        content += '<li>'+text[:text.find('.')+1]+'</li>'
        
        # get text from second last . (with white space at start trimmed) 
        pos = text.rfind('.', 0, text.rfind('.'))
        content += '<li>'+text[pos+1:].strip()+'</li>'
        
        content += '</ul>'
        content += 'Thanks to everyone for their participation and we hope to see everyone again next week. Remember to tell your friends parkrun is fun for everyone, it’s the taking part that counts.'
        
        html += self.getSectionContent(content) 
        html += self.getSectionSeparator()
        self.runReportHTML += html
    
    def addUpcomingSection(self):
        html = self.getSectionHeading('upcoming','Upcoming')
        content = '''
<div style="padding: 0px;width: 50%;margin-left: auto;margin-right: auto">
<img src="https://farm1.staticflickr.com/1741/41551984125_2039b19528_z.jpg" alt="runners" width="310" height="148" /></div>
<div>Please Note: Our anniversary is coming up on 23 June and Jells parkrun is turning <b>3</b>. We'll be having a pyjama run. Run in your pjs, dressing gown, bring your teddy bear along. And there will be cake!
<a href="https://www.facebook.com/events/1962473370464471/">Link to Facebook event</a></div>
'''
        html += self.getSectionContent(content)
        html += self.getSectionSeparator()
        self.runReportHTML += html
        
    def addVolunteerSection(self):  
        html = self.getSectionHeading('volunteers','Volunteers')
        content = '''
Jells parkrun relies on volunteers to bring you a free, timed event each week. We encourage everyone to volunteer because it's fun, rewarding, and giving to your community. 
Please review the <a href="https://www.parkrun.com.au/jells/volunteer/futureroster/">Future Roster</a> to see where volunteers are needed and email <a href="mailto:jellshelpers@parkrun.com">jellshelpers@parkrun.com</a> 
if you can help. The wonderful volunteers who made this event possible are:
'''
        content += self.getCurrentEventVolunteerHTML()
        html += self.getSectionContent(content)      
        html += self.getPhotoLinks('volunteer')
        html += self.getSectionSeparator()
        self.runReportHTML += html
    
    def addWeek1Section(self):
        html = self.getSectionHeading('agegroup','Age Group First Finishers')
        html += self.getAgeGroupFinisher()
        self.runReportHTML += html  
        
    def addWeek2Section(self):
        html = self.getSectionHeading('regular','Regular Runners / Volunteers')
        html += self.getRegularSummary(7,2)
        self.runReportHTML += html  
        
    def addWeek3Section(self):
        html = self.getSectionHeading('pbs','Regular PBs')
        html += self.getPBs()
        self.runReportHTML += html
            
    def addWeek4Section(self):
        html = self.getSectionHeading('fun','Having Fun')
        self.runReportHTML += html
        
    def addTimesSection(self):
        html = self.getSectionHeading('times','Aesthetically pleasing times')
        html += self.getAestheticTimes()
        html += self.getSectionSeparator()
        self.runReportHTML += html
            
    def addPhotoSection(self):
        html = self.getSectionHeading('photos','Photos')
        html += self.getPhotoLinks('photo')
        self.runReportHTML += html
        
    def getRunReportHeader(self):
        return '<h1 style="padding: 0;margin: 0">Run Report</h1>'
        
    def getRunReportTOC(self):
        html = '<ul>'
        for toc in self.toc:
           html += toc
        html += '</ul>'
        
        return html    
        
    def getFullRunReportHTML(self):
        html = self.getRunReportHeader()
        html += self.getRunReportTOC()
        html += self.runReportHTML
        
        return html
                
        
run = RunReportWeek()   

In [6]:
# copy text from http://results-service.parkrun.com/resultsSystem/App/eventJournoReportHTML.php?evNum=1153   
text = '''
This week 159 people ran, jogged and walked the course, of whom 29 were first timers and 34 recorded new Personal Bests. Representatives of 11 different clubs took part.

The event was made possible by 11 volunteers:

Ray TERNES • Mark VULLING • Gregory MOORE • Leanne OBERIN • Rosemary WAGHORN • Lizda AHAD • Phillip DANE • Anna DAVIDSON • Jonathan CHEN • Margaret HELLYER • Kyra LAWRENCE

Today's full results and a complete event history can be found on the Jells parkrun Results Page.

The female record is held by Sam PRIME who recorded a time of 18:36 on 4th July 2015 (event number 2).
The male record is held by John DUTTON who recorded a time of 15:36 on 3rd December 2016 (event number 75).
The female record is held by Lauren RYAN who recorded a time of 18:36 on 6th May 2017 (event number 97).
The Age Grade course record is held by Lavinia PETRIE who recorded 100.80% (22:50) on 30th April 2016 (event number 44).

Jells parkrun started on 27th June 2015. Since then 4,258 participants have completed 23,539 parkruns covering a total distance of 117,695 km, including 4,611 new Personal Bests.
'''
run.setResultsSystem(text)

# photos
# addPhoto, second param is width, height from the flickr
# third is the section for the photo to appear - options: summary, volunteer, photo
text = '''
[url=https://flic.kr/p/271GkQ3][img]https://farm2.staticflickr.com/1751/42014704334_e4711a669b_z.jpg[/img][/url][url=https://flic.kr/p/271GkQ3]jells_154_18[/url] by [url=https://www.flickr.com/photos/158549992@N02/]Rose Waghorn[/url], on Flickr
'''
run.addPhoto(text, ['640','427'], 'volunteers')
text = '''
[url=https://flic.kr/p/271GkQ3][img]https://farm2.staticflickr.com/1751/42014704334_e4711a669b_z.jpg[/img][/url][url=https://flic.kr/p/271GkQ3]jells_154_18[/url] by [url=https://www.flickr.com/photos/158549992@N02/]Rose Waghorn[/url], on Flickr
'''
run.addPhoto(text, ['427','640'], 'photo')

#http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=154
text = '''
<table class="sortable" id="results" align="center"><thead><tr><th class="pos">Pos</th><th style="width: 150px">parkrunner</th><th>Time</th><th>Age Cat</th><th>Age Grade</th><th/><th>Gender Pos</th><th style="width: 130px">Club</th><th>Note</th><th colspan="2">Total Runs</th></tr></thead><tbody><tr><td class="pos">1</td><td><a href="athletehistory?athleteNumber=208507" target="_top">John DUTTON</a></td><td>16:54</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>76.73 %</td><td>M</td><td>1</td><td><a href="../clubhistory?clubNum=18075">Box Hill AC</a></td><td>PB stays at 00:15:36</td><td>27</td><td style="min-width:72px"/></tr><tr><td class="pos">2</td><td><a href="athletehistory?athleteNumber=4574923" target="_top">Darren JEFFREYS</a></td><td>18:14</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>71.12 %</td><td>M</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">3</td><td><a href="athletehistory?athleteNumber=3691268" target="_top">Callum MCDONALD</a></td><td>20:13</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>63.81 %</td><td>M</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:18:49</td><td>30</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23436720"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">4</td><td><a href="athletehistory?athleteNumber=4114796" target="_top">Jamie TRUONG</a></td><td>20:26</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>63.21 %</td><td>M</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>8</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/3548245"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">5</td><td><a href="athletehistory?athleteNumber=1356829" target="_top">Evan ROBERTSON</a></td><td>20:48</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>69.79 %</td><td>M</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:19</td><td>64</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">6</td><td><a href="athletehistory?athleteNumber=2051972" target="_top">Jordan TEH</a></td><td>20:52</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>67.97 %</td><td>M</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>23</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">7</td><td><a href="athletehistory?athleteNumber=4162217" target="_top">Daniel CHEN</a></td><td>21:25</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>71.44 %</td><td>M</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">8</td><td><a href="athletehistory?athleteNumber=4547427" target="_top">Sydney SMITH</a></td><td>21:26</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>66.17 %</td><td>M</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:02</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">9</td><td><a href="athletehistory?athleteNumber=1534325" target="_top">Steve WATTS</a></td><td>21:28</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>66.07 %</td><td>M</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>140</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/136051"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">10</td><td><a href="athletehistory?athleteNumber=3808866" target="_top">Will SIMMONS</a></td><td>21:30</td><td><a href="../agecategorytable/?ageCat=SM18-19">SM18-19</a></td><td>61.32 %</td><td>M</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">11</td><td><a href="athletehistory?athleteNumber=760057" target="_top">Danny EDMUNDS</a></td><td>21:34</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>62.83 %</td><td>M</td><td>11</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:21:31</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7633292"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">12</td><td><a href="athletehistory?athleteNumber=4202269" target="_top">Joe KEENAN</a></td><td>22:02</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>61.95 %</td><td>M</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">13</td><td><a href="athletehistory?athleteNumber=4666340" target="_top">Munir AHMAD</a></td><td>22:44</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>67.30 %</td><td>M</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">14</td><td><a href="athletehistory?athleteNumber=2593832" target="_top">Wei-Han KUAN</a></td><td>22:54</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>62.37 %</td><td>M</td><td>14</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:00</td><td>49</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/20106243"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">15</td><td><a href="athletehistory?athleteNumber=4314582" target="_top">Hugh MCDONALD</a></td><td>22:55</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>63.85 %</td><td>M</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">16</td><td><a href="athletehistory?athleteNumber=4165673" target="_top">Matthew COLLINSON</a></td><td>22:59</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>56.13 %</td><td>M</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/26874230"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">17</td><td><a href="athletehistory?athleteNumber=3534437" target="_top">Sarah LUCAS</a></td><td>22:59</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>64.39 %</td><td>F</td><td>1</td><td><a href="../clubhistory?clubNum=18111">Melbourne University AC</a></td><td>First Timer!</td><td>3</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23392136"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">18</td><td><a href="athletehistory?athleteNumber=4139062" target="_top">Simon COSGRAVE</a></td><td>23:13</td><td><a href="../agecategorytable/?ageCat=JM15-17">JM15-17</a></td><td>58.15 %</td><td>M</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">19</td><td><a href="athletehistory?athleteNumber=3095999" target="_top">Michael HUNT</a></td><td>23:15</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>57.06 %</td><td>M</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>47</td><td style="min-width:72px"/></tr><tr><td class="pos">20</td><td><a href="athletehistory?athleteNumber=3951891" target="_top">Alan EDWARD</a></td><td>23:27</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>61.41 %</td><td>M</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">21</td><td><a href="athletehistory?athleteNumber=1065439" target="_top">Michael GOGERLY</a></td><td>23:28</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>58.17 %</td><td>M</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:49</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">22</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">23</td><td><a href="athletehistory?athleteNumber=4534022" target="_top">Travis ROBINSON</a></td><td>23:38</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>54.58 %</td><td>M</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">24</td><td><a href="athletehistory?athleteNumber=4012375" target="_top">Bronwen KUHLMANN</a></td><td>23:39</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>72.66 %</td><td>F</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:29</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">25</td><td><a href="athletehistory?athleteNumber=1345648" target="_top">Chris MCNEILL</a></td><td>23:43</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>57.55 %</td><td>M</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:56</td><td>133</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">26</td><td><a href="athletehistory?athleteNumber=2694977" target="_top">Alison CHAPMAN</a></td><td>23:49</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>71.24 %</td><td>F</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:51</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">27</td><td><a href="athletehistory?athleteNumber=2856359" target="_top">Paulo RAMOS</a></td><td>24:09</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>57.83 %</td><td>M</td><td>24</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:56</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/914269"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">28</td><td><a href="athletehistory?athleteNumber=1762967" target="_top">Tim BOWDEN</a></td><td>24:16</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>53.78 %</td><td>M</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/19227000"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">29</td><td><a href="athletehistory?athleteNumber=2666115" target="_top">David CHUNG</a></td><td>24:19</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>60.66 %</td><td>M</td><td>26</td><td><a href="../clubhistory?clubNum=18165">Victorian Masters Athletics</a></td><td>PB stays at 00:23:16</td><td>29</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/18457333"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">30</td><td><a href="athletehistory?athleteNumber=4070263" target="_top">Philippa ALBRECHT</a></td><td>24:33</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>60.29 %</td><td>F</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">31</td><td><a href="athletehistory?athleteNumber=2276777" target="_top">Adam KOOPS</a></td><td>24:38</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>52.98 %</td><td>M</td><td>27</td><td><a href="../clubhistory?clubNum=23644">Achilles Running Club Melbourne</a></td><td>PB stays at 00:20:15</td><td>88</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">32</td><td><a href="athletehistory?athleteNumber=1721691" target="_top">Kylie WICKERT</a></td><td>24:38</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>62.11 %</td><td>F</td><td>5</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:24:13</td><td>92</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">33</td><td><a href="athletehistory?athleteNumber=2842966" target="_top">Emma JOHNSON</a></td><td>25:15</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>58.61 %</td><td>F</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">34</td><td><a href="athletehistory?athleteNumber=4665835" target="_top">Dillon HETTIARACHCHI</a></td><td>25:19</td><td><a href="../agecategorytable/?ageCat=SM18-19">SM18-19</a></td><td>52.07 %</td><td>M</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">35</td><td><a href="athletehistory?athleteNumber=3558437" target="_top">Cherise NCUBE</a></td><td>25:28</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>63.94 %</td><td>F</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">36</td><td><a href="athletehistory?athleteNumber=3233492" target="_top">Sudhir KUMAR</a></td><td>25:32</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>55.55 %</td><td>M</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:47</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">37</td><td><a href="athletehistory?athleteNumber=852832" target="_top">Bessie CAMPBELL</a></td><td>25:43</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>57.68 %</td><td>F</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>22</td><td style="min-width:72px"/></tr><tr><td class="pos">38</td><td><a href="athletehistory?athleteNumber=2728709" target="_top">Alex STAMA</a></td><td>25:44</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>50.19 %</td><td>M</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:17</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">39</td><td><a href="athletehistory?athleteNumber=1991576" target="_top">Arianna WALLEY</a></td><td>25:46</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>58.67 %</td><td>F</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:30</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">40</td><td><a href="athletehistory?athleteNumber=3798762" target="_top">Georga BRUECHERT</a></td><td>25:49</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>57.33 %</td><td>F</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">41</td><td><a href="athletehistory?athleteNumber=4664549" target="_top">Helen LUCAS</a></td><td>26:04</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>72.38 %</td><td>F</td><td>11</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>First Timer!</td><td>1</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/30263858"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">42</td><td><a href="athletehistory?athleteNumber=3671091" target="_top">Michael WAN</a></td><td>26:07</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>52.65 %</td><td>M</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>22</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23772417"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">43</td><td><a href="athletehistory?athleteNumber=3056576" target="_top">Venkat DODDI</a></td><td>26:11</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>57.22 %</td><td>M</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:53</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">44</td><td><a href="athletehistory?athleteNumber=4664517" target="_top">Michelle PRITCHARD</a></td><td>26:15</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>59.49 %</td><td>F</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">45</td><td><a href="athletehistory?athleteNumber=4008975" target="_top">Cameron GODDARD</a></td><td>26:18</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>60.01 %</td><td>M</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:22</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">46</td><td><a href="athletehistory?athleteNumber=1938315" target="_top">Jason TEH</a></td><td>26:19</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>55.60 %</td><td>M</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:27</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">47</td><td><a href="athletehistory?athleteNumber=1750540" target="_top">Jake NGU</a></td><td>26:25</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>56.72 %</td><td>M</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:10</td><td>123</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11301806"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">48</td><td><a href="athletehistory?athleteNumber=3537860" target="_top">George MANOS</a></td><td>26:28</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>53.59 %</td><td>M</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:49</td><td>28</td><td style="min-width:72px"/></tr><tr><td class="pos">49</td><td><a href="athletehistory?athleteNumber=4114804" target="_top">Nik JAYAWANTHA</a></td><td>26:30</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>53.52 %</td><td>M</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:26</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">50</td><td><a href="athletehistory?athleteNumber=2051977" target="_top">Sarah TEH</a></td><td>26:34</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>61.29 %</td><td>F</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">51</td><td><a href="athletehistory?athleteNumber=898604" target="_top">Andrew PURCELL</a></td><td>26:37</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>52.04 %</td><td>M</td><td>38</td><td><a href="../clubhistory?clubNum=26149">Australia Post Group</a></td><td>PB stays at 00:23:29</td><td>56</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">52</td><td><a href="athletehistory?athleteNumber=2765339" target="_top">Duncan WHITE</a></td><td>26:39</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>55.78 %</td><td>M</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">53</td><td><a href="athletehistory?athleteNumber=3179070" target="_top">Ting CAO</a></td><td>26:42</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>50.00 %</td><td>M</td><td>40</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:21</td><td>24</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5244204"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">54</td><td><a href="athletehistory?athleteNumber=4269797" target="_top">Jessica LU</a></td><td>27:01</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>61.32 %</td><td>F</td><td>14</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:40</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">55</td><td><a href="athletehistory?athleteNumber=4483471" target="_top">Trevor GODDARD</a></td><td>27:04</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>53.20 %</td><td>M</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">56</td><td><a href="athletehistory?athleteNumber=3678564" target="_top">Danni RAMALINGAM</a></td><td>27:24</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>58.52 %</td><td>F</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:12</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">57</td><td><a href="athletehistory?athleteNumber=4656062" target="_top">Sarah SWILKS</a></td><td>27:30</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>54.18 %</td><td>F</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">58</td><td><a href="athletehistory?athleteNumber=4643376" target="_top">Geoff CANNING</a></td><td>27:32</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.51 %</td><td>M</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">59</td><td><a href="athletehistory?athleteNumber=2047120" target="_top">Simon EVANS</a></td><td>27:44</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>56.85 %</td><td>M</td><td>43</td><td><a href="../clubhistory?clubNum=18075">Box Hill AC</a></td><td>PB stays at 00:20:59</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">60</td><td><a href="athletehistory?athleteNumber=4413041" target="_top">Melissa ODENDAAL</a></td><td>27:45</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>53.33 %</td><td>F</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">61</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">62</td><td><a href="athletehistory?athleteNumber=4521874" target="_top">Angel Lu JIAO</a></td><td>27:52</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>58.73 %</td><td>F</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:23</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">63</td><td><a href="athletehistory?athleteNumber=4474975" target="_top">Biondi NATTAATMAJA</a></td><td>27:58</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>46.19 %</td><td>M</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">64</td><td><a href="athletehistory?athleteNumber=1719187" target="_top">Helena BOSHOFF</a></td><td>28:03</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>55.26 %</td><td>F</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:09</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">65</td><td><a href="athletehistory?athleteNumber=4145404" target="_top">Sam KAR</a></td><td>28:06</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>49.70 %</td><td>M</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:06</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">66</td><td><a href="athletehistory?athleteNumber=4114638" target="_top">Wayne PEARCE</a></td><td>28:07</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.44 %</td><td>M</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">67</td><td><a href="athletehistory?athleteNumber=1817638" target="_top">Emma PAPENFUS</a></td><td>28:11</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>57.78 %</td><td>F</td><td>20</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:27:06</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">68</td><td><a href="athletehistory?athleteNumber=1732776" target="_top">Michael PAPENFUS</a></td><td>28:12</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.06 %</td><td>M</td><td>48</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:23:14</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">69</td><td><a href="athletehistory?athleteNumber=1776109" target="_top">Kimberly IMPERIAL</a></td><td>28:24</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>52.11 %</td><td>F</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">70</td><td><a href="athletehistory?athleteNumber=4214936" target="_top">Zhao Hong SU</a></td><td>28:33</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>59.43 %</td><td>F</td><td>22</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">71</td><td><a href="athletehistory?athleteNumber=2395670" target="_top">Wendy SHEN</a></td><td>28:44</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>59.05 %</td><td>F</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:29</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">72</td><td><a href="athletehistory?athleteNumber=4272306" target="_top">Phil TROUGHTON</a></td><td>28:57</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>52.22 %</td><td>M</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">73</td><td><a href="athletehistory?athleteNumber=4114644" target="_top">Hannah PEARCE</a></td><td>29:01</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>55.03 %</td><td>F</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">74</td><td><a href="athletehistory?athleteNumber=2135357" target="_top">Tania STEWART</a></td><td>29:02</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>58.44 %</td><td>F</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">75</td><td><a href="athletehistory?athleteNumber=1785995" target="_top">Julie STAFFERTON</a></td><td>29:04</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>62.27 %</td><td>F</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:31</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">76</td><td><a href="athletehistory?athleteNumber=2848908" target="_top">Jenny PATERSON</a></td><td>29:06</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>54.07 %</td><td>F</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:55</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">77</td><td><a href="athletehistory?athleteNumber=2137630" target="_top">Mark STEWART</a></td><td>29:08</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>51.03 %</td><td>M</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">78</td><td><a href="athletehistory?athleteNumber=4666599" target="_top">Renee MEANEY</a></td><td>29:16</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>50.57 %</td><td>F</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">79</td><td><a href="athletehistory?athleteNumber=4243723" target="_top">Om KAR</a></td><td>29:19</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>49.46 %</td><td>M</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:53</td><td>11</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">80</td><td><a href="athletehistory?athleteNumber=3373087" target="_top">Nino MASCETTA</a></td><td>29:27</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>54.44 %</td><td>M</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:44</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">81</td><td><a href="athletehistory?athleteNumber=4290916" target="_top">Chin LIM</a></td><td>29:49</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>56.79 %</td><td>M</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/9773029"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">82</td><td><a href="athletehistory?athleteNumber=4526607" target="_top">Sam TYRER</a></td><td>30:01</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>43.09 %</td><td>M</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:19</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">83</td><td><a href="athletehistory?athleteNumber=1922729" target="_top">Louisa SOH</a></td><td>30:07</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>52.74 %</td><td>F</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:56</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">84</td><td><a href="athletehistory?athleteNumber=3911794" target="_top">Daniel ZHANG</a></td><td>30:11</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>54.17 %</td><td>M</td><td>55</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:27:35</td><td>24</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">85</td><td><a href="athletehistory?athleteNumber=3044764" target="_top">Susan BOND</a></td><td>30:19</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>49.48 %</td><td>F</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:17</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">86</td><td><a href="athletehistory?athleteNumber=4506751" target="_top">Liwen DAI</a></td><td>30:20</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>45.66 %</td><td>M</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:07</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">87</td><td><a href="athletehistory?athleteNumber=295674" target="_top">Milly DADD</a></td><td>30:26</td><td><a href="../agecategorytable/?ageCat=SW18-19">SW18-19</a></td><td>48.80 %</td><td>F</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>28</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">88</td><td><a href="athletehistory?athleteNumber=3971846" target="_top">Rebecca POSTLETHWAITE</a></td><td>30:30</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>48.52 %</td><td>F</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">89</td><td><a href="athletehistory?athleteNumber=4527635" target="_top">Erin PEARCE</a></td><td>30:42</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>54.18 %</td><td>F</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:27</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">90</td><td><a href="athletehistory?athleteNumber=1786065" target="_top">Tina TIAN</a></td><td>31:01</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>51.69 %</td><td>F</td><td>34</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:27:48</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">91</td><td><a href="athletehistory?athleteNumber=898600" target="_top">Stephanie PURCELL</a></td><td>31:02</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>49.57 %</td><td>F</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:11</td><td>56</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">92</td><td><a href="athletehistory?athleteNumber=1745657" target="_top">Marianne SMITH</a></td><td>31:03</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>57.54 %</td><td>F</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:03</td><td>89</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">93</td><td><a href="athletehistory?athleteNumber=2416833" target="_top">Paul COSGRAVE</a></td><td>31:04</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>48.23 %</td><td>M</td><td>57</td><td><a href="../clubhistory?clubNum=22575">Westerfoldians</a></td><td>PB stays at 00:29:04</td><td>105</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">94</td><td><a href="athletehistory?athleteNumber=4368476" target="_top">Grant BROWN</a></td><td>31:20</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>48.62 %</td><td>M</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:12</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">95</td><td><a href="athletehistory?athleteNumber=4516104" target="_top">Maryanne MCCONNELL</a></td><td>31:30</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>55.98 %</td><td>F</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">96</td><td><a href="athletehistory?athleteNumber=3205754" target="_top">Katherine HOBSON</a></td><td>31:38</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>55.48 %</td><td>F</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>25</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">97</td><td><a href="athletehistory?athleteNumber=3828548" target="_top">Brad HOBSON</a></td><td>31:39</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>41.92 %</td><td>M</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">98</td><td><a href="athletehistory?athleteNumber=4193948" target="_top">Ashley COLEMAN-BOCK</a></td><td>31:41</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>46.71 %</td><td>F</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">99</td><td><a href="athletehistory?athleteNumber=4607007" target="_top">Donna VIDMAR</a></td><td>31:52</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>58.37 %</td><td>F</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/24722361"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">100</td><td><a href="athletehistory?athleteNumber=4643776" target="_top">Vincent LIM</a></td><td>31:55</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>47.36 %</td><td>M</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">101</td><td><a href="athletehistory?athleteNumber=1735852" target="_top">Josephine CARDACI</a></td><td>32:14</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>52.02 %</td><td>F</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:20</td><td>117</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">102</td><td><a href="athletehistory?athleteNumber=1991570" target="_top">David WALLEY</a></td><td>32:37</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>50.49 %</td><td>M</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">103</td><td><a href="athletehistory?athleteNumber=2353796" target="_top">Warren PAGE</a></td><td>32:46</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>45.37 %</td><td>M</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:17</td><td>84</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/24529909"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">104</td><td><a href="athletehistory?athleteNumber=4269919" target="_top">Cathy CANNON</a></td><td>33:00</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>61.41 %</td><td>F</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">105</td><td><a href="athletehistory?athleteNumber=4565046" target="_top">Tamil Selvan VEERAPANDIAN</a></td><td>33:08</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>40.29 %</td><td>M</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7786977"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">106</td><td><a href="athletehistory?athleteNumber=4090905" target="_top">Natasha TRINKLE</a></td><td>33:37</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>45.02 %</td><td>F</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:24</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">107</td><td><a href="athletehistory?athleteNumber=3911796" target="_top">Nathaniel ZHANG</a></td><td>33:44</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>55.43 %</td><td>M</td><td>64</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:31:41</td><td>23</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">108</td><td><a href="athletehistory?athleteNumber=3886143" target="_top">Baoping ZHANG</a></td><td>33:45</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>43.01 %</td><td>M</td><td>65</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:19:53</td><td>26</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5221032"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">109</td><td><a href="athletehistory?athleteNumber=4453453" target="_top">Rochelle WHEATLEY</a></td><td>33:50</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>44.48 %</td><td>F</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">110</td><td><a href="athletehistory?athleteNumber=1783614" target="_top">Rachel SHEPPARD</a></td><td>34:07</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>46.56 %</td><td>F</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:54</td><td>80</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">111</td><td><a href="athletehistory?athleteNumber=2083536" target="_top">Jenny JAMES</a></td><td>34:21</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>58.13 %</td><td>F</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:20</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">112</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">113</td><td><a href="athletehistory?athleteNumber=4078707" target="_top">Kristi LLOYD</a></td><td>34:36</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>43.35 %</td><td>F</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:30</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">114</td><td><a href="athletehistory?athleteNumber=1345664" target="_top">Leonie MCNEILL</a></td><td>35:02</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>43.20 %</td><td>F</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:37</td><td>120</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">115</td><td><a href="athletehistory?athleteNumber=3650435" target="_top">Andrew HEALE</a></td><td>35:07</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>40.06 %</td><td>M</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:46</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">116</td><td><a href="athletehistory?athleteNumber=2334951" target="_top">Nathan WRIGHT</a></td><td>35:13</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>36.63 %</td><td>M</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">117</td><td><a href="athletehistory?athleteNumber=4086945" target="_top">Jerry ZHANG</a></td><td>35:15</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>41.51 %</td><td>M</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:05</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">118</td><td><a href="athletehistory?athleteNumber=4547419" target="_top">Rachel SMITH</a></td><td>35:18</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>48.06 %</td><td>F</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:12</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7243173"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">119</td><td><a href="athletehistory?athleteNumber=3212945" target="_top">Graham ROBERTSON</a></td><td>35:19</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>51.96 %</td><td>M</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:06</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">120</td><td><a href="athletehistory?athleteNumber=4551385" target="_top">Rachel CURTIS</a></td><td>35:28</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>49.72 %</td><td>F</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:09</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">121</td><td><a href="athletehistory?athleteNumber=1344650" target="_top">Clare BLACKSTOCK</a></td><td>36:01</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>44.52 %</td><td>F</td><td>51</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:31:10</td><td>97</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7649007"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">122</td><td><a href="athletehistory?athleteNumber=4666258" target="_top">Jenny MOOLENAAR</a></td><td>36:01</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>43.68 %</td><td>F</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">123</td><td><a href="athletehistory?athleteNumber=4176258" target="_top">Jing YE</a></td><td>36:01</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>41.37 %</td><td>F</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:27</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">124</td><td><a href="athletehistory?athleteNumber=2635904" target="_top">Fleur NIKOLAISON</a></td><td>36:13</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>50.67 %</td><td>F</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:14</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">125</td><td><a href="athletehistory?athleteNumber=3538757" target="_top">Dingaani NCUBE</a></td><td>36:25</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>38.35 %</td><td>M</td><td>71</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:30</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">126</td><td><a href="athletehistory?athleteNumber=4254332" target="_top">Ajantha KUMARI</a></td><td>36:55</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>43.43 %</td><td>F</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:04</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">127</td><td><a href="athletehistory?athleteNumber=3828542" target="_top">Samantha HOBSON</a></td><td>37:05</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>50.61 %</td><td>F</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">128</td><td><a href="athletehistory?athleteNumber=2094156" target="_top">Kelly HOBSON</a></td><td>37:08</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>40.39 %</td><td>F</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>37</td><td style="min-width:72px"/></tr><tr><td class="pos">129</td><td><a href="athletehistory?athleteNumber=4139061" target="_top">Hans VAN DE GRAAFF</a></td><td>37:44</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>41.08 %</td><td>M</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:21</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">130</td><td><a href="athletehistory?athleteNumber=4139067" target="_top">Ella COSGRAVE</a></td><td>37:44</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>45.23 %</td><td>F</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:32</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">131</td><td><a href="athletehistory?athleteNumber=4565049" target="_top">Divya SUKUMAR</a></td><td>37:58</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>39.03 %</td><td>F</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">132</td><td><a href="athletehistory?athleteNumber=4482563" target="_top">Susan SUN</a></td><td>38:13</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>42.83 %</td><td>F</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:16</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">133</td><td><a href="athletehistory?athleteNumber=4425090" target="_top">Melissa HARVEY</a></td><td>38:53</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>42.09 %</td><td>F</td><td>61</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>First Timer!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">134</td><td><a href="athletehistory?athleteNumber=2395087" target="_top">Ashley WILLIS</a></td><td>39:04</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>37.88 %</td><td>F</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/16902477"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">135</td><td><a href="athletehistory?athleteNumber=3971595" target="_top">Sue BROWN</a></td><td>39:15</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>42.72 %</td><td>F</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:38:32</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">136</td><td><a href="athletehistory?athleteNumber=4633063" target="_top">Sarika PATHAK</a></td><td>40:38</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>37.24 %</td><td>F</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">137</td><td><a href="athletehistory?athleteNumber=4363576" target="_top">Ann-Marie JEFFERY</a></td><td>40:43</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>41.67 %</td><td>F</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:33</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">138</td><td><a href="athletehistory?athleteNumber=2663927" target="_top">Tanya KERR</a></td><td>40:52</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>38.87 %</td><td>F</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:41</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">139</td><td><a href="athletehistory?athleteNumber=4628645" target="_top">Fiona HAN</a></td><td>41:11</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>47.43 %</td><td>F</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">140</td><td><a href="athletehistory?athleteNumber=4628642" target="_top">Kim TAI</a></td><td>41:12</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>36.73 %</td><td>F</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">141</td><td><a href="athletehistory?athleteNumber=4225423" target="_top">Richard SLUGGETT</a></td><td>42:22</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>38.87 %</td><td>M</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">142</td><td><a href="athletehistory?athleteNumber=2202224" target="_top">Alex ROBINS</a></td><td>42:23</td><td><a href="../agecategorytable/?ageCat=VM75-79">VM75-79</a></td><td>46.40 %</td><td>M</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:27</td><td>100</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">143</td><td><a href="athletehistory?athleteNumber=3438449" target="_top">Allen PRATT</a></td><td>42:48</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>40.73 %</td><td>M</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:52</td><td>34</td><td style="min-width:72px"/></tr><tr><td class="pos">144</td><td><a href="athletehistory?athleteNumber=4632261" target="_top">Carol ZHANG</a></td><td>43:05</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>38.92 %</td><td>F</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:42:57</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">145</td><td><a href="athletehistory?athleteNumber=3803840" target="_top">Annie CHEN</a></td><td>43:39</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>40.93 %</td><td>F</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:41</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">146</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">147</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">148</td><td><a href="athletehistory?athleteNumber=4515767" target="_top">Kaila SWYER</a></td><td>43:59</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>33.65 %</td><td>F</td><td>71</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:42:28</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">149</td><td><a href="athletehistory?athleteNumber=4425318" target="_top">Shaikh RAHMAN</a></td><td>44:07</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>30.26 %</td><td>M</td><td>78</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:18</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">150</td><td><a href="athletehistory?athleteNumber=4542816" target="_top">Julia LIM</a></td><td>44:16</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>46.50 %</td><td>F</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">151</td><td><a href="athletehistory?athleteNumber=4106933" target="_top">Rosemary SLUGGETT</a></td><td>44:37</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>42.88 %</td><td>F</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">152</td><td><a href="athletehistory?athleteNumber=4574818" target="_top">Pepi GRAY</a></td><td>45:11</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>36.81 %</td><td>F</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:31</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">153</td><td><a href="athletehistory?athleteNumber=2068233" target="_top">Leanne OBERIN</a></td><td>45:24</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>34.99 %</td><td>F</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">154</td><td><a href="athletehistory?athleteNumber=3719799" target="_top">Debra OSMAND</a></td><td>45:24</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>41.56 %</td><td>F</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:42:04</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">155</td><td><a href="athletehistory?athleteNumber=4569079" target="_top">Don DHAMMANARACHCHI</a></td><td>46:08</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>31.72 %</td><td>M</td><td>79</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">156</td><td><a href="athletehistory?athleteNumber=4395237" target="_top">Carol NOBLET</a></td><td>48:29</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>42.45 %</td><td>F</td><td>77</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">157</td><td><a href="athletehistory?athleteNumber=3975800" target="_top">Ken JAMES</a></td><td>49:09</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>36.32 %</td><td>M</td><td>80</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:53</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">158</td><td><a href="athletehistory?athleteNumber=4139058" target="_top">Liz COSGRAVE</a></td><td>49:14</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>33.24 %</td><td>F</td><td>78</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">159</td><td><a href="athletehistory?athleteNumber=2619285" target="_top">Lizda AHAD</a></td><td>1:02:57</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>23.91 %</td><td>F</td><td>79</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:56</td><td>44</td><td style="min-width:72px"/></tr></tbody></table></p><div class="paddedt left"><h3 class="paddedtandb">Thanks to the volunteers</h3><p class="paddedb">We are very grateful to the volunteers who made this event happen:Lizda AHAD, Jonathan CHEN, Phillip DANE, Anna DAVIDSON, Margaret HELLYER, Kyra LAWRENCE, Gregory MOORE, Leanne OBERIN, Ray TERNES, Mark VULLING, Rosemary WAGHORN</p><p class="paddedb">Why not <a href="../../volunteer/">get involved and volunteer</a>?</p><h3 class="paddetandb" id="explainTable">
	What is this table? </h3>
'''
run.setCurrentEvent(text)

run.addRunners(text)
run.addVolunteers(text)

#http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=153
text = '''
<table class="sortable" id="results" align="center"><thead><tr><th class="pos">Pos</th><th style="width: 150px">parkrunner</th><th>Time</th><th>Age Cat</th><th>Age Grade</th><th/><th>Gender Pos</th><th style="width: 130px">Club</th><th>Note</th><th colspan="2">Total Runs</th></tr></thead><tbody><tr><td class="pos">1</td><td><a href="athletehistory?athleteNumber=1171722" target="_top">Julie NORNEY PETRIE</a></td><td>19:57</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>83.04 %</td><td>F</td><td>1</td><td><a href="../clubhistory?clubNum=25648">Team PETRIE</a></td><td>PB stays at 00:19:18</td><td>69</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">2</td><td><a href="athletehistory?athleteNumber=3472907" target="_top">Alec FRANZKE</a></td><td>20:13</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>80.87 %</td><td>M</td><td>1</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>27</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">3</td><td><a href="athletehistory?athleteNumber=1356829" target="_top">Evan ROBERTSON</a></td><td>20:19</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>71.45 %</td><td>M</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:19</td><td>64</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">4</td><td><a href="athletehistory?athleteNumber=4008975" target="_top">Cameron GODDARD</a></td><td>20:35</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>76.68 %</td><td>M</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:22</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">5</td><td><a href="athletehistory?athleteNumber=3988032" target="_top">Tim HOWE</a></td><td>20:55</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>64.78 %</td><td>M</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">6</td><td><a href="athletehistory?athleteNumber=4547427" target="_top">Sydney SMITH</a></td><td>21:03</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>67.38 %</td><td>M</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:02</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">7</td><td><a href="athletehistory?athleteNumber=3851479" target="_top">Sepehr HOSSEINI</a></td><td>21:16</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>60.66 %</td><td>M</td><td>6</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>New PB!</td><td>16</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/13233188"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">8</td><td><a href="athletehistory?athleteNumber=4452235" target="_top">Luke BALCKE</a></td><td>21:31</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>62.97 %</td><td>M</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">9</td><td><a href="athletehistory?athleteNumber=2899338" target="_top">Phillip DANE</a></td><td>21:44</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>72.55 %</td><td>M</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:35</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">10</td><td><a href="athletehistory?athleteNumber=1779095" target="_top">Ray TERNES</a></td><td>21:51</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>60.03 %</td><td>M</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:04</td><td>47</td><td style="min-width:72px"/></tr><tr><td class="pos">11</td><td><a href="athletehistory?athleteNumber=4490875" target="_top">David IRVING</a></td><td>22:44</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>57.99 %</td><td>M</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:15</td><td>5</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23969193"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">12</td><td><a href="athletehistory?athleteNumber=1730739" target="_top">Andrew WILLIAMS</a></td><td>23:06</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>68.25 %</td><td>M</td><td>11</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:22:01</td><td>16</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/10738133"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">13</td><td><a href="athletehistory?athleteNumber=2856359" target="_top">Paulo RAMOS</a></td><td>23:09</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>60.33 %</td><td>M</td><td>12</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:56</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/914269"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">14</td><td><a href="athletehistory?athleteNumber=4314582" target="_top">Hugh MCDONALD</a></td><td>23:11</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>63.12 %</td><td>M</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">15</td><td><a href="athletehistory?athleteNumber=1065439" target="_top">Michael GOGERLY</a></td><td>23:17</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>58.63 %</td><td>M</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:49</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">16</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">17</td><td><a href="athletehistory?athleteNumber=3473017" target="_top">Eddy FRANZKE</a></td><td>23:21</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>76.23 %</td><td>M</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>16</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">18</td><td><a href="athletehistory?athleteNumber=2693212" target="_top">Felicity SOWERBUTTS</a></td><td>23:21</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>63.38 %</td><td>F</td><td>2</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>New PB!</td><td>29</td><td style="min-width:72px"/></tr><tr><td class="pos">19</td><td><a href="athletehistory?athleteNumber=4641726" target="_top">Alex ROEWER</a></td><td>23:22</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>59.27 %</td><td>M</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/3152570"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">20</td><td><a href="athletehistory?athleteNumber=4012375" target="_top">Bronwen KUHLMANN</a></td><td>23:29</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>73.17 %</td><td>F</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">21</td><td><a href="athletehistory?athleteNumber=2221609" target="_top">Megan PEREIRA</a></td><td>23:35</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>67.70 %</td><td>F</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:04</td><td>78</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">22</td><td><a href="athletehistory?athleteNumber=2694977" target="_top">Alison CHAPMAN</a></td><td>23:38</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>71.79 %</td><td>F</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:51</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">23</td><td><a href="athletehistory?athleteNumber=2593832" target="_top">Wei-Han KUAN</a></td><td>23:58</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>59.60 %</td><td>M</td><td>18</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:00</td><td>49</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/20106243"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">24</td><td><a href="athletehistory?athleteNumber=4500454" target="_top">David COE</a></td><td>24:22</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>55.61 %</td><td>M</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">25</td><td><a href="athletehistory?athleteNumber=4571608" target="_top">David RICKETTS</a></td><td>24:28</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>62.81 %</td><td>M</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/14862"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">26</td><td><a href="athletehistory?athleteNumber=1983802" target="_top">Mallory HEALEY</a></td><td>24:33</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>60.29 %</td><td>F</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:52</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">27</td><td><a href="athletehistory?athleteNumber=2288389" target="_top">Sean FERNANDEZ</a></td><td>24:38</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>53.52 %</td><td>M</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">28</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">29</td><td><a href="athletehistory?athleteNumber=2443307" target="_top">Alan TAYLOR</a></td><td>24:47</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>56.76 %</td><td>M</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:17</td><td>58</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">30</td><td><a href="athletehistory?athleteNumber=4641730" target="_top">Elijah ROEWER</a></td><td>25:01</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>57.96 %</td><td>M</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">31</td><td><a href="athletehistory?athleteNumber=1882866" target="_top">Elijah SUMMERS</a></td><td>25:01</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>68.02 %</td><td>M</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>72</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">32</td><td><a href="athletehistory?athleteNumber=3220535" target="_top">Wei HU</a></td><td>25:10</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>53.84 %</td><td>M</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>27</td><td style="min-width:72px"/></tr><tr><td class="pos">33</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">34</td><td><a href="athletehistory?athleteNumber=2728709" target="_top">Alex STAMA</a></td><td>25:35</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>50.49 %</td><td>M</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:17</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">35</td><td><a href="athletehistory?athleteNumber=1915199" target="_top">Wayne JEWELL</a></td><td>25:44</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>59.72 %</td><td>M</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:39</td><td>76</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11496423"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">36</td><td><a href="athletehistory?athleteNumber=4217346" target="_top">Xiwu CHEN</a></td><td>25:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>57.22 %</td><td>M</td><td>30</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:22:59</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">37</td><td><a href="athletehistory?athleteNumber=2371570" target="_top">Luke JOHNSTON</a></td><td>26:06</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>51.92 %</td><td>M</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:43</td><td>12</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/4139302"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">38</td><td><a href="athletehistory?athleteNumber=3671091" target="_top">Michael WAN</a></td><td>26:10</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>52.55 %</td><td>M</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>22</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23772417"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">39</td><td><a href="athletehistory?athleteNumber=2095309" target="_top">Rosemary WAGHORN</a></td><td>26:12</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>60.62 %</td><td>F</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">40</td><td><a href="athletehistory?athleteNumber=3233492" target="_top">Sudhir KUMAR</a></td><td>26:20</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>53.86 %</td><td>M</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:47</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">41</td><td><a href="athletehistory?athleteNumber=2647705" target="_top">Michael BLEWETT</a></td><td>26:23</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>55.91 %</td><td>M</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>34</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/12444211"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">42</td><td><a href="athletehistory?athleteNumber=3886143" target="_top">Baoping ZHANG</a></td><td>26:24</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>54.99 %</td><td>M</td><td>35</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:19:53</td><td>26</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5221032"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">43</td><td><a href="athletehistory?athleteNumber=4114804" target="_top">Nik JAYAWANTHA</a></td><td>26:26</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>53.66 %</td><td>M</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">44</td><td><a href="athletehistory?athleteNumber=3179070" target="_top">Ting CAO</a></td><td>26:32</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>50.31 %</td><td>M</td><td>37</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:21</td><td>24</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5244204"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">45</td><td><a href="athletehistory?athleteNumber=3537860" target="_top">George MANOS</a></td><td>26:35</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>53.35 %</td><td>M</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:49</td><td>28</td><td style="min-width:72px"/></tr><tr><td class="pos">46</td><td><a href="athletehistory?athleteNumber=3080689" target="_top">Gary MURPHY</a></td><td>26:50</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>55.84 %</td><td>M</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>47</td><td style="min-width:72px"/></tr><tr><td class="pos">47</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">48</td><td><a href="athletehistory?athleteNumber=4370064" target="_top">Braden MUNS</a></td><td>26:59</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>48.30 %</td><td>M</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">49</td><td><a href="athletehistory?athleteNumber=4316933" target="_top">Nathaniel PARSONS</a></td><td>27:00</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>47.78 %</td><td>M</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:53</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">50</td><td><a href="athletehistory?athleteNumber=4269797" target="_top">Jessica LU</a></td><td>27:05</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>61.17 %</td><td>F</td><td>8</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:40</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">51</td><td><a href="athletehistory?athleteNumber=4017316" target="_top">Laura WOOLFE</a></td><td>27:07</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>54.58 %</td><td>F</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/26214321"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">52</td><td><a href="athletehistory?athleteNumber=2378428" target="_top">Owen PEREIRA</a></td><td>27:11</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>65.48 %</td><td>M</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:50</td><td>71</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">53</td><td><a href="athletehistory?athleteNumber=2765339" target="_top">Duncan WHITE</a></td><td>27:12</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>54.66 %</td><td>M</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">54</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">55</td><td><a href="athletehistory?athleteNumber=1334399" target="_top">Sue HASTINGS</a></td><td>27:22</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>67.05 %</td><td>F</td><td>10</td><td><a href="../clubhistory?clubNum=19889">Hobson's Bay Running Club</a></td><td>First Timer!</td><td>119</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">56</td><td><a href="athletehistory?athleteNumber=4521874" target="_top">Angel Lu JIAO</a></td><td>27:23</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>59.77 %</td><td>F</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">57</td><td><a href="athletehistory?athleteNumber=3678564" target="_top">Danni RAMALINGAM</a></td><td>27:24</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>58.52 %</td><td>F</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:12</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">58</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">59</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">60</td><td><a href="athletehistory?athleteNumber=3368539" target="_top">Courtney SPLATT</a></td><td>27:44</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>53.61 %</td><td>F</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:53</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">61</td><td><a href="athletehistory?athleteNumber=3217176" target="_top">Martin MCCUSKER</a></td><td>27:49</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.99 %</td><td>M</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:13</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">62</td><td><a href="athletehistory?athleteNumber=1133854" target="_top">Vaughan DE RIGGS</a></td><td>27:59</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>61.64 %</td><td>M</td><td>49</td><td><a href="../clubhistory?clubNum=22580">Pakenham Road Runners</a></td><td>PB stays at 00:24:56</td><td>173</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7542528"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">63</td><td><a href="athletehistory?athleteNumber=4643376" target="_top">Geoff CANNING</a></td><td>28:09</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.38 %</td><td>M</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">64</td><td><a href="athletehistory?athleteNumber=1732776" target="_top">Michael PAPENFUS</a></td><td>28:09</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.15 %</td><td>M</td><td>51</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:23:14</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">65</td><td><a href="athletehistory?athleteNumber=4090888" target="_top">Hans TRINKLE</a></td><td>28:13</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.03 %</td><td>M</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">66</td><td><a href="athletehistory?athleteNumber=1719187" target="_top">Helena BOSHOFF</a></td><td>28:16</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>54.83 %</td><td>F</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:09</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">67</td><td><a href="athletehistory?athleteNumber=2135357" target="_top">Tania STEWART</a></td><td>28:20</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>59.88 %</td><td>F</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">68</td><td><a href="athletehistory?athleteNumber=3470293" target="_top">Claire SMALE</a></td><td>28:22</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>52.53 %</td><td>F</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">69</td><td><a href="athletehistory?athleteNumber=2369510" target="_top">Jackie MITCHELL</a></td><td>28:32</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>61.80 %</td><td>F</td><td>17</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:26:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">70</td><td><a href="athletehistory?athleteNumber=1750540" target="_top">Jake NGU</a></td><td>28:47</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>52.06 %</td><td>M</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:10</td><td>123</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11301806"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">71</td><td><a href="athletehistory?athleteNumber=3913328" target="_top">Jason MCNAMARA</a></td><td>29:04</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>49.54 %</td><td>M</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">72</td><td><a href="athletehistory?athleteNumber=2416833" target="_top">Paul COSGRAVE</a></td><td>29:04</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>51.55 %</td><td>M</td><td>55</td><td><a href="../clubhistory?clubNum=22575">Westerfoldians</a></td><td>New PB!</td><td>105</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">73</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">74</td><td><a href="athletehistory?athleteNumber=2850032" target="_top">Stefan MANNING</a></td><td>29:12</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>46.75 %</td><td>M</td><td>57</td><td><a href="../clubhistory?clubNum=21413">Derek Zoolander Centre for Kids Who Can't Run Good</a></td><td>First Timer!</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">75</td><td><a href="athletehistory?athleteNumber=3373087" target="_top">Nino MASCETTA</a></td><td>29:14</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>54.85 %</td><td>M</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:44</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">76</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">77</td><td><a href="athletehistory?athleteNumber=2168873" target="_top">Caroline HUMPHREYS</a></td><td>29:25</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>50.82 %</td><td>F</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">78</td><td><a href="athletehistory?athleteNumber=2137630" target="_top">Mark STEWART</a></td><td>29:26</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>50.51 %</td><td>M</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">79</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">80</td><td><a href="athletehistory?athleteNumber=2428952" target="_top">Terry RANDALL</a></td><td>29:43</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>53.51 %</td><td>M</td><td>62</td><td><a href="../clubhistory?clubNum=22580">Pakenham Road Runners</a></td><td>PB stays at 00:27:50</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/15367956"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">81</td><td><a href="athletehistory?athleteNumber=4043596" target="_top">Suzanne EVANS</a></td><td>29:58</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>69.74 %</td><td>F</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">82</td><td><a href="athletehistory?athleteNumber=1311284" target="_top">Peter HAMILTON</a></td><td>30:02</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>49.11 %</td><td>M</td><td>63</td><td><a href="../clubhistory?clubNum=23644">Achilles Running Club Melbourne</a></td><td>PB stays at 00:26:20</td><td>145</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7164369"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">83</td><td><a href="athletehistory?athleteNumber=4547419" target="_top">Rachel SMITH</a></td><td>30:12</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>56.18 %</td><td>F</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7243173"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">84</td><td><a href="athletehistory?athleteNumber=3472756" target="_top">Kim FRANZKE</a></td><td>30:14</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>51.27 %</td><td>F</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">85</td><td><a href="athletehistory?athleteNumber=4290916" target="_top">Chin LIM</a></td><td>30:23</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>55.73 %</td><td>M</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:49</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/9773029"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">86</td><td><a href="athletehistory?athleteNumber=3407380" target="_top">Letitia WEBBER</a></td><td>30:29</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>48.55 %</td><td>F</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:26</td><td>11</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22516738"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">87</td><td><a href="athletehistory?athleteNumber=2345651" target="_top">Jackie DOWLING</a></td><td>30:42</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>58.96 %</td><td>F</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:36</td><td>43</td><td style="min-width:72px"/></tr><tr><td class="pos">88</td><td><a href="athletehistory?athleteNumber=1882862" target="_top">Gregory MOORE</a></td><td>30:50</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>46.00 %</td><td>M</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:22</td><td>69</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">89</td><td><a href="athletehistory?athleteNumber=1745657" target="_top">Marianne SMITH</a></td><td>31:26</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>56.84 %</td><td>F</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:03</td><td>89</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">90</td><td><a href="athletehistory?athleteNumber=1834605" target="_top">Marlene HANSFORD</a></td><td>31:28</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>57.52 %</td><td>F</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:24</td><td>71</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">91</td><td><a href="athletehistory?athleteNumber=4516104" target="_top">Maryanne MCCONNELL</a></td><td>31:45</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>55.54 %</td><td>F</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">92</td><td><a href="athletehistory?athleteNumber=4622002" target="_top">Lap CHAN</a></td><td>31:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>42.68 %</td><td>M</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">93</td><td><a href="athletehistory?athleteNumber=4086945" target="_top">Jerry ZHANG</a></td><td>32:05</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>45.61 %</td><td>M</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">94</td><td><a href="athletehistory?athleteNumber=177983" target="_top">Mike GAGE</a></td><td>32:33</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>52.48 %</td><td>M</td><td>68</td><td><a href="../clubhistory?clubNum=18270">Muir of Ord jogscotland</a></td><td>PB stays at 00:27:13</td><td>140</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">95</td><td><a href="athletehistory?athleteNumber=1735852" target="_top">Josephine CARDACI</a></td><td>32:37</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>51.41 %</td><td>F</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:20</td><td>117</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">96</td><td><a href="athletehistory?athleteNumber=4607007" target="_top">Donna VIDMAR</a></td><td>32:38</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>57.00 %</td><td>F</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/24722361"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">97</td><td><a href="athletehistory?athleteNumber=4643776" target="_top">Vincent LIM</a></td><td>32:45</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>46.16 %</td><td>M</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">98</td><td><a href="athletehistory?athleteNumber=3510394" target="_top">Donna KENNY</a></td><td>32:49</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>52.36 %</td><td>F</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:29</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">99</td><td><a href="athletehistory?athleteNumber=3961354" target="_top">Carl CARVALHO</a></td><td>33:05</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>47.66 %</td><td>M</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">100</td><td><a href="athletehistory?athleteNumber=4551385" target="_top">Rachel CURTIS</a></td><td>33:09</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>53.19 %</td><td>F</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">101</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">102</td><td><a href="athletehistory?athleteNumber=4506751" target="_top">Liwen DAI</a></td><td>33:13</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>41.70 %</td><td>M</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:07</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">103</td><td><a href="athletehistory?athleteNumber=2353796" target="_top">Warren PAGE</a></td><td>33:13</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>44.76 %</td><td>M</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:17</td><td>84</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/24529909"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">104</td><td><a href="athletehistory?athleteNumber=2221582" target="_top">Rohini PEREIRA</a></td><td>33:14</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>46.64 %</td><td>F</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:50</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">105</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">106</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">107</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">108</td><td><a href="athletehistory?athleteNumber=4176258" target="_top">Jing YE</a></td><td>33:52</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>43.90 %</td><td>F</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:27</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">109</td><td><a href="athletehistory?athleteNumber=4269919" target="_top">Cathy CANNON</a></td><td>33:59</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>59.64 %</td><td>F</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:00</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">110</td><td><a href="athletehistory?athleteNumber=2769381" target="_top">Marina JORDAN</a></td><td>34:03</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>50.47 %</td><td>F</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:34</td><td>77</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">111</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">112</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">113</td><td><a href="athletehistory?athleteNumber=2748671" target="_top">Robyn WHEATLEY</a></td><td>34:40</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>60.29 %</td><td>F</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">114</td><td><a href="athletehistory?athleteNumber=4453453" target="_top">Rochelle WHEATLEY</a></td><td>34:41</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>43.39 %</td><td>F</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">115</td><td><a href="athletehistory?athleteNumber=1766265" target="_top">Darren JENKINS</a></td><td>34:44</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>39.30 %</td><td>M</td><td>79</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:39</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">116</td><td><a href="athletehistory?athleteNumber=2083536" target="_top">Jenny JAMES</a></td><td>35:19</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>56.54 %</td><td>F</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:20</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">117</td><td><a href="athletehistory?athleteNumber=3093991" target="_top">Michelle TAY</a></td><td>35:21</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>42.57 %</td><td>F</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:02</td><td>48</td><td style="min-width:72px"/></tr><tr><td class="pos">118</td><td><a href="athletehistory?athleteNumber=3212945" target="_top">Graham ROBERTSON</a></td><td>35:25</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>51.81 %</td><td>M</td><td>80</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:06</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">119</td><td><a href="athletehistory?athleteNumber=1745665" target="_top">Katie CROSS</a></td><td>35:38</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>50.80 %</td><td>F</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:09</td><td>61</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">120</td><td><a href="athletehistory?athleteNumber=2428953" target="_top">Janne RANDALL</a></td><td>35:39</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>53.67 %</td><td>F</td><td>40</td><td><a href="../clubhistory?clubNum=22580">Pakenham Road Runners</a></td><td>PB stays at 00:33:11</td><td>81</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">121</td><td><a href="athletehistory?athleteNumber=4254332" target="_top">Ajantha KUMARI</a></td><td>36:04</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>44.45 %</td><td>F</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">122</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">123</td><td><a href="athletehistory?athleteNumber=1927958" target="_top">John KIARIE</a></td><td>36:15</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>39.40 %</td><td>M</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:06</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">124</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">125</td><td><a href="athletehistory?athleteNumber=2207985" target="_top">Andrew WEATHERHEAD</a></td><td>36:39</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>47.57 %</td><td>M</td><td>84</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:16</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">126</td><td><a href="athletehistory?athleteNumber=2397940" target="_top">Jean RUDD</a></td><td>36:54</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>54.92 %</td><td>F</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:21</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">127</td><td><a href="athletehistory?athleteNumber=2950852" target="_top">Barry MOORE</a></td><td>37:10</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>46.41 %</td><td>M</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:55</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">128</td><td><a href="athletehistory?athleteNumber=4600220" target="_top">Ryan GODDARD</a></td><td>37:27</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>47.53 %</td><td>M</td><td>86</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">129</td><td><a href="athletehistory?athleteNumber=2619285" target="_top">Lizda AHAD</a></td><td>37:40</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>39.96 %</td><td>F</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:56</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">130</td><td><a href="athletehistory?athleteNumber=3051474" target="_top">Anthony VAROS</a></td><td>37:44</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>41.43 %</td><td>M</td><td>87</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:37</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">131</td><td><a href="athletehistory?athleteNumber=4483471" target="_top">Trevor GODDARD</a></td><td>37:46</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>38.13 %</td><td>M</td><td>88</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">132</td><td><a href="athletehistory?athleteNumber=3051236" target="_top">Jane TAYLOR</a></td><td>37:57</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>59.64 %</td><td>F</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:37:26</td><td>40</td><td style="min-width:72px"/></tr><tr><td class="pos">133</td><td><a href="athletehistory?athleteNumber=3757996" target="_top">Elizabeth ALDA</a></td><td>37:58</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>45.87 %</td><td>F</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">134</td><td><a href="athletehistory?athleteNumber=3999632" target="_top">Luisa MIRDI</a></td><td>38:37</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>38.50 %</td><td>F</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">135</td><td><a href="athletehistory?athleteNumber=4578742" target="_top">Jessica FERNANDEZ</a></td><td>39:00</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>37.95 %</td><td>F</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">136</td><td><a href="athletehistory?athleteNumber=2828897" target="_top">Meagan EDWARDS</a></td><td>39:10</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>40.55 %</td><td>F</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:59</td><td>41</td><td style="min-width:72px"/></tr><tr><td class="pos">137</td><td><a href="athletehistory?athleteNumber=4606060" target="_top">Kirsty HALL</a></td><td>39:32</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>39.50 %</td><td>F</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">138</td><td><a href="athletehistory?athleteNumber=4482563" target="_top">Susan SUN</a></td><td>39:52</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>41.05 %</td><td>F</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:16</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">139</td><td><a href="athletehistory?athleteNumber=178444" target="_top">Val GAGE</a></td><td>41:05</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>52.45 %</td><td>F</td><td>51</td><td><a href="../clubhistory?clubNum=18270">Muir of Ord jogscotland</a></td><td>PB stays at 00:34:11</td><td>147</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">140</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">141</td><td><a href="athletehistory?athleteNumber=2207989" target="_top">Mary WEATHERHEAD</a></td><td>41:17</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>50.63 %</td><td>F</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">142</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">143</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">144</td><td><a href="athletehistory?athleteNumber=3803840" target="_top">Annie CHEN</a></td><td>41:26</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>43.12 %</td><td>F</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:41</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">145</td><td><a href="athletehistory?athleteNumber=4638499" target="_top">Chase COLLER</a></td><td>41:36</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>44.95 %</td><td>M</td><td>92</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">146</td><td><a href="athletehistory?athleteNumber=3650435" target="_top">Andrew HEALE</a></td><td>41:40</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>33.76 %</td><td>M</td><td>93</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:46</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">147</td><td><a href="athletehistory?athleteNumber=2551399" target="_top">Glenn COLLER</a></td><td>41:41</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>32.51 %</td><td>M</td><td>94</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:19:45</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">148</td><td><a href="athletehistory?athleteNumber=3438449" target="_top">Allen PRATT</a></td><td>42:02</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>41.48 %</td><td>M</td><td>95</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:52</td><td>34</td><td style="min-width:72px"/></tr><tr><td class="pos">149</td><td><a href="athletehistory?athleteNumber=4322666" target="_top">Christine PATERSON</a></td><td>42:10</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>49.57 %</td><td>F</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">150</td><td><a href="athletehistory?athleteNumber=2180571" target="_top">Jennifer HOGARTY</a></td><td>42:19</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>50.93 %</td><td>F</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:40:33</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">151</td><td><a href="athletehistory?athleteNumber=2180623" target="_top">Jacqui CARTER</a></td><td>42:19</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>49.39 %</td><td>F</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:17</td><td>61</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">152</td><td><a href="athletehistory?athleteNumber=4225423" target="_top">Richard SLUGGETT</a></td><td>42:29</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>38.76 %</td><td>M</td><td>96</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">153</td><td><a href="athletehistory?athleteNumber=2202224" target="_top">Alex ROBINS</a></td><td>42:51</td><td><a href="../agecategorytable/?ageCat=VM75-79">VM75-79</a></td><td>45.90 %</td><td>M</td><td>97</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:27</td><td>100</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">154</td><td><a href="athletehistory?athleteNumber=4632261" target="_top">Carol ZHANG</a></td><td>42:57</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>39.04 %</td><td>F</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">155</td><td><a href="athletehistory?athleteNumber=4272564" target="_top">Margaret HELLYER</a></td><td>43:11</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>42.49 %</td><td>F</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">156</td><td><a href="athletehistory?athleteNumber=1716573" target="_top">Nina MUSUMECI</a></td><td>44:00</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>34.96 %</td><td>F</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:23</td><td>87</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">157</td><td><a href="athletehistory?athleteNumber=4542816" target="_top">Julia LIM</a></td><td>44:22</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>46.39 %</td><td>F</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">158</td><td><a href="athletehistory?athleteNumber=4106933" target="_top">Rosemary SLUGGETT</a></td><td>44:43</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>42.79 %</td><td>F</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">159</td><td><a href="athletehistory?athleteNumber=2071200" target="_top">Heather O'CONNOR</a></td><td>44:58</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>47.18 %</td><td>F</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">160</td><td><a href="athletehistory?athleteNumber=1737510" target="_top">Helen PARR</a></td><td>47:26</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>36.72 %</td><td>F</td><td>63</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:29:37</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11621051"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">161</td><td><a href="athletehistory?athleteNumber=1737530" target="_top">Derek PARR</a></td><td>47:27</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>32.95 %</td><td>M</td><td>98</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:27:36</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">162</td><td><a href="athletehistory?athleteNumber=2068233" target="_top">Leanne OBERIN</a></td><td>47:47</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>33.24 %</td><td>F</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:45:24</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">163</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">164</td><td><a href="athletehistory?athleteNumber=3890975" target="_top">Ian O'CONNOR</a></td><td>48:04</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>35.89 %</td><td>M</td><td>100</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:21</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">165</td><td><a href="athletehistory?athleteNumber=2829389" target="_top">Carly MANNING</a></td><td>48:14</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>30.89 %</td><td>F</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">166</td><td><a href="athletehistory?athleteNumber=4633063" target="_top">Sarika PATHAK</a></td><td>49:08</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>30.63 %</td><td>F</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">167</td><td><a href="athletehistory?athleteNumber=3286975" target="_top">Sonia BHATIA</a></td><td>1:02:02</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>27.03 %</td><td>F</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:56:42</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">168</td><td><a href="athletehistory?athleteNumber=4425318" target="_top">Shaikh RAHMAN</a></td><td>1:04:39</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>20.65 %</td><td>M</td><td>101</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:18</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">169</td><td><a href="athletehistory?athleteNumber=1968284" target="_top">Eric NEGROS</a></td><td>1:04:40</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>23.76 %</td><td>M</td><td>102</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:52:13</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">170</td><td><a href="athletehistory?athleteNumber=4037877" target="_top">Sadia NAZIER</a></td><td>1:04:40</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>23.66 %</td><td>F</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:29</td><td>20</td><td style="min-width:72px"/></tr></tbody></table></p><div class="paddedt left"><h3 class="paddedtandb">Thanks to the volunteers</h3><p class="paddedb">We are very grateful to the volunteers who made this event happen:Aaryan BHATIA, Naomi (Tullae) CROTTY, Darren JEFFREYS, Gregory MOORE, Sadia NAZIER, Jenny PATERSON, Elijah SUMMERS, Kaila SWYER, Rosemary WAGHORN, Ashley WILLIS, Nathan WRIGHT</p><p class="paddedb">Why not <a href="../../volunteer/">get involved and volunteer</a>?</p><h3 class="paddetandb" id="explainTable">
	What is this table? </h3>
'''
run.addRunners(text)
run.addVolunteers(text)

#http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=152
text = '''
<table class="sortable" id="results" align="center"><thead><tr><th class="pos">Pos</th><th style="width: 150px">parkrunner</th><th>Time</th><th>Age Cat</th><th>Age Grade</th><th/><th>Gender Pos</th><th style="width: 130px">Club</th><th>Note</th><th colspan="2">Total Runs</th></tr></thead><tbody><tr><td class="pos">1</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">2</td><td><a href="athletehistory?athleteNumber=4475084" target="_top">Joel NG</a></td><td>20:10</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>78.26 %</td><td>M</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">3</td><td><a href="athletehistory?athleteNumber=3447233" target="_top">Sarah WALL</a></td><td>20:25</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>75.92 %</td><td>F</td><td>1</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:14</td><td>32</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">4</td><td><a href="athletehistory?athleteNumber=4404027" target="_top">Alex GITLITS</a></td><td>20:57</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>66.11 %</td><td>M</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">5</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">6</td><td><a href="athletehistory?athleteNumber=4475081" target="_top">Jayden NG</a></td><td>21:25</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>73.70 %</td><td>M</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:04</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">7</td><td><a href="athletehistory?athleteNumber=1294660" target="_top">June PETRIE</a></td><td>21:33</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>81.83 %</td><td>F</td><td>2</td><td><a href="../clubhistory?clubNum=25648">Team PETRIE</a></td><td>PB stays at 00:20:33</td><td>100</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">8</td><td><a href="athletehistory?athleteNumber=2899338" target="_top">Phillip DANE</a></td><td>21:35</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>73.05 %</td><td>M</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">9</td><td><a href="athletehistory?athleteNumber=1701126" target="_top">Brendan HILL</a></td><td>21:50</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>72.21 %</td><td>M</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>37</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">10</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">11</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">12</td><td><a href="athletehistory?athleteNumber=1434738" target="_top">Zoe DOWLING</a></td><td>22:14</td><td><a href="../agecategorytable/?ageCat=SW18-19">SW18-19</a></td><td>66.79 %</td><td>F</td><td>3</td><td><a href="../clubhistory?clubNum=18085">Eastern Suburbs AC</a></td><td>New PB!</td><td>18</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">13</td><td><a href="athletehistory?athleteNumber=4490875" target="_top">David IRVING</a></td><td>22:15</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>59.25 %</td><td>M</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23969193"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">14</td><td><a href="athletehistory?athleteNumber=1556785" target="_top">Brett MUGRIDGE</a></td><td>22:43</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>56.86 %</td><td>M</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>46</td><td style="min-width:72px"/></tr><tr><td class="pos">15</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">16</td><td><a href="athletehistory?athleteNumber=1927958" target="_top">John KIARIE</a></td><td>22:55</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>62.33 %</td><td>M</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:06</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">17</td><td><a href="athletehistory?athleteNumber=2856359" target="_top">Paulo RAMOS</a></td><td>22:56</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>60.90 %</td><td>M</td><td>14</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>New PB!</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/914269"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">18</td><td><a href="athletehistory?athleteNumber=4147617" target="_top">Rebekah POWIERSKI</a></td><td>23:00</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>67.39 %</td><td>F</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>10</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">19</td><td><a href="athletehistory?athleteNumber=3095999" target="_top">Michael HUNT</a></td><td>23:15</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>57.06 %</td><td>M</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>47</td><td style="min-width:72px"/></tr><tr><td class="pos">20</td><td><a href="athletehistory?athleteNumber=4162217" target="_top">Daniel CHEN</a></td><td>23:26</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>65.29 %</td><td>M</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:25</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">21</td><td><a href="athletehistory?athleteNumber=3646698" target="_top">Ashley HUSTWAYTE</a></td><td>23:30</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>55.18 %</td><td>M</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">22</td><td><a href="athletehistory?athleteNumber=4314582" target="_top">Hugh MCDONALD</a></td><td>23:31</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>62.23 %</td><td>M</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:55</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">23</td><td><a href="athletehistory?athleteNumber=4053891" target="_top">Jason CLEVELEY</a></td><td>23:49</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>62.42 %</td><td>M</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">24</td><td><a href="athletehistory?athleteNumber=2221609" target="_top">Megan PEREIRA</a></td><td>23:54</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>66.81 %</td><td>F</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:04</td><td>78</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">25</td><td><a href="athletehistory?athleteNumber=4210599" target="_top">Elliana SALTALAMACCHIA</a></td><td>23:55</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>61.88 %</td><td>F</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:42</td><td>13</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/13341061"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">26</td><td><a href="athletehistory?athleteNumber=4012375" target="_top">Bronwen KUHLMANN</a></td><td>24:01</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>71.55 %</td><td>F</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">27</td><td><a href="athletehistory?athleteNumber=1406388" target="_top">Sarah TRAINER</a></td><td>24:18</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>60.91 %</td><td>F</td><td>8</td><td><a href="../clubhistory?clubNum=23716">TEAM Tri Coaching</a></td><td>First Timer!</td><td>43</td><td style="min-width:72px"/></tr><tr><td class="pos">28</td><td><a href="athletehistory?athleteNumber=3205411" target="_top">Ian SKELTON</a></td><td>24:23</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>58.17 %</td><td>M</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:07</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">29</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">30</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">31</td><td><a href="athletehistory?athleteNumber=2276777" target="_top">Adam KOOPS</a></td><td>24:44</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>52.76 %</td><td>M</td><td>23</td><td><a href="../clubhistory?clubNum=23644">Achilles Running Club Melbourne</a></td><td>PB stays at 00:20:15</td><td>88</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">32</td><td><a href="athletehistory?athleteNumber=2927404" target="_top">Simon REYMERS</a></td><td>24:44</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>53.64 %</td><td>M</td><td>24</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:19:19</td><td>40</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/18022867"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">33</td><td><a href="athletehistory?athleteNumber=2589482" target="_top">Dirk HERAT</a></td><td>24:47</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>59.05 %</td><td>M</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">34</td><td><a href="athletehistory?athleteNumber=4475086" target="_top">Joshua NG</a></td><td>25:08</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>65.05 %</td><td>M</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">35</td><td><a href="athletehistory?athleteNumber=3671091" target="_top">Michael WAN</a></td><td>25:24</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>54.13 %</td><td>M</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>22</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23772417"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">36</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">37</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">38</td><td><a href="athletehistory?athleteNumber=3447301" target="_top">Arthur WALL</a></td><td>25:45</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>57.73 %</td><td>M</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:59</td><td>44</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/29599152"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">39</td><td><a href="athletehistory?athleteNumber=898604" target="_top">Andrew PURCELL</a></td><td>25:49</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>53.65 %</td><td>M</td><td>31</td><td><a href="../clubhistory?clubNum=26149">Australia Post Group</a></td><td>PB stays at 00:23:29</td><td>56</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">40</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">41</td><td><a href="athletehistory?athleteNumber=562428" target="_top">Peter CRAIG</a></td><td>25:54</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>61.39 %</td><td>M</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:43</td><td>194</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">42</td><td><a href="athletehistory?athleteNumber=3828690" target="_top">Ray NG</a></td><td>25:56</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>49.87 %</td><td>M</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:35</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">43</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">44</td><td><a href="athletehistory?athleteNumber=2336000" target="_top">Peter YEOMAN</a></td><td>26:19</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>50.41 %</td><td>M</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">45</td><td><a href="athletehistory?athleteNumber=2647705" target="_top">Michael BLEWETT</a></td><td>26:31</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>55.63 %</td><td>M</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>34</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/12444211"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">46</td><td><a href="athletehistory?athleteNumber=3903734" target="_top">Andrew CHEN</a></td><td>26:41</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>55.72 %</td><td>M</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>15</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">47</td><td><a href="athletehistory?athleteNumber=4316933" target="_top">Nathaniel PARSONS</a></td><td>26:53</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>47.99 %</td><td>M</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">48</td><td><a href="athletehistory?athleteNumber=4342106" target="_top">Steven BEASLEY</a></td><td>27:04</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>47.91 %</td><td>M</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:19</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22624622"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">49</td><td><a href="athletehistory?athleteNumber=4017316" target="_top">Laura WOOLFE</a></td><td>27:12</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>54.41 %</td><td>F</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/26214321"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">50</td><td><a href="athletehistory?athleteNumber=1750540" target="_top">Jake NGU</a></td><td>27:16</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>54.95 %</td><td>M</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:10</td><td>123</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11301806"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">51</td><td><a href="athletehistory?athleteNumber=2765339" target="_top">Duncan WHITE</a></td><td>27:22</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>54.32 %</td><td>M</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">52</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">53</td><td><a href="athletehistory?athleteNumber=1719187" target="_top">Helena BOSHOFF</a></td><td>27:41</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>55.99 %</td><td>F</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:09</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">54</td><td><a href="athletehistory?athleteNumber=3780185" target="_top">James PICKERING</a></td><td>27:47</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>52.67 %</td><td>M</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">55</td><td><a href="athletehistory?athleteNumber=4114804" target="_top">Nik JAYAWANTHA</a></td><td>28:04</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.53 %</td><td>M</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:26</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">56</td><td><a href="athletehistory?athleteNumber=2285678" target="_top">Nicki WHITFIELD</a></td><td>28:06</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>60.38 %</td><td>F</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">57</td><td><a href="athletehistory?athleteNumber=2223986" target="_top">Emily PEREIRA</a></td><td>28:14</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>60.45 %</td><td>F</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">58</td><td><a href="athletehistory?athleteNumber=3519429" target="_top">Biljana ZJAJIC</a></td><td>28:20</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>52.29 %</td><td>F</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:04</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">59</td><td><a href="athletehistory?athleteNumber=2378428" target="_top">Owen PEREIRA</a></td><td>28:21</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>62.79 %</td><td>M</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:50</td><td>71</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">60</td><td><a href="athletehistory?athleteNumber=4090888" target="_top">Hans TRINKLE</a></td><td>28:22</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.76 %</td><td>M</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:13</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">61</td><td><a href="athletehistory?athleteNumber=3373087" target="_top">Nino MASCETTA</a></td><td>28:22</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>56.52 %</td><td>M</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:44</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">62</td><td><a href="athletehistory?athleteNumber=1739258" target="_top">Madison JOY</a></td><td>28:29</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>58.40 %</td><td>F</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:49</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">63</td><td><a href="athletehistory?athleteNumber=1739102" target="_top">Raymond JOY</a></td><td>28:31</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.91 %</td><td>M</td><td>49</td><td><a href="../clubhistory?clubNum=19990">Collingwood Harriers</a></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">64</td><td><a href="athletehistory?athleteNumber=4269724" target="_top">Jordan BROWN</a></td><td>28:33</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>45.18 %</td><td>M</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">65</td><td><a href="athletehistory?athleteNumber=4521874" target="_top">Angel Lu JIAO</a></td><td>28:34</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>57.29 %</td><td>F</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">66</td><td><a href="athletehistory?athleteNumber=3233492" target="_top">Sudhir KUMAR</a></td><td>28:35</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>49.62 %</td><td>M</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:47</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">67</td><td><a href="athletehistory?athleteNumber=2135357" target="_top">Tania STEWART</a></td><td>28:37</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>59.29 %</td><td>F</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">68</td><td><a href="athletehistory?athleteNumber=2848908" target="_top">Jenny PATERSON</a></td><td>28:55</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>54.41 %</td><td>F</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">69</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">70</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">71</td><td><a href="athletehistory?athleteNumber=3407380" target="_top">Letitia WEBBER</a></td><td>29:15</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>50.60 %</td><td>F</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:26</td><td>11</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22516738"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">72</td><td><a href="athletehistory?athleteNumber=2952261" target="_top">Lian TUNKS</a></td><td>29:26</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>61.49 %</td><td>F</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">73</td><td><a href="athletehistory?athleteNumber=1767271" target="_top">Reid GERRARD</a></td><td>29:33</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>52.90 %</td><td>M</td><td>54</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:52</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">74</td><td><a href="athletehistory?athleteNumber=2395087" target="_top">Ashley WILLIS</a></td><td>29:46</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>49.72 %</td><td>F</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/16902477"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">75</td><td><a href="athletehistory?athleteNumber=2599590" target="_top">Peter SELL</a></td><td>30:02</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>57.44 %</td><td>M</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:06</td><td>86</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">76</td><td><a href="athletehistory?athleteNumber=4116148" target="_top">Lyn BEARD</a></td><td>30:03</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>59.46 %</td><td>F</td><td>21</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:27:31</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">77</td><td><a href="athletehistory?athleteNumber=4043596" target="_top">Suzanne EVANS</a></td><td>30:06</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>69.44 %</td><td>F</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:58</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">78</td><td><a href="athletehistory?athleteNumber=2345651" target="_top">Jackie DOWLING</a></td><td>30:36</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>59.15 %</td><td>F</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>43</td><td style="min-width:72px"/></tr><tr><td class="pos">79</td><td><a href="athletehistory?athleteNumber=3904164" target="_top">Jonathan CHEN</a></td><td>30:38</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>51.52 %</td><td>M</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:11</td><td>18</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">80</td><td><a href="athletehistory?athleteNumber=4217346" target="_top">Xiwu CHEN</a></td><td>30:39</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>48.50 %</td><td>M</td><td>57</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:22:59</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">81</td><td><a href="athletehistory?athleteNumber=2988047" target="_top">Anna DAVIDSON</a></td><td>30:39</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>49.37 %</td><td>F</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:39</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">82</td><td><a href="athletehistory?athleteNumber=2395670" target="_top">Wendy SHEN</a></td><td>30:55</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>54.88 %</td><td>F</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:29</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">83</td><td><a href="athletehistory?athleteNumber=241262" target="_top">Brianna KING</a></td><td>31:19</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>47.26 %</td><td>F</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">84</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">85</td><td><a href="athletehistory?athleteNumber=1745657" target="_top">Marianne SMITH</a></td><td>31:26</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>56.84 %</td><td>F</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:03</td><td>89</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">86</td><td><a href="athletehistory?athleteNumber=898600" target="_top">Stephanie PURCELL</a></td><td>31:37</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>48.66 %</td><td>F</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:11</td><td>56</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">87</td><td><a href="athletehistory?athleteNumber=3985226" target="_top">Joy YIN</a></td><td>31:41</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>55.39 %</td><td>F</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">88</td><td><a href="athletehistory?athleteNumber=4575204" target="_top">Kristina CUI</a></td><td>31:42</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>50.11 %</td><td>F</td><td>30</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">89</td><td><a href="athletehistory?athleteNumber=1721534" target="_top">Susie WILSON</a></td><td>31:47</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>47.82 %</td><td>F</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:17</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">90</td><td><a href="athletehistory?athleteNumber=1721691" target="_top">Kylie WICKERT</a></td><td>31:48</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>48.11 %</td><td>F</td><td>32</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:24:13</td><td>92</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">91</td><td><a href="athletehistory?athleteNumber=4506751" target="_top">Liwen DAI</a></td><td>31:50</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>43.51 %</td><td>M</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:07</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">92</td><td><a href="athletehistory?athleteNumber=4547419" target="_top">Rachel SMITH</a></td><td>31:50</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>53.30 %</td><td>F</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:12</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7243173"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">93</td><td><a href="athletehistory?athleteNumber=1786065" target="_top">Tina TIAN</a></td><td>31:51</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>50.34 %</td><td>F</td><td>34</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:27:48</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">94</td><td><a href="athletehistory?athleteNumber=2137630" target="_top">Mark STEWART</a></td><td>31:57</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>46.53 %</td><td>M</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">95</td><td><a href="athletehistory?athleteNumber=4277072" target="_top">Jane HARRIS</a></td><td>32:08</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.90 %</td><td>F</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">96</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">97</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">98</td><td><a href="athletehistory?athleteNumber=2187279" target="_top">Nanci MOORE</a></td><td>32:20</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>59.18 %</td><td>F</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:59</td><td>51</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">99</td><td><a href="athletehistory?athleteNumber=3510394" target="_top">Donna KENNY</a></td><td>32:29</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>52.90 %</td><td>F</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">100</td><td><a href="athletehistory?athleteNumber=1783614" target="_top">Rachel SHEPPARD</a></td><td>32:46</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>48.47 %</td><td>F</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:54</td><td>80</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">101</td><td><a href="athletehistory?athleteNumber=3375523" target="_top">Flynn BARTLETT</a></td><td>32:54</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>47.97 %</td><td>M</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:06</td><td>33</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">102</td><td><a href="athletehistory?athleteNumber=3373279" target="_top">Peter BARTLETT</a></td><td>32:54</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>44.48 %</td><td>M</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:07</td><td>32</td><td style="min-width:72px"/></tr><tr><td class="pos">103</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">104</td><td><a href="athletehistory?athleteNumber=3970628" target="_top">Marina LI</a></td><td>33:06</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.45 %</td><td>F</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:27</td><td>27</td><td style="min-width:72px"/></tr><tr><td class="pos">105</td><td><a href="athletehistory?athleteNumber=4309711" target="_top">Perrin BROWN</a></td><td>33:10</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>39.20 %</td><td>M</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:51</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">106</td><td><a href="athletehistory?athleteNumber=4516104" target="_top">Maryanne MCCONNELL</a></td><td>33:14</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>53.06 %</td><td>F</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">107</td><td><a href="athletehistory?athleteNumber=2299274" target="_top">Dani BOLTON</a></td><td>33:15</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.22 %</td><td>F</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:01</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">108</td><td><a href="athletehistory?athleteNumber=4269797" target="_top">Jessica LU</a></td><td>33:16</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.80 %</td><td>F</td><td>42</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:40</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">109</td><td><a href="athletehistory?athleteNumber=4607007" target="_top">Donna VIDMAR</a></td><td>33:18</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>55.86 %</td><td>F</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/24722361"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">110</td><td><a href="athletehistory?athleteNumber=2083536" target="_top">Jenny JAMES</a></td><td>33:20</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>59.90 %</td><td>F</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">111</td><td><a href="athletehistory?athleteNumber=2299278" target="_top">Lachie RICHTER</a></td><td>33:21</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>43.48 %</td><td>M</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:57</td><td>20</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">112</td><td><a href="athletehistory?athleteNumber=4269919" target="_top">Cathy CANNON</a></td><td>33:23</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>60.71 %</td><td>F</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">113</td><td><a href="athletehistory?athleteNumber=4370064" target="_top">Braden MUNS</a></td><td>33:25</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>39.00 %</td><td>M</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:59</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">114</td><td><a href="athletehistory?athleteNumber=1748710" target="_top">Matthew RUSHTON</a></td><td>33:29</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>41.07 %</td><td>M</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:08</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">115</td><td><a href="athletehistory?athleteNumber=2221582" target="_top">Rohini PEREIRA</a></td><td>33:41</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>46.02 %</td><td>F</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:50</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">116</td><td><a href="athletehistory?athleteNumber=1344947" target="_top">Shawn DUFF</a></td><td>33:42</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>40.80 %</td><td>M</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">117</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">118</td><td><a href="athletehistory?athleteNumber=2135388" target="_top">Stephen WHITE</a></td><td>33:47</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>45.49 %</td><td>M</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:29</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">119</td><td><a href="athletehistory?athleteNumber=2657927" target="_top">Penny STEVENSON</a></td><td>33:51</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>44.71 %</td><td>F</td><td>47</td><td><a href="../clubhistory?clubNum=23644">Achilles Running Club Melbourne</a></td><td>First Timer!</td><td>19</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/19079045"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">120</td><td><a href="athletehistory?athleteNumber=1562450" target="_top">Sarah HERAT</a></td><td>33:51</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>47.37 %</td><td>F</td><td>48</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:25:02</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">121</td><td><a href="athletehistory?athleteNumber=2374136" target="_top">Regina NEVILLE</a></td><td>33:53</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>46.09 %</td><td>F</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:26</td><td>62</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">122</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">123</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">124</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">125</td><td><a href="athletehistory?athleteNumber=4086945" target="_top">Jerry ZHANG</a></td><td>34:30</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>42.42 %</td><td>M</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:05</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">126</td><td><a href="athletehistory?athleteNumber=3226535" target="_top">Daniel STRICKLAND</a></td><td>34:34</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>39.49 %</td><td>M</td><td>77</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:36</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">127</td><td><a href="athletehistory?athleteNumber=1766265" target="_top">Darren JENKINS</a></td><td>34:41</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>39.36 %</td><td>M</td><td>78</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:39</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">128</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">129</td><td><a href="athletehistory?athleteNumber=4622002" target="_top">Lap CHAN</a></td><td>34:45</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>39.28 %</td><td>M</td><td>80</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">130</td><td><a href="athletehistory?athleteNumber=4176258" target="_top">Jing YE</a></td><td>34:49</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>42.70 %</td><td>F</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:27</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">131</td><td><a href="athletehistory?athleteNumber=3093991" target="_top">Michelle TAY</a></td><td>35:02</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>42.96 %</td><td>F</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>48</td><td style="min-width:72px"/></tr><tr><td class="pos">132</td><td><a href="athletehistory?athleteNumber=3212945" target="_top">Graham ROBERTSON</a></td><td>35:06</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>52.28 %</td><td>M</td><td>81</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">133</td><td><a href="athletehistory?athleteNumber=4453453" target="_top">Rochelle WHEATLEY</a></td><td>35:55</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>41.90 %</td><td>F</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">134</td><td><a href="athletehistory?athleteNumber=4475078" target="_top">Freddy NG</a></td><td>36:06</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>38.09 %</td><td>M</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">135</td><td><a href="athletehistory?athleteNumber=1344650" target="_top">Clare BLACKSTOCK</a></td><td>36:11</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>44.31 %</td><td>F</td><td>53</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:31:10</td><td>97</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7649007"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">136</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">137</td><td><a href="athletehistory?athleteNumber=3051474" target="_top">Anthony VAROS</a></td><td>36:40</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>42.64 %</td><td>M</td><td>84</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:37</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">138</td><td><a href="athletehistory?athleteNumber=4574997" target="_top">Gerald MCDORNAN</a></td><td>36:45</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>41.13 %</td><td>M</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/8316973"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">139</td><td><a href="athletehistory?athleteNumber=4254332" target="_top">Ajantha KUMARI</a></td><td>36:49</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>43.55 %</td><td>F</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">140</td><td><a href="athletehistory?athleteNumber=4358918" target="_top">Lais MATOS</a></td><td>37:23</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>39.59 %</td><td>F</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:29</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">141</td><td><a href="athletehistory?athleteNumber=4008975" target="_top">Cameron GODDARD</a></td><td>37:48</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>41.75 %</td><td>M</td><td>86</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:22</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">142</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">143</td><td><a href="athletehistory?athleteNumber=4311721" target="_top">Karuna BHODAY</a></td><td>38:18</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>43.78 %</td><td>F</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:08</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">144</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">145</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">146</td><td><a href="athletehistory?athleteNumber=3179070" target="_top">Ting CAO</a></td><td>39:29</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>33.81 %</td><td>M</td><td>90</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:21</td><td>24</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5244204"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">147</td><td><a href="athletehistory?athleteNumber=4363576" target="_top">Ann-Marie JEFFERY</a></td><td>39:33</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>42.90 %</td><td>F</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">148</td><td><a href="athletehistory?athleteNumber=2663927" target="_top">Tanya KERR</a></td><td>39:41</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>40.03 %</td><td>F</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">149</td><td><a href="athletehistory?athleteNumber=2777633" target="_top">Nathan SCOTT</a></td><td>39:49</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>39.64 %</td><td>M</td><td>91</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:53</td><td>66</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">150</td><td><a href="athletehistory?athleteNumber=2777601" target="_top">Christine KAUCNER</a></td><td>39:50</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>41.09 %</td><td>F</td><td>59</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:26:16</td><td>76</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/12533560"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">151</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">152</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">153</td><td><a href="athletehistory?athleteNumber=2935289" target="_top">Suraj JOSE</a></td><td>40:09</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>32.21 %</td><td>M</td><td>94</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:26</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">154</td><td><a href="athletehistory?athleteNumber=2022173" target="_top">Michelle WOODMAN</a></td><td>40:38</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>39.46 %</td><td>F</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>102</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/17474260"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">155</td><td><a href="athletehistory?athleteNumber=4272066" target="_top">Danielle SALTALAMACCHIA</a></td><td>41:25</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>39.52 %</td><td>F</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">156</td><td><a href="athletehistory?athleteNumber=656410" target="_top">Naomi (Tullae) CROTTY</a></td><td>41:25</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>36.54 %</td><td>F</td><td>62</td><td><a href="../clubhistory?clubNum=20982">Wooters Runners</a></td><td>PB stays at 00:39:11</td><td>146</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/25440878"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">157</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">158</td><td><a href="athletehistory?athleteNumber=4612068" target="_top">Joshua WARDROBE</a></td><td>41:46</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>36.63 %</td><td>M</td><td>96</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">159</td><td><a href="athletehistory?athleteNumber=1731070" target="_top">Katie WARDROBE</a></td><td>41:47</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>38.37 %</td><td>F</td><td>63</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:24:41</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">160</td><td><a href="athletehistory?athleteNumber=2777655" target="_top">Toby SCOTT</a></td><td>42:12</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>40.32 %</td><td>M</td><td>97</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:22</td><td>68</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/25563106"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">161</td><td><a href="athletehistory?athleteNumber=2777641" target="_top">Andrew SCOTT</a></td><td>42:19</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>32.49 %</td><td>M</td><td>98</td><td><a href="../clubhistory?clubNum=21413">Derek Zoolander Centre for Kids Who Can't Run Good</a></td><td>PB stays at 00:24:49</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/199341"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">162</td><td><a href="athletehistory?athleteNumber=4600220" target="_top">Ryan GODDARD</a></td><td>42:21</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>42.03 %</td><td>M</td><td>99</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">163</td><td><a href="athletehistory?athleteNumber=4114345" target="_top">Marilyn STEVENSON</a></td><td>42:25</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>54.26 %</td><td>F</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">164</td><td><a href="athletehistory?athleteNumber=4251212" target="_top">Louis WYCISK</a></td><td>42:32</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>34.95 %</td><td>M</td><td>100</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:20</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">165</td><td><a href="athletehistory?athleteNumber=4451227" target="_top">Emily CRAWFORD</a></td><td>42:52</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>36.43 %</td><td>F</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">166</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">167</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">168</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">169</td><td><a href="athletehistory?athleteNumber=4272564" target="_top">Margaret HELLYER</a></td><td>43:20</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>42.35 %</td><td>F</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">170</td><td><a href="athletehistory?athleteNumber=3438449" target="_top">Allen PRATT</a></td><td>43:30</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>40.08 %</td><td>M</td><td>104</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:52</td><td>34</td><td style="min-width:72px"/></tr><tr><td class="pos">171</td><td><a href="athletehistory?athleteNumber=4225423" target="_top">Richard SLUGGETT</a></td><td>44:16</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>37.20 %</td><td>M</td><td>105</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:42:22</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">172</td><td><a href="athletehistory?athleteNumber=1792850" target="_top">Lisa BULLOCK</a></td><td>45:10</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>33.87 %</td><td>F</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:17</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">173</td><td><a href="athletehistory?athleteNumber=4106933" target="_top">Rosemary SLUGGETT</a></td><td>45:23</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>42.16 %</td><td>F</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">174</td><td><a href="athletehistory?athleteNumber=4606060" target="_top">Kirsty HALL</a></td><td>45:35</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>34.26 %</td><td>F</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">175</td><td><a href="athletehistory?athleteNumber=3314054" target="_top">Ken MATHERS</a></td><td>45:38</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>39.63 %</td><td>M</td><td>106</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:43:42</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">176</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">177</td><td><a href="athletehistory?athleteNumber=2599612" target="_top">Janet FREIBERG</a></td><td>48:40</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>36.23 %</td><td>F</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>87</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">178</td><td><a href="athletehistory?athleteNumber=2599585" target="_top">Margaret SELL</a></td><td>48:41</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>42.93 %</td><td>F</td><td>71</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>85</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">179</td><td><a href="athletehistory?athleteNumber=4037905" target="_top">Denzil CUPIDO</a></td><td>50:50</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>26.66 %</td><td>M</td><td>108</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:04</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">180</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">181</td><td><a href="athletehistory?athleteNumber=3591013" target="_top">Robyn DIMOPOULOS</a></td><td>53:59</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>27.60 %</td><td>F</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:13</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">182</td><td><a href="athletehistory?athleteNumber=4037877" target="_top">Sadia NAZIER</a></td><td>54:04</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>28.30 %</td><td>F</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:29</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">183</td><td><a href="athletehistory?athleteNumber=2559133" target="_top">Jude VULLING</a></td><td>54:09</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>26.19 %</td><td>M</td><td>110</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:52:03</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">184</td><td><a href="athletehistory?athleteNumber=1864696" target="_top">Mark VULLING</a></td><td>54:12</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>26.78 %</td><td>M</td><td>111</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:41</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">185</td><td><a href="athletehistory?athleteNumber=3357574" target="_top">Kapil BHATIA</a></td><td>1:00:22</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.63 %</td><td>M</td><td>112</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:56:39</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">186</td><td><a href="athletehistory?athleteNumber=3286975" target="_top">Sonia BHATIA</a></td><td>1:00:23</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>27.77 %</td><td>F</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:56:42</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">187</td><td><a href="athletehistory?athleteNumber=1968284" target="_top">Eric NEGROS</a></td><td>1:00:37</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>25.35 %</td><td>M</td><td>113</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:52:13</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">188</td><td><a href="athletehistory?athleteNumber=2619285" target="_top">Lizda AHAD</a></td><td>1:00:38</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>24.82 %</td><td>F</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:56</td><td>44</td><td style="min-width:72px"/></tr></tbody></table></p><div class="paddedt left"><h3 class="paddedtandb">Thanks to the volunteers</h3><p class="paddedb">We are very grateful to the volunteers who made this event happen:Lizda AHAD, Aaryan BHATIA, Sue BROWN, Trevor GODDARD, Sarah HERAT, Leanne OBERIN, Simon REYMERS, Evan ROBERTSON, Alex ROBINS, Claire SMALE, Rosemary WAGHORN</p><p class="paddedb">Why not <a href="../../volunteer/">get involved and volunteer</a>?</p><h3 class="paddetandb" id="explainTable">
	What is this table? </h3>
'''
run.addRunners(text)
run.addVolunteers(text)

#http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=151
text = '''
<table class="sortable" id="results" align="center"><thead><tr><th class="pos">Pos</th><th style="width: 150px">parkrunner</th><th>Time</th><th>Age Cat</th><th>Age Grade</th><th/><th>Gender Pos</th><th style="width: 130px">Club</th><th>Note</th><th colspan="2">Total Runs</th></tr></thead><tbody><tr><td class="pos">1</td><td><a href="athletehistory?athleteNumber=3609446" target="_top">Ross THOMAS</a></td><td>17:28</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>73.85 %</td><td>M</td><td>1</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">2</td><td><a href="athletehistory?athleteNumber=3964524" target="_top">Matt GRANT</a></td><td>17:36</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>74.15 %</td><td>M</td><td>2</td><td><a href="../clubhistory?clubNum=19917">Victorian Cross Country League</a></td><td>New PB!</td><td>18</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/24946535"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">3</td><td><a href="athletehistory?athleteNumber=3558536" target="_top">Long NGUYEN</a></td><td>17:42</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>72.88 %</td><td>M</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">4</td><td><a href="athletehistory?athleteNumber=4053650" target="_top">Philip WEATHERLAKE</a></td><td>18:25</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>82.08 %</td><td>M</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">5</td><td><a href="athletehistory?athleteNumber=1232135" target="_top">Peter RUSHEN</a></td><td>19:40</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>79.49 %</td><td>M</td><td>5</td><td><a href="../clubhistory?clubNum=19905">Melbourne Midday Milers</a></td><td>New PB!</td><td>78</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">6</td><td><a href="athletehistory?athleteNumber=1739084" target="_top">Paul CONNOR</a></td><td>19:56</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>77.09 %</td><td>M</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">7</td><td><a href="athletehistory?athleteNumber=2927404" target="_top">Simon REYMERS</a></td><td>19:57</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>66.50 %</td><td>M</td><td>7</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:19:19</td><td>40</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/18022867"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">8</td><td><a href="athletehistory?athleteNumber=3124966" target="_top">Thomas BAKER</a></td><td>20:14</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>71.75 %</td><td>M</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">9</td><td><a href="athletehistory?athleteNumber=3447233" target="_top">Sarah WALL</a></td><td>20:20</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>76.23 %</td><td>F</td><td>1</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:14</td><td>32</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">10</td><td><a href="athletehistory?athleteNumber=4008975" target="_top">Cameron GODDARD</a></td><td>20:22</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>77.50 %</td><td>M</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">11</td><td><a href="athletehistory?athleteNumber=1356829" target="_top">Evan ROBERTSON</a></td><td>20:24</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>71.16 %</td><td>M</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:19</td><td>64</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">12</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">13</td><td><a href="athletehistory?athleteNumber=2655215" target="_top">Dominic MASCETTA</a></td><td>20:47</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>62.39 %</td><td>M</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>60</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/26826765"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">14</td><td><a href="athletehistory?athleteNumber=4547427" target="_top">Sydney SMITH</a></td><td>21:02</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>67.43 %</td><td>M</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">15</td><td><a href="athletehistory?athleteNumber=3988032" target="_top">Tim HOWE</a></td><td>21:09</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>64.07 %</td><td>M</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">16</td><td><a href="athletehistory?athleteNumber=4245030" target="_top">Caleb PAYNE</a></td><td>21:17</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>61.24 %</td><td>M</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">17</td><td><a href="athletehistory?athleteNumber=4105933" target="_top">Craig WASSENBERG</a></td><td>21:19</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>63.10 %</td><td>M</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">18</td><td><a href="athletehistory?athleteNumber=2899338" target="_top">Phillip DANE</a></td><td>21:51</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>72.16 %</td><td>M</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">19</td><td><a href="athletehistory?athleteNumber=1256687" target="_top">Justin EAGLETON</a></td><td>22:10</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>63.46 %</td><td>M</td><td>18</td><td><a href="../clubhistory?clubNum=22136">Yarra Ranges Athletics</a></td><td>First Timer!</td><td>50</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">20</td><td><a href="athletehistory?athleteNumber=434079" target="_top">Kym OSMAND</a></td><td>22:34</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>66.25 %</td><td>F</td><td>2</td><td><a href="../clubhistory?clubNum=18125">Oakleigh AC</a></td><td>First Timer!</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">21</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">22</td><td><a href="athletehistory?athleteNumber=2668532" target="_top">Tyler CARTLEDGE</a></td><td>22:52</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>74.42 %</td><td>M</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>30</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">23</td><td><a href="athletehistory?athleteNumber=2668506" target="_top">Laura CARTLEDGE</a></td><td>22:53</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>65.77 %</td><td>F</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>29</td><td style="min-width:72px"/></tr><tr><td class="pos">24</td><td><a href="athletehistory?athleteNumber=664309" target="_top">Adrian MASCETTA</a></td><td>23:06</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>55.92 %</td><td>M</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:59</td><td>27</td><td style="min-width:72px"/></tr><tr><td class="pos">25</td><td><a href="athletehistory?athleteNumber=4314582" target="_top">Hugh MCDONALD</a></td><td>23:30</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>62.27 %</td><td>M</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">26</td><td><a href="athletehistory?athleteNumber=2856359" target="_top">Paulo RAMOS</a></td><td>23:35</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>59.22 %</td><td>M</td><td>23</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:56</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/914269"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">27</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">28</td><td><a href="athletehistory?athleteNumber=2221609" target="_top">Megan PEREIRA</a></td><td>24:03</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>66.39 %</td><td>F</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:04</td><td>78</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">29</td><td><a href="athletehistory?athleteNumber=4210599" target="_top">Elliana SALTALAMACCHIA</a></td><td>24:04</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>61.50 %</td><td>F</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:42</td><td>13</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/13341061"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">30</td><td><a href="athletehistory?athleteNumber=3903734" target="_top">Andrew CHEN</a></td><td>24:10</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>61.52 %</td><td>M</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>15</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">31</td><td><a href="athletehistory?athleteNumber=4571608" target="_top">David RICKETTS</a></td><td>24:33</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>62.59 %</td><td>M</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/14862"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">32</td><td><a href="athletehistory?athleteNumber=898604" target="_top">Andrew PURCELL</a></td><td>24:48</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>55.85 %</td><td>M</td><td>27</td><td><a href="../clubhistory?clubNum=26149">Australia Post Group</a></td><td>PB stays at 00:23:29</td><td>56</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">33</td><td><a href="athletehistory?athleteNumber=2635993" target="_top">Michael MCNAMARA</a></td><td>25:03</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>60.35 %</td><td>M</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">34</td><td><a href="athletehistory?athleteNumber=3120027" target="_top">Alex MOCHALOV</a></td><td>25:21</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>51.74 %</td><td>M</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:58</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">35</td><td><a href="athletehistory?athleteNumber=1991576" target="_top">Arianna WALLEY</a></td><td>25:30</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>59.28 %</td><td>F</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">36</td><td><a href="athletehistory?athleteNumber=3447301" target="_top">Arthur WALL</a></td><td>25:37</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>58.04 %</td><td>M</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:59</td><td>44</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/29599152"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">37</td><td><a href="athletehistory?athleteNumber=2038372" target="_top">Nathan DONOVAN</a></td><td>25:55</td><td><a href="../agecategorytable/?ageCat=JM15-17">JM15-17</a></td><td>53.70 %</td><td>M</td><td>31</td><td><a href="../clubhistory?clubNum=21765">South East Juniors Umpiring</a></td><td>PB stays at 00:24:22</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">38</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">39</td><td><a href="athletehistory?athleteNumber=1755976" target="_top">James TAYLOR</a></td><td>26:07</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>56.48 %</td><td>M</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:17</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">40</td><td><a href="athletehistory?athleteNumber=4342106" target="_top">Steven BEASLEY</a></td><td>26:32</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>48.87 %</td><td>M</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:19</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22624622"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">41</td><td><a href="athletehistory?athleteNumber=1065439" target="_top">Michael GOGERLY</a></td><td>26:37</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>50.91 %</td><td>M</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:49</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">42</td><td><a href="athletehistory?athleteNumber=4169254" target="_top">Amy CLISSOLD</a></td><td>26:39</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>55.53 %</td><td>F</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">43</td><td><a href="athletehistory?athleteNumber=2269017" target="_top">Raymond HARVEY</a></td><td>26:43</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>58.52 %</td><td>M</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>55</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">44</td><td><a href="athletehistory?athleteNumber=1882866" target="_top">Elijah SUMMERS</a></td><td>26:48</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>63.50 %</td><td>M</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:01</td><td>72</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">45</td><td><a href="athletehistory?athleteNumber=1161216" target="_top">Toscha STOPAR</a></td><td>26:55</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>60.80 %</td><td>F</td><td>8</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>First Timer!</td><td>103</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">46</td><td><a href="athletehistory?athleteNumber=4316933" target="_top">Nathaniel PARSONS</a></td><td>27:01</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>47.75 %</td><td>M</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">47</td><td><a href="athletehistory?athleteNumber=1750540" target="_top">Jake NGU</a></td><td>27:11</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>55.12 %</td><td>M</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:10</td><td>123</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11301806"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">48</td><td><a href="athletehistory?athleteNumber=3104637" target="_top">Damian DONOVAN</a></td><td>27:21</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>53.50 %</td><td>M</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">49</td><td><a href="athletehistory?athleteNumber=3233492" target="_top">Sudhir KUMAR</a></td><td>27:32</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.51 %</td><td>M</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:47</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">50</td><td><a href="athletehistory?athleteNumber=1767271" target="_top">Reid GERRARD</a></td><td>27:50</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>56.17 %</td><td>M</td><td>42</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:52</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">51</td><td><a href="athletehistory?athleteNumber=4116148" target="_top">Lyn BEARD</a></td><td>27:53</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>64.08 %</td><td>F</td><td>9</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:27:31</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">52</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">53</td><td><a href="athletehistory?athleteNumber=2765339" target="_top">Duncan WHITE</a></td><td>28:01</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>53.06 %</td><td>M</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">54</td><td><a href="athletehistory?athleteNumber=3519429" target="_top">Biljana ZJAJIC</a></td><td>28:04</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>52.79 %</td><td>F</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">55</td><td><a href="athletehistory?athleteNumber=4371920" target="_top">Jeff KENNY</a></td><td>28:08</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>55.57 %</td><td>M</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:43</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">56</td><td><a href="athletehistory?athleteNumber=3217176" target="_top">Martin MCCUSKER</a></td><td>28:11</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.33 %</td><td>M</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:13</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">57</td><td><a href="athletehistory?athleteNumber=4145404" target="_top">Sam KAR</a></td><td>28:17</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>49.38 %</td><td>M</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:06</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">58</td><td><a href="athletehistory?athleteNumber=2378428" target="_top">Owen PEREIRA</a></td><td>28:19</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>62.86 %</td><td>M</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:50</td><td>71</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">59</td><td><a href="athletehistory?athleteNumber=2334951" target="_top">Nathan WRIGHT</a></td><td>28:20</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>45.53 %</td><td>M</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">60</td><td><a href="athletehistory?athleteNumber=3373087" target="_top">Nino MASCETTA</a></td><td>28:24</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>56.46 %</td><td>M</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:44</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">61</td><td><a href="athletehistory?athleteNumber=2135357" target="_top">Tania STEWART</a></td><td>28:25</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>59.71 %</td><td>F</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">62</td><td><a href="athletehistory?athleteNumber=3407380" target="_top">Letitia WEBBER</a></td><td>28:26</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>52.05 %</td><td>F</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>11</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22516738"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">63</td><td><a href="athletehistory?athleteNumber=3249513" target="_top">Kashyp BHATIA</a></td><td>28:47</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.43 %</td><td>M</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:59</td><td>39</td><td style="min-width:72px"/></tr><tr><td class="pos">64</td><td><a href="athletehistory?athleteNumber=1719187" target="_top">Helena BOSHOFF</a></td><td>28:56</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>53.57 %</td><td>F</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:09</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">65</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">66</td><td><a href="athletehistory?athleteNumber=2137630" target="_top">Mark STEWART</a></td><td>29:06</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>51.09 %</td><td>M</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">67</td><td><a href="athletehistory?athleteNumber=4114817" target="_top">Achintha JAYAWANTHA</a></td><td>29:09</td><td><a href="../agecategorytable/?ageCat=JM15-17">JM15-17</a></td><td>47.74 %</td><td>M</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">68</td><td><a href="athletehistory?athleteNumber=1665830" target="_top">Sally-Anne CARROLL</a></td><td>29:13</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>53.85 %</td><td>F</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:02</td><td>86</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/16265934"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">69</td><td><a href="athletehistory?athleteNumber=2848908" target="_top">Jenny PATERSON</a></td><td>29:16</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>53.76 %</td><td>F</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:55</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">70</td><td><a href="athletehistory?athleteNumber=4526607" target="_top">Sam TYRER</a></td><td>29:19</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>44.12 %</td><td>M</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">71</td><td><a href="athletehistory?athleteNumber=2395087" target="_top">Ashley WILLIS</a></td><td>29:38</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>49.94 %</td><td>F</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/16902477"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">72</td><td><a href="athletehistory?athleteNumber=3904164" target="_top">Jonathan CHEN</a></td><td>29:50</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>52.91 %</td><td>M</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:11</td><td>18</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">73</td><td><a href="athletehistory?athleteNumber=4217346" target="_top">Xiwu CHEN</a></td><td>29:52</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>49.78 %</td><td>M</td><td>57</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:22:59</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">74</td><td><a href="athletehistory?athleteNumber=4443869" target="_top">Brett INDER</a></td><td>30:16</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>52.09 %</td><td>M</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">75</td><td><a href="athletehistory?athleteNumber=2168873" target="_top">Caroline HUMPHREYS</a></td><td>30:25</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>49.15 %</td><td>F</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">76</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">77</td><td><a href="athletehistory?athleteNumber=4527635" target="_top">Erin PEARCE</a></td><td>30:27</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>54.63 %</td><td>F</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">78</td><td><a href="athletehistory?athleteNumber=4114644" target="_top">Hannah PEARCE</a></td><td>30:30</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>52.35 %</td><td>F</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">79</td><td><a href="athletehistory?athleteNumber=2223986" target="_top">Emily PEREIRA</a></td><td>30:34</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>55.83 %</td><td>F</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">80</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">81</td><td><a href="athletehistory?athleteNumber=4396444" target="_top">Stephen John JEFFERY</a></td><td>30:43</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>49.21 %</td><td>M</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/3405039"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">82</td><td><a href="athletehistory?athleteNumber=2369510" target="_top">Jackie MITCHELL</a></td><td>30:49</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>57.22 %</td><td>F</td><td>21</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:26:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">83</td><td><a href="athletehistory?athleteNumber=4090905" target="_top">Natasha TRINKLE</a></td><td>31:15</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>48.43 %</td><td>F</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:24</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">84</td><td><a href="athletehistory?athleteNumber=1745657" target="_top">Marianne SMITH</a></td><td>31:30</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>56.72 %</td><td>F</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:03</td><td>89</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">85</td><td><a href="athletehistory?athleteNumber=4043596" target="_top">Suzanne EVANS</a></td><td>31:55</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>65.48 %</td><td>F</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:58</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">86</td><td><a href="athletehistory?athleteNumber=898600" target="_top">Stephanie PURCELL</a></td><td>32:03</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>48.00 %</td><td>F</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:11</td><td>56</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">87</td><td><a href="athletehistory?athleteNumber=4368476" target="_top">Grant BROWN</a></td><td>32:11</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>47.33 %</td><td>M</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:12</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">88</td><td><a href="athletehistory?athleteNumber=4114804" target="_top">Nik JAYAWANTHA</a></td><td>32:27</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>43.71 %</td><td>M</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:26</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">89</td><td><a href="athletehistory?athleteNumber=1748710" target="_top">Matthew RUSHTON</a></td><td>32:32</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>42.26 %</td><td>M</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:08</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">90</td><td><a href="athletehistory?athleteNumber=1776109" target="_top">Kimberly IMPERIAL</a></td><td>32:37</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>45.38 %</td><td>F</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:24</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">91</td><td><a href="athletehistory?athleteNumber=4277072" target="_top">Jane HARRIS</a></td><td>32:39</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.11 %</td><td>F</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">92</td><td><a href="athletehistory?athleteNumber=1786065" target="_top">Tina TIAN</a></td><td>32:41</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.06 %</td><td>F</td><td>28</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:27:48</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">93</td><td><a href="athletehistory?athleteNumber=725933" target="_top">Kym LAZARIDIS</a></td><td>32:43</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>52.52 %</td><td>F</td><td>29</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:31:00</td><td>124</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">94</td><td><a href="athletehistory?athleteNumber=3510394" target="_top">Donna KENNY</a></td><td>32:54</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>52.23 %</td><td>F</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">95</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">96</td><td><a href="athletehistory?athleteNumber=2135388" target="_top">Stephen WHITE</a></td><td>33:11</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>46.31 %</td><td>M</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:29</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">97</td><td><a href="athletehistory?athleteNumber=2221582" target="_top">Rohini PEREIRA</a></td><td>33:22</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>46.45 %</td><td>F</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:50</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">98</td><td><a href="athletehistory?athleteNumber=2769381" target="_top">Marina JORDAN</a></td><td>33:34</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>51.19 %</td><td>F</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>77</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">99</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">100</td><td><a href="athletehistory?athleteNumber=4516104" target="_top">Maryanne MCCONNELL</a></td><td>34:00</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>51.86 %</td><td>F</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">101</td><td><a href="athletehistory?athleteNumber=4269919" target="_top">Cathy CANNON</a></td><td>34:01</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>59.58 %</td><td>F</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">102</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">103</td><td><a href="athletehistory?athleteNumber=4554329" target="_top">Lu ZHANG</a></td><td>34:26</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>46.56 %</td><td>F</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">104</td><td><a href="athletehistory?athleteNumber=2083536" target="_top">Jenny JAMES</a></td><td>34:30</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>57.87 %</td><td>F</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:20</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">105</td><td><a href="athletehistory?athleteNumber=3650435" target="_top">Andrew HEALE</a></td><td>34:33</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>40.71 %</td><td>M</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:46</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">106</td><td><a href="athletehistory?athleteNumber=1260208" target="_top">Ed WILSON</a></td><td>34:49</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>49.07 %</td><td>M</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:20</td><td>136</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">107</td><td><a href="athletehistory?athleteNumber=396821" target="_top">Fiona GARDINER</a></td><td>34:51</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>43.42 %</td><td>F</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:42</td><td>43</td><td style="min-width:72px"/></tr><tr><td class="pos">108</td><td><a href="athletehistory?athleteNumber=4547419" target="_top">Rachel SMITH</a></td><td>35:02</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>48.43 %</td><td>F</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:12</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7243173"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">109</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">110</td><td><a href="athletehistory?athleteNumber=1766265" target="_top">Darren JENKINS</a></td><td>35:06</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>38.89 %</td><td>M</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:39</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">111</td><td><a href="athletehistory?athleteNumber=4551385" target="_top">Rachel CURTIS</a></td><td>35:10</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>50.14 %</td><td>F</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:09</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">112</td><td><a href="athletehistory?athleteNumber=3212945" target="_top">Graham ROBERTSON</a></td><td>35:15</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>52.06 %</td><td>M</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:06</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">113</td><td><a href="athletehistory?athleteNumber=2207985" target="_top">Andrew WEATHERHEAD</a></td><td>36:21</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>47.46 %</td><td>M</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:16</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">114</td><td><a href="athletehistory?athleteNumber=4574818" target="_top">Pepi GRAY</a></td><td>36:28</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>45.61 %</td><td>F</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:31</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">115</td><td><a href="athletehistory?athleteNumber=1717831" target="_top">Thomas SOM</a></td><td>36:32</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>41.70 %</td><td>M</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:17</td><td>95</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">116</td><td><a href="athletehistory?athleteNumber=3093991" target="_top">Michelle TAY</a></td><td>36:50</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>40.86 %</td><td>F</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:02</td><td>48</td><td style="min-width:72px"/></tr><tr><td class="pos">117</td><td><a href="athletehistory?athleteNumber=4453453" target="_top">Rochelle WHEATLEY</a></td><td>37:05</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>40.58 %</td><td>F</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">118</td><td><a href="athletehistory?athleteNumber=1783047" target="_top">Helen TAYLOR</a></td><td>37:12</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>45.61 %</td><td>F</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:36</td><td>32</td><td style="min-width:72px"/></tr><tr><td class="pos">119</td><td><a href="athletehistory?athleteNumber=4086945" target="_top">Jerry ZHANG</a></td><td>37:17</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>39.25 %</td><td>M</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:05</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">120</td><td><a href="athletehistory?athleteNumber=2033999" target="_top">Sylvia JEONG</a></td><td>37:43</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>47.99 %</td><td>F</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">121</td><td><a href="athletehistory?athleteNumber=4254332" target="_top">Ajantha KUMARI</a></td><td>38:15</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>41.92 %</td><td>F</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:04</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">122</td><td><a href="athletehistory?athleteNumber=3757996" target="_top">Elizabeth ALDA</a></td><td>38:38</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>45.08 %</td><td>F</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:37:58</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">123</td><td><a href="athletehistory?athleteNumber=581899" target="_top">Lisa OSMAND</a></td><td>39:10</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>37.83 %</td><td>F</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>8</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7189257"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">124</td><td><a href="athletehistory?athleteNumber=4542585" target="_top">Lyndie GARDNER</a></td><td>39:13</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>47.43 %</td><td>F</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">125</td><td><a href="athletehistory?athleteNumber=1792850" target="_top">Lisa BULLOCK</a></td><td>39:17</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>38.95 %</td><td>F</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">126</td><td><a href="athletehistory?athleteNumber=4574997" target="_top">Gerald MCDORNAN</a></td><td>39:23</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>38.38 %</td><td>M</td><td>77</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:45</td><td>3</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/8316973"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">127</td><td><a href="athletehistory?athleteNumber=3803840" target="_top">Annie CHEN</a></td><td>39:51</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>44.83 %</td><td>F</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:41</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">128</td><td><a href="athletehistory?athleteNumber=4506751" target="_top">Liwen DAI</a></td><td>40:06</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>34.54 %</td><td>M</td><td>78</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:07</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">129</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">130</td><td><a href="athletehistory?athleteNumber=4578742" target="_top">Jessica FERNANDEZ</a></td><td>40:58</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>36.13 %</td><td>F</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">131</td><td><a href="athletehistory?athleteNumber=2663927" target="_top">Tanya KERR</a></td><td>41:10</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>38.58 %</td><td>F</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:41</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">132</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">133</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">134</td><td><a href="athletehistory?athleteNumber=1716573" target="_top">Nina MUSUMECI</a></td><td>43:24</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>35.45 %</td><td>F</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:23</td><td>87</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">135</td><td><a href="athletehistory?athleteNumber=4358918" target="_top">Lais MATOS</a></td><td>44:03</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>33.60 %</td><td>F</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:29</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">136</td><td><a href="athletehistory?athleteNumber=2867022" target="_top">Viviane RAMOS</a></td><td>44:04</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>35.44 %</td><td>F</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/2075388"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">137</td><td><a href="athletehistory?athleteNumber=4225423" target="_top">Richard SLUGGETT</a></td><td>45:10</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>36.46 %</td><td>M</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:42:22</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">138</td><td><a href="athletehistory?athleteNumber=1393531" target="_top">Dianne HARVEY</a></td><td>45:16</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>39.99 %</td><td>F</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:10</td><td>43</td><td style="min-width:72px"/></tr><tr><td class="pos">139</td><td><a href="athletehistory?athleteNumber=2336872" target="_top">Tim STICKLEY</a></td><td>45:17</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>32.06 %</td><td>M</td><td>83</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:19:40</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/148656"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">140</td><td><a href="athletehistory?athleteNumber=4600220" target="_top">Ryan GODDARD</a></td><td>45:25</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>39.19 %</td><td>M</td><td>84</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">141</td><td><a href="athletehistory?athleteNumber=2068233" target="_top">Leanne OBERIN</a></td><td>45:32</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>34.88 %</td><td>F</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">142</td><td><a href="athletehistory?athleteNumber=4106933" target="_top">Rosemary SLUGGETT</a></td><td>45:33</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>42.01 %</td><td>F</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">143</td><td><a href="athletehistory?athleteNumber=2761418" target="_top">Carolyn COYLE</a></td><td>45:35</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>35.17 %</td><td>F</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">144</td><td><a href="athletehistory?athleteNumber=4483471" target="_top">Trevor GODDARD</a></td><td>45:55</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>31.36 %</td><td>M</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">145</td><td><a href="athletehistory?athleteNumber=4272564" target="_top">Margaret HELLYER</a></td><td>46:11</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>39.73 %</td><td>F</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:43:11</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">146</td><td><a href="athletehistory?athleteNumber=4574844" target="_top">Marketta GRAY</a></td><td>46:35</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>32.84 %</td><td>F</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">147</td><td><a href="athletehistory?athleteNumber=4272066" target="_top">Danielle SALTALAMACCHIA</a></td><td>46:56</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>34.87 %</td><td>F</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:25</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">148</td><td><a href="athletehistory?athleteNumber=2071200" target="_top">Heather O'CONNOR</a></td><td>47:14</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>44.92 %</td><td>F</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">149</td><td><a href="athletehistory?athleteNumber=3890975" target="_top">Ian O'CONNOR</a></td><td>47:21</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>36.43 %</td><td>M</td><td>86</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">150</td><td><a href="athletehistory?athleteNumber=1864696" target="_top">Mark VULLING</a></td><td>47:51</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>30.34 %</td><td>M</td><td>87</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:41</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">151</td><td><a href="athletehistory?athleteNumber=4482563" target="_top">Susan SUN</a></td><td>47:56</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>34.14 %</td><td>F</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:16</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">152</td><td><a href="athletehistory?athleteNumber=2451717" target="_top">Nola NUGENT</a></td><td>50:43</td><td><a href="../agecategorytable/?ageCat=VW75-79">VW75-79</a></td><td>50.51 %</td><td>F</td><td>65</td><td><a href="../clubhistory?clubNum=23016">Heart Foundation Walking</a></td><td>PB stays at 00:44:08</td><td>48</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/18007012"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">153</td><td><a href="athletehistory?athleteNumber=1245122" target="_top">Pauline EDER</a></td><td>50:54</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>45.22 %</td><td>F</td><td>66</td><td><a href="../clubhistory?clubNum=23016">Heart Foundation Walking</a></td><td>PB stays at 00:46:19</td><td>69</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">154</td><td><a href="athletehistory?athleteNumber=1245151" target="_top">Herbert EDER</a></td><td>50:55</td><td><a href="../agecategorytable/?ageCat=VM85-89">VM85-89</a></td><td>49.46 %</td><td>M</td><td>88</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:04</td><td>72</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">155</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">156</td><td><a href="athletehistory?athleteNumber=4548439" target="_top">Christine DAWSON</a></td><td>52:55</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>31.69 %</td><td>F</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">157</td><td><a href="athletehistory?athleteNumber=2619285" target="_top">Lizda AHAD</a></td><td>55:23</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>27.08 %</td><td>F</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:56</td><td>44</td><td style="min-width:72px"/></tr></tbody></table></p><div class="paddedt left"><h3 class="paddedtandb">Thanks to the volunteers</h3><p class="paddedb">We are very grateful to the volunteers who made this event happen:Lizda AHAD, Glenn BERRY, Aaryan BHATIA, Ann-Marie JEFFERY, Gregory MOORE, Pauline PAINE, Leonora POWRIE, Danni RAMALINGAM, Elijah SUMMERS, Lizzie SUMMERS, Rosemary WAGHORN</p><p class="paddedb">Why not <a href="../../volunteer/">get involved and volunteer</a>?</p><h3 class="paddetandb" id="explainTable">
	What is this table? </h3>
'''
run.addRunners(text)
run.addVolunteers(text)

#http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=150
text = '''
<table class="sortable" id="results" align="center"><thead><tr><th class="pos">Pos</th><th style="width: 150px">parkrunner</th><th>Time</th><th>Age Cat</th><th>Age Grade</th><th/><th>Gender Pos</th><th style="width: 130px">Club</th><th>Note</th><th colspan="2">Total Runs</th></tr></thead><tbody><tr><td class="pos">1</td><td><a href="athletehistory?athleteNumber=541216" target="_top">Kevin MULLER</a></td><td>17:47</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>81.63 %</td><td>M</td><td>1</td><td><a href="../clubhistory?clubNum=19919">Wodonga Athletic Club</a></td><td>First Timer!</td><td>144</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/13193568"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">2</td><td><a href="athletehistory?athleteNumber=3558536" target="_top">Long NGUYEN</a></td><td>18:17</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>70.56 %</td><td>M</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:17:42</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">3</td><td><a href="athletehistory?athleteNumber=1356829" target="_top">Evan ROBERTSON</a></td><td>20:22</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>71.28 %</td><td>M</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:19</td><td>64</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">4</td><td><a href="athletehistory?athleteNumber=3447233" target="_top">Sarah WALL</a></td><td>20:25</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>75.92 %</td><td>F</td><td>1</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:14</td><td>32</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">5</td><td><a href="athletehistory?athleteNumber=4271309" target="_top">John NANKERVIS</a></td><td>20:53</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>61.77 %</td><td>M</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:10</td><td>7</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/738302"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">6</td><td><a href="athletehistory?athleteNumber=4475081" target="_top">Jayden NG</a></td><td>21:04</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>74.92 %</td><td>M</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">7</td><td><a href="athletehistory?athleteNumber=4475084" target="_top">Joel NG</a></td><td>21:04</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>74.92 %</td><td>M</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:10</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">8</td><td><a href="athletehistory?athleteNumber=4547427" target="_top">Sydney SMITH</a></td><td>21:45</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>65.21 %</td><td>M</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">9</td><td><a href="athletehistory?athleteNumber=4355332" target="_top">Nathan HUNTER</a></td><td>21:55</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>60.15 %</td><td>M</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/29499751"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">10</td><td><a href="athletehistory?athleteNumber=2668578" target="_top">Scott CARTLEDGE</a></td><td>22:21</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>61.97 %</td><td>M</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>23</td><td style="min-width:72px"/></tr><tr><td class="pos">11</td><td><a href="athletehistory?athleteNumber=3691268" target="_top">Callum MCDONALD</a></td><td>22:32</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>57.25 %</td><td>M</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:18:49</td><td>30</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23436720"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">12</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">13</td><td><a href="athletehistory?athleteNumber=4008975" target="_top">Cameron GODDARD</a></td><td>22:46</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>69.33 %</td><td>M</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">14</td><td><a href="athletehistory?athleteNumber=2899195" target="_top">Ranga HALWALAGE</a></td><td>23:08</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>59.01 %</td><td>M</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:56</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">15</td><td><a href="athletehistory?athleteNumber=4147617" target="_top">Rebekah POWIERSKI</a></td><td>23:21</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>66.38 %</td><td>F</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:00</td><td>10</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">16</td><td><a href="athletehistory?athleteNumber=1927958" target="_top">John KIARIE</a></td><td>23:23</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>61.08 %</td><td>M</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:06</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">17</td><td><a href="athletehistory?athleteNumber=3095999" target="_top">Michael HUNT</a></td><td>23:27</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>56.57 %</td><td>M</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>47</td><td style="min-width:72px"/></tr><tr><td class="pos">18</td><td><a href="athletehistory?athleteNumber=4210599" target="_top">Elliana SALTALAMACCHIA</a></td><td>23:43</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>62.40 %</td><td>F</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:42</td><td>13</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/13341061"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">19</td><td><a href="athletehistory?athleteNumber=2694977" target="_top">Alison CHAPMAN</a></td><td>23:44</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>71.49 %</td><td>F</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:51</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">20</td><td><a href="athletehistory?athleteNumber=4490875" target="_top">David IRVING</a></td><td>23:48</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>55.39 %</td><td>M</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23969193"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">21</td><td><a href="athletehistory?athleteNumber=2382370" target="_top">Scott PORTBURY</a></td><td>24:04</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>57.55 %</td><td>M</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:37</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/4319859"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">22</td><td><a href="athletehistory?athleteNumber=4012375" target="_top">Bronwen KUHLMANN</a></td><td>24:07</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>71.25 %</td><td>F</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">23</td><td><a href="athletehistory?athleteNumber=4501566" target="_top">Cecai ZHANG</a></td><td>24:19</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>61.14 %</td><td>M</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">24</td><td><a href="athletehistory?athleteNumber=4475086" target="_top">Joshua NG</a></td><td>24:23</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>67.05 %</td><td>M</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">25</td><td><a href="athletehistory?athleteNumber=3120027" target="_top">Alex MOCHALOV</a></td><td>24:35</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>53.36 %</td><td>M</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:58</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">26</td><td><a href="athletehistory?athleteNumber=2668532" target="_top">Tyler CARTLEDGE</a></td><td>24:51</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>68.48 %</td><td>M</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>30</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">27</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">28</td><td><a href="athletehistory?athleteNumber=2443307" target="_top">Alan TAYLOR</a></td><td>24:55</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>56.45 %</td><td>M</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:17</td><td>58</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">29</td><td><a href="athletehistory?athleteNumber=2728709" target="_top">Alex STAMA</a></td><td>25:06</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>51.46 %</td><td>M</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:17</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">30</td><td><a href="athletehistory?athleteNumber=569294" target="_top">Lizzie SUMMERS</a></td><td>25:09</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>60.83 %</td><td>F</td><td>6</td><td><a href="../clubhistory?clubNum=21990">UP Coaching</a></td><td>PB stays at 00:24:00</td><td>75</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">31</td><td><a href="athletehistory?athleteNumber=3233492" target="_top">Sudhir KUMAR</a></td><td>25:16</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>56.13 %</td><td>M</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:47</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">32</td><td><a href="athletehistory?athleteNumber=3447301" target="_top">Arthur WALL</a></td><td>25:31</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>58.26 %</td><td>M</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:59</td><td>44</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/29599152"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">33</td><td><a href="athletehistory?athleteNumber=1741011" target="_top">Elsa MC GANN</a></td><td>25:36</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>69.79 %</td><td>F</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:36</td><td>29</td><td style="min-width:72px"/></tr><tr><td class="pos">34</td><td><a href="athletehistory?athleteNumber=3076792" target="_top">Aleisha WILLS</a></td><td>25:38</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>57.74 %</td><td>F</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:35</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">35</td><td><a href="athletehistory?athleteNumber=1433599" target="_top">Ajay KAVASSERI</a></td><td>25:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>56.32 %</td><td>M</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:40</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">36</td><td><a href="athletehistory?athleteNumber=3671091" target="_top">Michael WAN</a></td><td>26:02</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>52.82 %</td><td>M</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>22</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23772417"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">37</td><td><a href="athletehistory?athleteNumber=3056576" target="_top">Venkat DODDI</a></td><td>26:10</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>57.26 %</td><td>M</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:53</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">38</td><td><a href="athletehistory?athleteNumber=1750540" target="_top">Jake NGU</a></td><td>26:19</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>56.93 %</td><td>M</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:10</td><td>123</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11301806"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">39</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">40</td><td><a href="athletehistory?athleteNumber=2095309" target="_top">Rosemary WAGHORN</a></td><td>26:24</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>60.16 %</td><td>F</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">41</td><td><a href="athletehistory?athleteNumber=4259462" target="_top">Ruby RUBYCHANDRAN</a></td><td>26:33</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>52.61 %</td><td>M</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">42</td><td><a href="athletehistory?athleteNumber=2668558" target="_top">Ashlyn CARTLEDGE</a></td><td>26:58</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>72.44 %</td><td>F</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>27</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">43</td><td><a href="athletehistory?athleteNumber=2668506" target="_top">Laura CARTLEDGE</a></td><td>26:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>55.78 %</td><td>F</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:53</td><td>29</td><td style="min-width:72px"/></tr><tr><td class="pos">44</td><td><a href="athletehistory?athleteNumber=3179070" target="_top">Ting CAO</a></td><td>27:13</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>49.05 %</td><td>M</td><td>33</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:21</td><td>24</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5244204"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">45</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">46</td><td><a href="athletehistory?athleteNumber=2336000" target="_top">Peter YEOMAN</a></td><td>27:17</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>48.63 %</td><td>M</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">47</td><td><a href="athletehistory?athleteNumber=4181093" target="_top">Kai HINGMANN</a></td><td>27:21</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>51.43 %</td><td>M</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:22</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">48</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">49</td><td><a href="athletehistory?athleteNumber=2647705" target="_top">Michael BLEWETT</a></td><td>27:38</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>53.38 %</td><td>M</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>34</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/12444211"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">50</td><td><a href="athletehistory?athleteNumber=4114804" target="_top">Nik JAYAWANTHA</a></td><td>28:04</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.53 %</td><td>M</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:26</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">51</td><td><a href="athletehistory?athleteNumber=4316933" target="_top">Nathaniel PARSONS</a></td><td>28:10</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>45.80 %</td><td>M</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:53</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">52</td><td><a href="athletehistory?athleteNumber=4090888" target="_top">Hans TRINKLE</a></td><td>28:19</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.85 %</td><td>M</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:13</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">53</td><td><a href="athletehistory?athleteNumber=3368539" target="_top">Courtney SPLATT</a></td><td>28:23</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>52.38 %</td><td>F</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:53</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">54</td><td><a href="athletehistory?athleteNumber=4116148" target="_top">Lyn BEARD</a></td><td>28:28</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>62.76 %</td><td>F</td><td>13</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:27:31</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">55</td><td><a href="athletehistory?athleteNumber=1311284" target="_top">Peter HAMILTON</a></td><td>28:30</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.35 %</td><td>M</td><td>42</td><td><a href="../clubhistory?clubNum=23644">Achilles Running Club Melbourne</a></td><td>PB stays at 00:26:20</td><td>145</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7164369"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">56</td><td><a href="athletehistory?athleteNumber=1882866" target="_top">Elijah SUMMERS</a></td><td>28:31</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>59.67 %</td><td>M</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:01</td><td>72</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">57</td><td><a href="athletehistory?athleteNumber=4145404" target="_top">Sam KAR</a></td><td>28:33</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>48.92 %</td><td>M</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:06</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">58</td><td><a href="athletehistory?athleteNumber=3911794" target="_top">Daniel ZHANG</a></td><td>28:46</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>56.84 %</td><td>M</td><td>45</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:27:35</td><td>24</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">59</td><td><a href="athletehistory?athleteNumber=3373087" target="_top">Nino MASCETTA</a></td><td>28:49</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>55.64 %</td><td>M</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:44</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">60</td><td><a href="athletehistory?athleteNumber=2135357" target="_top">Tania STEWART</a></td><td>28:51</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>58.12 %</td><td>F</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">61</td><td><a href="athletehistory?athleteNumber=1767271" target="_top">Reid GERRARD</a></td><td>28:53</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>54.13 %</td><td>M</td><td>47</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:52</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">62</td><td><a href="athletehistory?athleteNumber=4214936" target="_top">Zhao Hong SU</a></td><td>29:05</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>57.65 %</td><td>F</td><td>15</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>First Timer!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">63</td><td><a href="athletehistory?athleteNumber=1748710" target="_top">Matthew RUSHTON</a></td><td>29:07</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>47.22 %</td><td>M</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:08</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">64</td><td><a href="athletehistory?athleteNumber=2848908" target="_top">Jenny PATERSON</a></td><td>29:10</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>53.94 %</td><td>F</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">65</td><td><a href="athletehistory?athleteNumber=3904164" target="_top">Jonathan CHEN</a></td><td>29:11</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>54.08 %</td><td>M</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>18</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">66</td><td><a href="athletehistory?athleteNumber=4217346" target="_top">Xiwu CHEN</a></td><td>29:13</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>50.88 %</td><td>M</td><td>50</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:22:59</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">67</td><td><a href="athletehistory?athleteNumber=2285678" target="_top">Nicki WHITFIELD</a></td><td>29:13</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>58.07 %</td><td>F</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:06</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">68</td><td><a href="athletehistory?athleteNumber=1065439" target="_top">Michael GOGERLY</a></td><td>29:15</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>46.32 %</td><td>M</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:49</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">69</td><td><a href="athletehistory?athleteNumber=3407380" target="_top">Letitia WEBBER</a></td><td>29:17</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>50.54 %</td><td>F</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:26</td><td>11</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22516738"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">70</td><td><a href="athletehistory?athleteNumber=4342106" target="_top">Steven BEASLEY</a></td><td>29:18</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>44.25 %</td><td>M</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:19</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22624622"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">71</td><td><a href="athletehistory?athleteNumber=1744665" target="_top">Maree DALZOTTO</a></td><td>29:19</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>58.61 %</td><td>F</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:21</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">72</td><td><a href="athletehistory?athleteNumber=4371920" target="_top">Jeff KENNY</a></td><td>29:31</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>52.96 %</td><td>M</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:43</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">73</td><td><a href="athletehistory?athleteNumber=3217176" target="_top">Martin MCCUSKER</a></td><td>29:39</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>47.84 %</td><td>M</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:13</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">74</td><td><a href="athletehistory?athleteNumber=3470293" target="_top">Claire SMALE</a></td><td>29:47</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>50.03 %</td><td>F</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:22</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">75</td><td><a href="athletehistory?athleteNumber=1735935" target="_top">David GEORGE</a></td><td>30:02</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>43.29 %</td><td>M</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:19</td><td>39</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/19204898"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">76</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">77</td><td><a href="athletehistory?athleteNumber=2137630" target="_top">Mark STEWART</a></td><td>30:12</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>49.23 %</td><td>M</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">78</td><td><a href="athletehistory?athleteNumber=3514440" target="_top">Gary WYATT</a></td><td>30:17</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>53.44 %</td><td>M</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:59</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">79</td><td><a href="athletehistory?athleteNumber=4526607" target="_top">Sam TYRER</a></td><td>30:21</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>42.61 %</td><td>M</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">80</td><td><a href="athletehistory?athleteNumber=2428952" target="_top">Terry RANDALL</a></td><td>30:24</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>52.30 %</td><td>M</td><td>60</td><td><a href="../clubhistory?clubNum=22580">Pakenham Road Runners</a></td><td>PB stays at 00:27:50</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/15367956"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">81</td><td><a href="athletehistory?athleteNumber=2369510" target="_top">Jackie MITCHELL</a></td><td>30:33</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>57.72 %</td><td>F</td><td>21</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:26:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">82</td><td><a href="athletehistory?athleteNumber=3971846" target="_top">Rebecca POSTLETHWAITE</a></td><td>30:35</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>48.39 %</td><td>F</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">83</td><td><a href="athletehistory?athleteNumber=1822194" target="_top">Alexander ROBERTSON</a></td><td>30:40</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>46.25 %</td><td>M</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:18</td><td>19</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">84</td><td><a href="athletehistory?athleteNumber=4290916" target="_top">Chin LIM</a></td><td>30:51</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>54.89 %</td><td>M</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:49</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/9773029"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">85</td><td><a href="athletehistory?athleteNumber=2266645" target="_top">Kirsten LIGHTFOOT</a></td><td>31:05</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>48.42 %</td><td>F</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:43</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">86</td><td><a href="athletehistory?athleteNumber=4368476" target="_top">Grant BROWN</a></td><td>31:12</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>48.82 %</td><td>M</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">87</td><td><a href="athletehistory?athleteNumber=1745657" target="_top">Marianne SMITH</a></td><td>31:24</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>56.90 %</td><td>F</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:03</td><td>89</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">88</td><td><a href="athletehistory?athleteNumber=2395087" target="_top">Ashley WILLIS</a></td><td>31:25</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>47.11 %</td><td>F</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/16902477"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">89</td><td><a href="athletehistory?athleteNumber=4396444" target="_top">Stephen John JEFFERY</a></td><td>31:31</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>47.96 %</td><td>M</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/3405039"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">90</td><td><a href="athletehistory?athleteNumber=4547419" target="_top">Rachel SMITH</a></td><td>31:34</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>53.75 %</td><td>F</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7243173"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">91</td><td><a href="athletehistory?athleteNumber=2345651" target="_top">Jackie DOWLING</a></td><td>31:35</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>57.31 %</td><td>F</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:36</td><td>43</td><td style="min-width:72px"/></tr><tr><td class="pos">92</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">93</td><td><a href="athletehistory?athleteNumber=2416833" target="_top">Paul COSGRAVE</a></td><td>31:44</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>46.85 %</td><td>M</td><td>66</td><td><a href="../clubhistory?clubNum=22575">Westerfoldians</a></td><td>PB stays at 00:29:04</td><td>105</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">94</td><td><a href="athletehistory?athleteNumber=4498741" target="_top">Ivan DE SOUZA</a></td><td>31:48</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>47.12 %</td><td>M</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">95</td><td><a href="athletehistory?athleteNumber=1424117" target="_top">David FOSKEY</a></td><td>31:49</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>56.10 %</td><td>M</td><td>68</td><td><a href="../clubhistory?clubNum=19912">Spartans</a></td><td>First Timer!</td><td>35</td><td style="min-width:72px"/></tr><tr><td class="pos">96</td><td><a href="athletehistory?athleteNumber=1783614" target="_top">Rachel SHEPPARD</a></td><td>31:50</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.90 %</td><td>F</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:54</td><td>80</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">97</td><td><a href="athletehistory?athleteNumber=1882862" target="_top">Gregory MOORE</a></td><td>32:05</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>44.21 %</td><td>M</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:22</td><td>69</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">98</td><td><a href="athletehistory?athleteNumber=4243723" target="_top">Om KAR</a></td><td>32:09</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>45.10 %</td><td>M</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:53</td><td>11</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">99</td><td><a href="athletehistory?athleteNumber=3985226" target="_top">Joy YIN</a></td><td>32:22</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>54.22 %</td><td>F</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">100</td><td><a href="athletehistory?athleteNumber=3970628" target="_top">Marina LI</a></td><td>32:27</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>50.44 %</td><td>F</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>27</td><td style="min-width:72px"/></tr><tr><td class="pos">101</td><td><a href="athletehistory?athleteNumber=1458926" target="_top">Anita AJAY</a></td><td>32:55</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>48.71 %</td><td>F</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:06</td><td>93</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">102</td><td><a href="athletehistory?athleteNumber=3510394" target="_top">Donna KENNY</a></td><td>33:15</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>51.68 %</td><td>F</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:29</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">103</td><td><a href="athletehistory?athleteNumber=4574818" target="_top">Pepi GRAY</a></td><td>33:31</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>49.63 %</td><td>F</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">104</td><td><a href="athletehistory?athleteNumber=2769381" target="_top">Marina JORDAN</a></td><td>33:37</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>51.12 %</td><td>F</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>77</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">105</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">106</td><td><a href="athletehistory?athleteNumber=2135388" target="_top">Stephen WHITE</a></td><td>33:48</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>45.46 %</td><td>M</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:29</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">107</td><td><a href="athletehistory?athleteNumber=3286687" target="_top">Aaryan BHATIA</a></td><td>33:50</td><td><a href="../agecategorytable/?ageCat=JM15-17">JM15-17</a></td><td>41.13 %</td><td>M</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:23</td><td>32</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">108</td><td><a href="athletehistory?athleteNumber=4516104" target="_top">Maryanne MCCONNELL</a></td><td>34:03</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>51.79 %</td><td>F</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">109</td><td><a href="athletehistory?athleteNumber=2130247" target="_top">Rachel SCHRODER</a></td><td>34:16</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>43.29 %</td><td>F</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">110</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">111</td><td><a href="athletehistory?athleteNumber=4554329" target="_top">Lu ZHANG</a></td><td>34:44</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>46.16 %</td><td>F</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">112</td><td><a href="athletehistory?athleteNumber=2083536" target="_top">Jenny JAMES</a></td><td>34:48</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>57.38 %</td><td>F</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:20</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">113</td><td><a href="athletehistory?athleteNumber=4269919" target="_top">Cathy CANNON</a></td><td>34:58</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>57.96 %</td><td>F</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:00</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">114</td><td><a href="athletehistory?athleteNumber=3212945" target="_top">Graham ROBERTSON</a></td><td>35:15</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>52.06 %</td><td>M</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">115</td><td><a href="athletehistory?athleteNumber=4176258" target="_top">Jing YE</a></td><td>35:22</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>42.04 %</td><td>F</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:27</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">116</td><td><a href="athletehistory?athleteNumber=2428953" target="_top">Janne RANDALL</a></td><td>36:12</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>52.85 %</td><td>F</td><td>41</td><td><a href="../clubhistory?clubNum=22580">Pakenham Road Runners</a></td><td>PB stays at 00:33:11</td><td>81</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">117</td><td><a href="athletehistory?athleteNumber=1345664" target="_top">Leonie MCNEILL</a></td><td>36:16</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>41.73 %</td><td>F</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:37</td><td>120</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">118</td><td><a href="athletehistory?athleteNumber=1786065" target="_top">Tina TIAN</a></td><td>36:19</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>44.15 %</td><td>F</td><td>43</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:27:48</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">119</td><td><a href="athletehistory?athleteNumber=3051474" target="_top">Anthony VAROS</a></td><td>36:21</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>43.01 %</td><td>M</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:37</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">120</td><td><a href="athletehistory?athleteNumber=2353796" target="_top">Warren PAGE</a></td><td>36:31</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>40.71 %</td><td>M</td><td>77</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:17</td><td>84</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/24529909"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">121</td><td><a href="athletehistory?athleteNumber=3911796" target="_top">Nathaniel ZHANG</a></td><td>36:57</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>50.61 %</td><td>M</td><td>78</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:31:41</td><td>23</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">122</td><td><a href="athletehistory?athleteNumber=3886143" target="_top">Baoping ZHANG</a></td><td>36:58</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>39.27 %</td><td>M</td><td>79</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:19:53</td><td>26</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5221032"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">123</td><td><a href="athletehistory?athleteNumber=4254332" target="_top">Ajantha KUMARI</a></td><td>37:15</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>43.04 %</td><td>F</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">124</td><td><a href="athletehistory?athleteNumber=4453453" target="_top">Rochelle WHEATLEY</a></td><td>37:16</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>40.38 %</td><td>F</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:50</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">125</td><td><a href="athletehistory?athleteNumber=4442556" target="_top">Bernie HEV</a></td><td>37:35</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>39.78 %</td><td>F</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">126</td><td><a href="athletehistory?athleteNumber=3591013" target="_top">Robyn DIMOPOULOS</a></td><td>37:43</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>39.51 %</td><td>F</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:13</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">127</td><td><a href="athletehistory?athleteNumber=1735852" target="_top">Josephine CARDACI</a></td><td>37:46</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>44.40 %</td><td>F</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:20</td><td>117</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">128</td><td><a href="athletehistory?athleteNumber=4574997" target="_top">Gerald MCDORNAN</a></td><td>38:41</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>39.08 %</td><td>M</td><td>80</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/8316973"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">129</td><td><a href="athletehistory?athleteNumber=3911799" target="_top">Sally ZHOU</a></td><td>39:03</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>40.67 %</td><td>F</td><td>49</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:39:01</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">130</td><td><a href="athletehistory?athleteNumber=3971595" target="_top">Sue BROWN</a></td><td>39:10</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>42.81 %</td><td>F</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:38:32</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">131</td><td><a href="athletehistory?athleteNumber=3803840" target="_top">Annie CHEN</a></td><td>39:41</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>45.02 %</td><td>F</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">132</td><td><a href="athletehistory?athleteNumber=2207985" target="_top">Andrew WEATHERHEAD</a></td><td>40:04</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>43.05 %</td><td>M</td><td>81</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:16</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">133</td><td><a href="athletehistory?athleteNumber=4475265" target="_top">Amelia NG</a></td><td>40:10</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>50.83 %</td><td>F</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">134</td><td><a href="athletehistory?athleteNumber=4475078" target="_top">Freddy NG</a></td><td>40:12</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>34.20 %</td><td>M</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:06</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">135</td><td><a href="athletehistory?athleteNumber=4542585" target="_top">Lyndie GARDNER</a></td><td>41:16</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>45.07 %</td><td>F</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">136</td><td><a href="athletehistory?athleteNumber=1792850" target="_top">Lisa BULLOCK</a></td><td>41:17</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>37.06 %</td><td>F</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">137</td><td><a href="athletehistory?athleteNumber=3750431" target="_top">Isha DODDI</a></td><td>41:18</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>41.32 %</td><td>F</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:52</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">138</td><td><a href="athletehistory?athleteNumber=2777655" target="_top">Toby SCOTT</a></td><td>41:31</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>40.99 %</td><td>M</td><td>83</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:22</td><td>68</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/25563106"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">139</td><td><a href="athletehistory?athleteNumber=2777601" target="_top">Christine KAUCNER</a></td><td>41:42</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>39.25 %</td><td>F</td><td>56</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:26:16</td><td>76</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/12533560"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">140</td><td><a href="athletehistory?athleteNumber=2207989" target="_top">Mary WEATHERHEAD</a></td><td>41:51</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>49.94 %</td><td>F</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">141</td><td><a href="athletehistory?athleteNumber=4363576" target="_top">Ann-Marie JEFFERY</a></td><td>42:16</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>40.14 %</td><td>F</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:33</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">142</td><td><a href="athletehistory?athleteNumber=4515767" target="_top">Kaila SWYER</a></td><td>42:28</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>34.85 %</td><td>F</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">143</td><td><a href="athletehistory?athleteNumber=4574923" target="_top">Darren JEFFREYS</a></td><td>42:29</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>30.52 %</td><td>M</td><td>84</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">144</td><td><a href="athletehistory?athleteNumber=4272741" target="_top">Janelle HINGMANN</a></td><td>42:40</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>36.60 %</td><td>F</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:38:18</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">145</td><td><a href="athletehistory?athleteNumber=2202224" target="_top">Alex ROBINS</a></td><td>42:46</td><td><a href="../agecategorytable/?ageCat=VM75-79">VM75-79</a></td><td>45.99 %</td><td>M</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:27</td><td>100</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">146</td><td><a href="athletehistory?athleteNumber=3438449" target="_top">Allen PRATT</a></td><td>42:47</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>40.75 %</td><td>M</td><td>86</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:52</td><td>34</td><td style="min-width:72px"/></tr><tr><td class="pos">147</td><td><a href="athletehistory?athleteNumber=2180623" target="_top">Jacqui CARTER</a></td><td>44:12</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>47.29 %</td><td>F</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:17</td><td>61</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">148</td><td><a href="athletehistory?athleteNumber=2180571" target="_top">Jennifer HOGARTY</a></td><td>44:37</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>48.30 %</td><td>F</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:40:33</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">149</td><td><a href="athletehistory?athleteNumber=4542816" target="_top">Julia LIM</a></td><td>44:50</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>45.20 %</td><td>F</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">150</td><td><a href="athletehistory?athleteNumber=3249513" target="_top">Kashyp BHATIA</a></td><td>45:12</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>32.12 %</td><td>M</td><td>87</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:59</td><td>39</td><td style="min-width:72px"/></tr><tr><td class="pos">151</td><td><a href="athletehistory?athleteNumber=4482563" target="_top">Susan SUN</a></td><td>45:18</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>36.13 %</td><td>F</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:16</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">152</td><td><a href="athletehistory?athleteNumber=4037877" target="_top">Sadia NAZIER</a></td><td>46:52</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>32.65 %</td><td>F</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:29</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">153</td><td><a href="athletehistory?athleteNumber=4574844" target="_top">Marketta GRAY</a></td><td>47:34</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>32.17 %</td><td>F</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">154</td><td><a href="athletehistory?athleteNumber=2068233" target="_top">Leanne OBERIN</a></td><td>49:35</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>32.03 %</td><td>F</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:45:24</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">155</td><td><a href="athletehistory?athleteNumber=4037850" target="_top">Shanaaz NAZIER-CUPIDO</a></td><td>49:51</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>30.19 %</td><td>F</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:48:37</td><td>23</td><td style="min-width:72px"/></tr><tr><td class="pos">156</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">157</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">158</td><td><a href="athletehistory?athleteNumber=2777633" target="_top">Nathan SCOTT</a></td><td>51:14</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>30.81 %</td><td>M</td><td>90</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:53</td><td>66</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">159</td><td><a href="athletehistory?athleteNumber=2777641" target="_top">Andrew SCOTT</a></td><td>51:15</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>26.83 %</td><td>M</td><td>91</td><td><a href="../clubhistory?clubNum=21413">Derek Zoolander Centre for Kids Who Can't Run Good</a></td><td>PB stays at 00:24:49</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/199341"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">160</td><td><a href="athletehistory?athleteNumber=4037905" target="_top">Denzil CUPIDO</a></td><td>51:47</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>26.17 %</td><td>M</td><td>92</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:04</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">161</td><td><a href="athletehistory?athleteNumber=3286975" target="_top">Sonia BHATIA</a></td><td>59:23</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>28.23 %</td><td>F</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:56:42</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">162</td><td><a href="athletehistory?athleteNumber=1968284" target="_top">Eric NEGROS</a></td><td>1:00:10</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>25.54 %</td><td>M</td><td>93</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:52:13</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">163</td><td><a href="athletehistory?athleteNumber=3291967" target="_top">Tvisha MEHRA</a></td><td>1:01:07</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>27.92 %</td><td>F</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:46:39</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">164</td><td><a href="athletehistory?athleteNumber=1716573" target="_top">Nina MUSUMECI</a></td><td>1:01:22</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>25.07 %</td><td>F</td><td>71</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:23</td><td>87</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr></tbody></table></p><div class="paddedt left"><h3 class="paddedtandb">Thanks to the volunteers</h3><p class="paddedb">We are very grateful to the volunteers who made this event happen:Naomi (Tullae) CROTTY, Phillip DANE, Wayne JEWELL, Chris MCNEILL, Nina MUSUMECI, Jos PEREIRA, Rohini PEREIRA, Megan PEREIRA, Owen PEREIRA</p><p class="paddedb">Why not <a href="../../volunteer/">get involved and volunteer</a>?</p><h3 class="paddetandb" id="explainTable">
	What is this table? </h3>
'''
run.addRunners(text)
run.addVolunteers(text)

#http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=149
text = '''
<table class="sortable" id="results" align="center"><thead><tr><th class="pos">Pos</th><th style="width: 150px">parkrunner</th><th>Time</th><th>Age Cat</th><th>Age Grade</th><th/><th>Gender Pos</th><th style="width: 130px">Club</th><th>Note</th><th colspan="2">Total Runs</th></tr></thead><tbody><tr><td class="pos">1</td><td><a href="athletehistory?athleteNumber=2118251" target="_top">Bradley SIMPSON</a></td><td>17:08</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>75.29 %</td><td>M</td><td>1</td><td><a href="../clubhistory?clubNum=18070">Athletics Nunawading</a></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">2</td><td><a href="athletehistory?athleteNumber=1344427" target="_top">David VENOUR</a></td><td>17:14</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>84.91 %</td><td>M</td><td>2</td><td><a href="../clubhistory?clubNum=19905">Melbourne Midday Milers</a></td><td>PB stays at 00:16:53</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">3</td><td><a href="athletehistory?athleteNumber=1967289" target="_top">Taylor KEARNEY</a></td><td>17:37</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>73.23 %</td><td>M</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">4</td><td><a href="athletehistory?athleteNumber=3558536" target="_top">Long NGUYEN</a></td><td>18:28</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>69.86 %</td><td>M</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:17:42</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">5</td><td><a href="athletehistory?athleteNumber=3447233" target="_top">Sarah WALL</a></td><td>20:22</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>76.10 %</td><td>F</td><td>1</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:14</td><td>32</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">6</td><td><a href="athletehistory?athleteNumber=1356829" target="_top">Evan ROBERTSON</a></td><td>20:24</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>71.16 %</td><td>M</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:19</td><td>64</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">7</td><td><a href="athletehistory?athleteNumber=4245030" target="_top">Caleb PAYNE</a></td><td>21:38</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>60.25 %</td><td>M</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:17</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">8</td><td><a href="athletehistory?athleteNumber=3988032" target="_top">Tim HOWE</a></td><td>22:01</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>61.09 %</td><td>M</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:55</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">9</td><td><a href="athletehistory?athleteNumber=4547427" target="_top">Sydney SMITH</a></td><td>22:04</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>64.27 %</td><td>M</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">10</td><td><a href="athletehistory?athleteNumber=2899338" target="_top">Phillip DANE</a></td><td>22:08</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>71.23 %</td><td>M</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">11</td><td><a href="athletehistory?athleteNumber=3785904" target="_top">Brooke SIMPSON</a></td><td>22:41</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>65.25 %</td><td>F</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">12</td><td><a href="athletehistory?athleteNumber=3147330" target="_top">Mark F</a></td><td>23:05</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>59.13 %</td><td>M</td><td>10</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>First Timer!</td><td>35</td><td style="min-width:72px"/></tr><tr><td class="pos">13</td><td><a href="athletehistory?athleteNumber=4449056" target="_top">Ben KEATING</a></td><td>23:08</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>57.35 %</td><td>M</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">14</td><td><a href="athletehistory?athleteNumber=1927958" target="_top">John KIARIE</a></td><td>23:15</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>61.43 %</td><td>M</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:06</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">15</td><td><a href="athletehistory?athleteNumber=4147617" target="_top">Rebekah POWIERSKI</a></td><td>23:15</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>66.67 %</td><td>F</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>10</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">16</td><td><a href="athletehistory?athleteNumber=4449070" target="_top">Tina MCCARTHY</a></td><td>23:40</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>62.96 %</td><td>F</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:28</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">17</td><td><a href="athletehistory?athleteNumber=2856359" target="_top">Paulo RAMOS</a></td><td>23:47</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>58.72 %</td><td>M</td><td>13</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:56</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/914269"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">18</td><td><a href="athletehistory?athleteNumber=3095999" target="_top">Michael HUNT</a></td><td>23:54</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>55.51 %</td><td>M</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>47</td><td style="min-width:72px"/></tr><tr><td class="pos">19</td><td><a href="athletehistory?athleteNumber=2382370" target="_top">Scott PORTBURY</a></td><td>24:13</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>57.19 %</td><td>M</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:37</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/4319859"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">20</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">21</td><td><a href="athletehistory?athleteNumber=2744740" target="_top">Bruce LYON</a></td><td>24:27</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>65.58 %</td><td>M</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:18</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">22</td><td><a href="athletehistory?athleteNumber=3205411" target="_top">Ian SKELTON</a></td><td>24:31</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>57.38 %</td><td>M</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:07</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">23</td><td><a href="athletehistory?athleteNumber=3617092" target="_top">David WILSON</a></td><td>24:32</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>52.58 %</td><td>M</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>20</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/13660213"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">24</td><td><a href="athletehistory?athleteNumber=3120027" target="_top">Alex MOCHALOV</a></td><td>24:33</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>53.43 %</td><td>M</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:58</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">25</td><td><a href="athletehistory?athleteNumber=4012375" target="_top">Bronwen KUHLMANN</a></td><td>24:35</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>69.90 %</td><td>F</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:29</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">26</td><td><a href="athletehistory?athleteNumber=2589482" target="_top">Dirk HERAT</a></td><td>24:36</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>59.49 %</td><td>M</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">27</td><td><a href="athletehistory?athleteNumber=2221609" target="_top">Megan PEREIRA</a></td><td>24:42</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>64.64 %</td><td>F</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:04</td><td>78</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">28</td><td><a href="athletehistory?athleteNumber=2986450" target="_top">Adam BROWN</a></td><td>24:47</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>57.23 %</td><td>M</td><td>22</td><td><a href="../clubhistory?clubNum=22623">Secret Rocky Runners</a></td><td>First Timer!</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">29</td><td><a href="athletehistory?athleteNumber=1133856" target="_top">Kathy SOUTER</a></td><td>24:48</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>75.00 %</td><td>F</td><td>7</td><td><a href="../clubhistory?clubNum=19901">Knox Roadrunners</a></td><td>PB stays at 00:23:24</td><td>25</td><td style="min-width:72px"/></tr><tr><td class="pos">30</td><td><a href="athletehistory?athleteNumber=2100211" target="_top">Megan BLACKMAN</a></td><td>24:49</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>59.70 %</td><td>F</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>37</td><td style="min-width:72px"/></tr><tr><td class="pos">31</td><td><a href="athletehistory?athleteNumber=4319164" target="_top">Bryan HALL</a></td><td>24:53</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>57.40 %</td><td>M</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">32</td><td><a href="athletehistory?athleteNumber=3176187" target="_top">Damien O'HARA</a></td><td>25:04</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>57.91 %</td><td>M</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:22</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">33</td><td><a href="athletehistory?athleteNumber=2728709" target="_top">Alex STAMA</a></td><td>25:19</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>51.02 %</td><td>M</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:17</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">34</td><td><a href="athletehistory?athleteNumber=4490875" target="_top">David IRVING</a></td><td>25:23</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>51.94 %</td><td>M</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23969193"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">35</td><td><a href="athletehistory?athleteNumber=4495527" target="_top">Jason SMITH</a></td><td>25:44</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>50.13 %</td><td>M</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:18</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">36</td><td><a href="athletehistory?athleteNumber=3492109" target="_top">Robert CRANMER</a></td><td>25:46</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>50.06 %</td><td>M</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">37</td><td><a href="athletehistory?athleteNumber=4210599" target="_top">Elliana SALTALAMACCHIA</a></td><td>25:47</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>57.40 %</td><td>F</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:42</td><td>13</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/13341061"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">38</td><td><a href="athletehistory?athleteNumber=4342106" target="_top">Steven BEASLEY</a></td><td>25:49</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>50.23 %</td><td>M</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:19</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22624622"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">39</td><td><a href="athletehistory?athleteNumber=3056576" target="_top">Venkat DODDI</a></td><td>25:53</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>57.89 %</td><td>M</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">40</td><td><a href="athletehistory?athleteNumber=3988030" target="_top">Linda FAIRHURST</a></td><td>26:00</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>62.95 %</td><td>F</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:42</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">41</td><td><a href="athletehistory?athleteNumber=3447301" target="_top">Arthur WALL</a></td><td>26:02</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>57.11 %</td><td>M</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:59</td><td>44</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/29599152"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">42</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">43</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">44</td><td><a href="athletehistory?athleteNumber=3233492" target="_top">Sudhir KUMAR</a></td><td>26:15</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>54.03 %</td><td>M</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:47</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">45</td><td><a href="athletehistory?athleteNumber=3537860" target="_top">George MANOS</a></td><td>26:28</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>53.59 %</td><td>M</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:49</td><td>28</td><td style="min-width:72px"/></tr><tr><td class="pos">46</td><td><a href="athletehistory?athleteNumber=1915199" target="_top">Wayne JEWELL</a></td><td>26:45</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>57.45 %</td><td>M</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:39</td><td>76</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11496423"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">47</td><td><a href="athletehistory?athleteNumber=1750540" target="_top">Jake NGU</a></td><td>26:48</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>55.91 %</td><td>M</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:10</td><td>123</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11301806"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">48</td><td><a href="athletehistory?athleteNumber=1562450" target="_top">Sarah HERAT</a></td><td>26:56</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>59.53 %</td><td>F</td><td>11</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:25:02</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">49</td><td><a href="athletehistory?athleteNumber=2647705" target="_top">Michael BLEWETT</a></td><td>27:05</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>54.46 %</td><td>M</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>34</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/12444211"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">50</td><td><a href="athletehistory?athleteNumber=3678564" target="_top">Danni RAMALINGAM</a></td><td>27:12</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>58.95 %</td><td>F</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">51</td><td><a href="athletehistory?athleteNumber=2378428" target="_top">Owen PEREIRA</a></td><td>27:25</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>64.92 %</td><td>M</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:50</td><td>71</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">52</td><td><a href="athletehistory?athleteNumber=4328012" target="_top">Dhanush RUBYCHANDRAN</a></td><td>27:25</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>57.57 %</td><td>M</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:24</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">53</td><td><a href="athletehistory?athleteNumber=4259462" target="_top">Ruby RUBYCHANDRAN</a></td><td>27:42</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>50.42 %</td><td>M</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:33</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">54</td><td><a href="athletehistory?athleteNumber=4114804" target="_top">Nik JAYAWANTHA</a></td><td>27:46</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.08 %</td><td>M</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:26</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">55</td><td><a href="athletehistory?athleteNumber=4553217" target="_top">Janelle THOMAS</a></td><td>27:48</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>58.27 %</td><td>F</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">56</td><td><a href="athletehistory?athleteNumber=1947254" target="_top">Ann-Maree BAKER</a></td><td>28:01</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>55.74 %</td><td>F</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">57</td><td><a href="athletehistory?athleteNumber=2135357" target="_top">Tania STEWART</a></td><td>28:03</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>59.77 %</td><td>F</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">58</td><td><a href="athletehistory?athleteNumber=4145404" target="_top">Sam KAR</a></td><td>28:06</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>49.70 %</td><td>M</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">59</td><td><a href="athletehistory?athleteNumber=4408547" target="_top">Kristeen WONG</a></td><td>28:07</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>52.64 %</td><td>F</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">60</td><td><a href="athletehistory?athleteNumber=4506751" target="_top">Liwen DAI</a></td><td>28:07</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>49.26 %</td><td>M</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">61</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">62</td><td><a href="athletehistory?athleteNumber=4090888" target="_top">Hans TRINKLE</a></td><td>28:15</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.97 %</td><td>M</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">63</td><td><a href="athletehistory?athleteNumber=1719187" target="_top">Helena BOSHOFF</a></td><td>28:16</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>54.83 %</td><td>F</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:09</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">64</td><td><a href="athletehistory?athleteNumber=3470293" target="_top">Claire SMALE</a></td><td>28:24</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>52.46 %</td><td>F</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">65</td><td><a href="athletehistory?athleteNumber=4467181" target="_top">Florence WANG</a></td><td>28:31</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>52.02 %</td><td>F</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">66</td><td><a href="athletehistory?athleteNumber=3591013" target="_top">Robyn DIMOPOULOS</a></td><td>28:33</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>52.19 %</td><td>F</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:13</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">67</td><td><a href="athletehistory?athleteNumber=3373087" target="_top">Nino MASCETTA</a></td><td>28:44</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>55.80 %</td><td>M</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:44</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">68</td><td><a href="athletehistory?athleteNumber=1776109" target="_top">Kimberly IMPERIAL</a></td><td>28:45</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>51.48 %</td><td>F</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:24</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">69</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">70</td><td><a href="athletehistory?athleteNumber=4270234" target="_top">Eleassa PRESTON</a></td><td>28:47</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>54.26 %</td><td>F</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">71</td><td><a href="athletehistory?athleteNumber=2334951" target="_top">Nathan WRIGHT</a></td><td>29:40</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>43.48 %</td><td>M</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">72</td><td><a href="athletehistory?athleteNumber=4521874" target="_top">Angel Lu JIAO</a></td><td>29:48</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>54.92 %</td><td>F</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">73</td><td><a href="athletehistory?athleteNumber=2137630" target="_top">Mark STEWART</a></td><td>29:50</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>49.83 %</td><td>M</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">74</td><td><a href="athletehistory?athleteNumber=1927983" target="_top">Darren POWIERSKI</a></td><td>29:50</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>49.83 %</td><td>M</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:55</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">75</td><td><a href="athletehistory?athleteNumber=2848908" target="_top">Jenny PATERSON</a></td><td>29:51</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>52.71 %</td><td>F</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">76</td><td><a href="athletehistory?athleteNumber=1744665" target="_top">Maree DALZOTTO</a></td><td>29:54</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>57.47 %</td><td>F</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:21</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">77</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">78</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">79</td><td><a href="athletehistory?athleteNumber=4290916" target="_top">Chin LIM</a></td><td>30:07</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>56.23 %</td><td>M</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/9773029"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">80</td><td><a href="athletehistory?athleteNumber=1927985" target="_top">Catherine POWIERSKI</a></td><td>30:14</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>57.61 %</td><td>F</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:26</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">81</td><td><a href="athletehistory?athleteNumber=4090905" target="_top">Natasha TRINKLE</a></td><td>30:24</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>49.51 %</td><td>F</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">82</td><td><a href="athletehistory?athleteNumber=972999" target="_top">Alan HARRIS</a></td><td>30:27</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>48.44 %</td><td>M</td><td>55</td><td><a href="../clubhistory?clubNum=22596">Up 'n Active</a></td><td>PB stays at 00:23:09</td><td>135</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7161536"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">83</td><td><a href="athletehistory?athleteNumber=1736959" target="_top">Jock CRAVANA</a></td><td>30:28</td><td><a href="../agecategorytable/?ageCat=VM80-84">VM80-84</a></td><td>68.98 %</td><td>M</td><td>56</td><td><a href="../clubhistory?clubNum=18165">Victorian Masters Athletics</a></td><td>PB stays at 00:28:23</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">84</td><td><a href="athletehistory?athleteNumber=4550685" target="_top">Fraser SMITH</a></td><td>30:32</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>42.25 %</td><td>M</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">85</td><td><a href="athletehistory?athleteNumber=2223986" target="_top">Emily PEREIRA</a></td><td>30:40</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>55.65 %</td><td>F</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">86</td><td><a href="athletehistory?athleteNumber=3292314" target="_top">Laura DE RANGO</a></td><td>30:54</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>47.90 %</td><td>F</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>23</td><td style="min-width:72px"/></tr><tr><td class="pos">87</td><td><a href="athletehistory?athleteNumber=4443869" target="_top">Brett INDER</a></td><td>30:56</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>50.54 %</td><td>M</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">88</td><td><a href="athletehistory?athleteNumber=3249513" target="_top">Kashyp BHATIA</a></td><td>31:04</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>46.73 %</td><td>M</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:59</td><td>39</td><td style="min-width:72px"/></tr><tr><td class="pos">89</td><td><a href="athletehistory?athleteNumber=2727180" target="_top">Pauline PAINE</a></td><td>31:13</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>49.01 %</td><td>F</td><td>30</td><td><a href="../clubhistory?clubNum=23836">Human Performance Development</a></td><td>New PB!</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23806054"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">90</td><td><a href="athletehistory?athleteNumber=3481708" target="_top">Stella HEWSON</a></td><td>31:22</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>51.65 %</td><td>F</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:49</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">91</td><td><a href="athletehistory?athleteNumber=4368476" target="_top">Grant BROWN</a></td><td>31:23</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>48.54 %</td><td>M</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">92</td><td><a href="athletehistory?athleteNumber=2003877" target="_top">Julie LIN</a></td><td>31:26</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>56.84 %</td><td>F</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:23</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">93</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">94</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">95</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">96</td><td><a href="athletehistory?athleteNumber=2353796" target="_top">Warren PAGE</a></td><td>31:37</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>47.02 %</td><td>M</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:17</td><td>84</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/24529909"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">97</td><td><a href="athletehistory?athleteNumber=4551184" target="_top">Libby SZETHO</a></td><td>31:42</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>46.69 %</td><td>F</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">98</td><td><a href="athletehistory?athleteNumber=271155" target="_top">Kevin PARDY</a></td><td>31:51</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>47.04 %</td><td>M</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:52</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">99</td><td><a href="athletehistory?athleteNumber=2299278" target="_top">Lachie RICHTER</a></td><td>31:57</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>45.38 %</td><td>M</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>20</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">100</td><td><a href="athletehistory?athleteNumber=4553661" target="_top">Juli Lirong ZHOU</a></td><td>32:01</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>51.74 %</td><td>F</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">101</td><td><a href="athletehistory?athleteNumber=1260208" target="_top">Ed WILSON</a></td><td>32:02</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>53.33 %</td><td>M</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:20</td><td>136</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">102</td><td><a href="athletehistory?athleteNumber=4297251" target="_top">Kirsty HALL</a></td><td>32:06</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.48 %</td><td>F</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">103</td><td><a href="athletehistory?athleteNumber=2299274" target="_top">Dani BOLTON</a></td><td>32:09</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>50.91 %</td><td>F</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:01</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">104</td><td><a href="athletehistory?athleteNumber=3051475" target="_top">Dimitra VAROS</a></td><td>32:09</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>55.57 %</td><td>F</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">105</td><td><a href="athletehistory?athleteNumber=3951953" target="_top">Jillian RIGG</a></td><td>32:20</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>50.10 %</td><td>F</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:01</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">106</td><td><a href="athletehistory?athleteNumber=2453206" target="_top">Beverly TROUGHTON</a></td><td>32:26</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>52.98 %</td><td>F</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">107</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">108</td><td><a href="athletehistory?athleteNumber=3635855" target="_top">Amy WILSON</a></td><td>32:49</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>45.10 %</td><td>F</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">109</td><td><a href="athletehistory?athleteNumber=4551174" target="_top">Larry MEYER</a></td><td>32:52</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>42.80 %</td><td>M</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">110</td><td><a href="athletehistory?athleteNumber=3951798" target="_top">Catherine LEIPOLD</a></td><td>32:54</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>48.28 %</td><td>F</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">111</td><td><a href="athletehistory?athleteNumber=3970628" target="_top">Marina LI</a></td><td>32:56</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.70 %</td><td>F</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:27</td><td>27</td><td style="min-width:72px"/></tr><tr><td class="pos">112</td><td><a href="athletehistory?athleteNumber=1735852" target="_top">Josephine CARDACI</a></td><td>33:08</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>50.60 %</td><td>F</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:20</td><td>117</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">113</td><td><a href="athletehistory?athleteNumber=1345664" target="_top">Leonie MCNEILL</a></td><td>33:10</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>45.63 %</td><td>F</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:37</td><td>120</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">114</td><td><a href="athletehistory?athleteNumber=1345648" target="_top">Chris MCNEILL</a></td><td>33:10</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>41.16 %</td><td>M</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:56</td><td>133</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">115</td><td><a href="athletehistory?athleteNumber=2395087" target="_top">Ashley WILLIS</a></td><td>33:11</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>44.60 %</td><td>F</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/16902477"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">116</td><td><a href="athletehistory?athleteNumber=3510394" target="_top">Donna KENNY</a></td><td>33:16</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>51.65 %</td><td>F</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:29</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">117</td><td><a href="athletehistory?athleteNumber=3284734" target="_top">Kate WILKINS</a></td><td>33:23</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>56.52 %</td><td>F</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">118</td><td><a href="athletehistory?athleteNumber=4176258" target="_top">Jing YE</a></td><td>33:27</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>44.44 %</td><td>F</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">119</td><td><a href="athletehistory?athleteNumber=4086945" target="_top">Jerry ZHANG</a></td><td>33:43</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>43.40 %</td><td>M</td><td>71</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:05</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">120</td><td><a href="athletehistory?athleteNumber=2769381" target="_top">Marina JORDAN</a></td><td>33:46</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>50.89 %</td><td>F</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>77</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">121</td><td><a href="athletehistory?athleteNumber=3911794" target="_top">Daniel ZHANG</a></td><td>34:03</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>48.02 %</td><td>M</td><td>72</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:27:35</td><td>24</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">122</td><td><a href="athletehistory?athleteNumber=2083536" target="_top">Jenny JAMES</a></td><td>34:07</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>58.52 %</td><td>F</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">123</td><td><a href="athletehistory?athleteNumber=3911796" target="_top">Nathaniel ZHANG</a></td><td>34:09</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>54.76 %</td><td>M</td><td>73</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:31:41</td><td>23</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">124</td><td><a href="athletehistory?athleteNumber=3375523" target="_top">Flynn BARTLETT</a></td><td>34:09</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>46.22 %</td><td>M</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:06</td><td>33</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">125</td><td><a href="athletehistory?athleteNumber=3886143" target="_top">Baoping ZHANG</a></td><td>34:09</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>42.51 %</td><td>M</td><td>75</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:19:53</td><td>26</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5221032"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">126</td><td><a href="athletehistory?athleteNumber=3373279" target="_top">Peter BARTLETT</a></td><td>34:13</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>42.77 %</td><td>M</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:07</td><td>32</td><td style="min-width:72px"/></tr><tr><td class="pos">127</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">128</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">129</td><td><a href="athletehistory?athleteNumber=1786065" target="_top">Tina TIAN</a></td><td>34:46</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>46.12 %</td><td>F</td><td>51</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:27:48</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">130</td><td><a href="athletehistory?athleteNumber=4269919" target="_top">Cathy CANNON</a></td><td>34:50</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>58.18 %</td><td>F</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">131</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">132</td><td><a href="athletehistory?athleteNumber=4547419" target="_top">Rachel SMITH</a></td><td>35:07</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>48.32 %</td><td>F</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7243173"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">133</td><td><a href="athletehistory?athleteNumber=4551385" target="_top">Rachel CURTIS</a></td><td>35:07</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>50.21 %</td><td>F</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">134</td><td><a href="athletehistory?athleteNumber=4516104" target="_top">Maryanne MCCONNELL</a></td><td>35:16</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>50.00 %</td><td>F</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">135</td><td><a href="athletehistory?athleteNumber=2207985" target="_top">Andrew WEATHERHEAD</a></td><td>35:16</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>48.91 %</td><td>M</td><td>80</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">136</td><td><a href="athletehistory?athleteNumber=3485648" target="_top">Amanda BROWN</a></td><td>35:35</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>46.00 %</td><td>F</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:25</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">137</td><td><a href="athletehistory?athleteNumber=4451023" target="_top">Kathir APPADURAI</a></td><td>35:41</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>40.03 %</td><td>M</td><td>81</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">138</td><td><a href="athletehistory?athleteNumber=1717831" target="_top">Thomas SOM</a></td><td>35:52</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>42.47 %</td><td>M</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:17</td><td>95</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">139</td><td><a href="athletehistory?athleteNumber=3212945" target="_top">Graham ROBERTSON</a></td><td>36:07</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>50.81 %</td><td>M</td><td>83</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">140</td><td><a href="athletehistory?athleteNumber=4554329" target="_top">Lu ZHANG</a></td><td>37:01</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>43.31 %</td><td>F</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">141</td><td><a href="athletehistory?athleteNumber=3093991" target="_top">Michelle TAY</a></td><td>37:05</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>40.58 %</td><td>F</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:02</td><td>48</td><td style="min-width:72px"/></tr><tr><td class="pos">142</td><td><a href="athletehistory?athleteNumber=4469184" target="_top">Kyra LAWRENCE</a></td><td>37:12</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>39.78 %</td><td>F</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">143</td><td><a href="athletehistory?athleteNumber=4469035" target="_top">Savannah KNIGHT</a></td><td>37:12</td><td><a href="../agecategorytable/?ageCat=SW18-19">SW18-19</a></td><td>39.92 %</td><td>F</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">144</td><td><a href="athletehistory?athleteNumber=3051474" target="_top">Anthony VAROS</a></td><td>37:27</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>41.74 %</td><td>M</td><td>84</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:37</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">145</td><td><a href="athletehistory?athleteNumber=2635904" target="_top">Fleur NIKOLAISON</a></td><td>37:55</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>48.40 %</td><td>F</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:14</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">146</td><td><a href="athletehistory?athleteNumber=3757996" target="_top">Elizabeth ALDA</a></td><td>38:16</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>45.51 %</td><td>F</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:37:58</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">147</td><td><a href="athletehistory?athleteNumber=4116148" target="_top">Lyn BEARD</a></td><td>38:24</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>46.53 %</td><td>F</td><td>63</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:27:31</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">148</td><td><a href="athletehistory?athleteNumber=1745665" target="_top">Katie CROSS</a></td><td>38:24</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>47.14 %</td><td>F</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:09</td><td>61</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">149</td><td><a href="athletehistory?athleteNumber=4442556" target="_top">Bernie HEV</a></td><td>38:42</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>38.63 %</td><td>F</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">150</td><td><a href="athletehistory?athleteNumber=4453453" target="_top">Rochelle WHEATLEY</a></td><td>38:45</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>38.84 %</td><td>F</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:50</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">151</td><td><a href="athletehistory?athleteNumber=4254332" target="_top">Ajantha KUMARI</a></td><td>39:10</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>40.94 %</td><td>F</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:04</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">152</td><td><a href="athletehistory?athleteNumber=3971595" target="_top">Sue BROWN</a></td><td>39:49</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>42.11 %</td><td>F</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:38:32</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">153</td><td><a href="athletehistory?athleteNumber=3803840" target="_top">Annie CHEN</a></td><td>40:16</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>44.37 %</td><td>F</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:41</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">154</td><td><a href="athletehistory?athleteNumber=4551196" target="_top">Aaron LEE</a></td><td>40:37</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>34.92 %</td><td>M</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">155</td><td><a href="athletehistory?athleteNumber=2207989" target="_top">Mary WEATHERHEAD</a></td><td>40:44</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>51.31 %</td><td>F</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">156</td><td><a href="athletehistory?athleteNumber=656410" target="_top">Naomi (Tullae) CROTTY</a></td><td>41:07</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>36.81 %</td><td>F</td><td>71</td><td><a href="../clubhistory?clubNum=20982">Wooters Runners</a></td><td>PB stays at 00:39:11</td><td>146</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/25440878"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">157</td><td><a href="athletehistory?athleteNumber=4521846" target="_top">Thomas LIU</a></td><td>42:00</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>38.93 %</td><td>M</td><td>86</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">158</td><td><a href="athletehistory?athleteNumber=2202224" target="_top">Alex ROBINS</a></td><td>42:08</td><td><a href="../agecategorytable/?ageCat=VM75-79">VM75-79</a></td><td>45.77 %</td><td>M</td><td>87</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:27</td><td>100</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">159</td><td><a href="athletehistory?athleteNumber=3438449" target="_top">Allen PRATT</a></td><td>42:10</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>41.34 %</td><td>M</td><td>88</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:52</td><td>34</td><td style="min-width:72px"/></tr><tr><td class="pos">160</td><td><a href="athletehistory?athleteNumber=2180623" target="_top">Jacqui CARTER</a></td><td>42:41</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>48.97 %</td><td>F</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:17</td><td>61</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">161</td><td><a href="athletehistory?athleteNumber=2180571" target="_top">Jennifer HOGARTY</a></td><td>42:43</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>50.45 %</td><td>F</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:40:33</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">162</td><td><a href="athletehistory?athleteNumber=3314054" target="_top">Ken MATHERS</a></td><td>43:42</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>41.38 %</td><td>M</td><td>89</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">163</td><td><a href="athletehistory?athleteNumber=4542585" target="_top">Lyndie GARDNER</a></td><td>43:43</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>42.55 %</td><td>F</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">164</td><td><a href="athletehistory?athleteNumber=1792850" target="_top">Lisa BULLOCK</a></td><td>43:44</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>34.98 %</td><td>F</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">165</td><td><a href="athletehistory?athleteNumber=4515767" target="_top">Kaila SWYER</a></td><td>43:55</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>33.70 %</td><td>F</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">166</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">167</td><td><a href="athletehistory?athleteNumber=4272564" target="_top">Margaret HELLYER</a></td><td>44:30</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>41.24 %</td><td>F</td><td>77</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">168</td><td><a href="athletehistory?athleteNumber=4551304" target="_top">Belinda MEYER</a></td><td>45:19</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>34.46 %</td><td>F</td><td>78</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">169</td><td><a href="athletehistory?athleteNumber=4037877" target="_top">Sadia NAZIER</a></td><td>45:22</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>33.73 %</td><td>F</td><td>79</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:29</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">170</td><td><a href="athletehistory?athleteNumber=4052297" target="_top">Sharon SUMNER</a></td><td>45:54</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>36.09 %</td><td>F</td><td>80</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>19</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/25735002"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">171</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">172</td><td><a href="athletehistory?athleteNumber=4106933" target="_top">Rosemary SLUGGETT</a></td><td>46:09</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>41.46 %</td><td>F</td><td>81</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">173</td><td><a href="athletehistory?athleteNumber=3308365" target="_top">Pamela MATHERS</a></td><td>46:14</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>48.95 %</td><td>F</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:45:41</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">174</td><td><a href="athletehistory?athleteNumber=4542816" target="_top">Julia LIM</a></td><td>46:18</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>43.77 %</td><td>F</td><td>83</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">175</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">176</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">177</td><td><a href="athletehistory?athleteNumber=4358918" target="_top">Lais MATOS</a></td><td>47:27</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>31.19 %</td><td>F</td><td>84</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:29</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">178</td><td><a href="athletehistory?athleteNumber=4345698" target="_top">Peter DUDLEY</a></td><td>47:29</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>32.64 %</td><td>M</td><td>94</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:46:19</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">179</td><td><a href="athletehistory?athleteNumber=2867022" target="_top">Viviane RAMOS</a></td><td>47:42</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>32.74 %</td><td>F</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/2075388"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">180</td><td><a href="athletehistory?athleteNumber=4559524" target="_top">Sikander ALAM</a></td><td>47:51</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>28.11 %</td><td>M</td><td>95</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">181</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">182</td><td><a href="athletehistory?athleteNumber=4054641" target="_top">Mikayli MARLOWE</a></td><td>47:56</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>30.88 %</td><td>F</td><td>86</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:24</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">183</td><td><a href="athletehistory?athleteNumber=1737510" target="_top">Helen PARR</a></td><td>48:24</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>35.98 %</td><td>F</td><td>87</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:29:37</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11621051"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">184</td><td><a href="athletehistory?athleteNumber=1737530" target="_top">Derek PARR</a></td><td>48:25</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>32.29 %</td><td>M</td><td>97</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:27:36</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">185</td><td><a href="athletehistory?athleteNumber=4477406" target="_top">Pinki SHUKLA</a></td><td>49:40</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>30.60 %</td><td>F</td><td>88</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">186</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">187</td><td><a href="athletehistory?athleteNumber=1245122" target="_top">Pauline EDER</a></td><td>52:33</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>43.80 %</td><td>F</td><td>89</td><td><a href="../clubhistory?clubNum=23016">Heart Foundation Walking</a></td><td>PB stays at 00:46:19</td><td>69</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">188</td><td><a href="athletehistory?athleteNumber=1245151" target="_top">Herbert EDER</a></td><td>52:34</td><td><a href="../agecategorytable/?ageCat=VM85-89">VM85-89</a></td><td>47.91 %</td><td>M</td><td>99</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:04</td><td>72</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">189</td><td><a href="athletehistory?athleteNumber=1922742" target="_top">Luigi CARDACI</a></td><td>53:34</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>28.69 %</td><td>M</td><td>100</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:37</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">190</td><td><a href="athletehistory?athleteNumber=4037905" target="_top">Denzil CUPIDO</a></td><td>55:28</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>24.43 %</td><td>M</td><td>101</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:04</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">191</td><td><a href="athletehistory?athleteNumber=4425318" target="_top">Shaikh RAHMAN</a></td><td>56:00</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>23.84 %</td><td>M</td><td>102</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:18</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">192</td><td><a href="athletehistory?athleteNumber=4548439" target="_top">Christine DAWSON</a></td><td>56:49</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>29.51 %</td><td>F</td><td>90</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">193</td><td><a href="athletehistory?athleteNumber=136262" target="_top">Oyiela LITABA</a></td><td>1:00:56</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>29.32 %</td><td>F</td><td>91</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:37:43</td><td>209</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">194</td><td><a href="athletehistory?athleteNumber=2068233" target="_top">Leanne OBERIN</a></td><td>1:00:56</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>26.07 %</td><td>F</td><td>92</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:45:24</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">195</td><td><a href="athletehistory?athleteNumber=3286975" target="_top">Sonia BHATIA</a></td><td>1:01:01</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>27.48 %</td><td>F</td><td>93</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:56:42</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">196</td><td><a href="athletehistory?athleteNumber=1968284" target="_top">Eric NEGROS</a></td><td>1:02:08</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>24.73 %</td><td>M</td><td>103</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:52:13</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">197</td><td><a href="athletehistory?athleteNumber=1716573" target="_top">Nina MUSUMECI</a></td><td>1:02:26</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>24.64 %</td><td>F</td><td>94</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:23</td><td>87</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr></tbody></table></p><div class="paddedt left"><h3 class="paddedtandb">Thanks to the volunteers</h3><p class="paddedb">We are very grateful to the volunteers who made this event happen:Aaryan BHATIA, Ting CAO, Xiwu CHEN, Nanci MOORE, Nina MUSUMECI, Shanaaz NAZIER-CUPIDO, Joan PERRY, Nicole QUINN, Aidan QUINN</p><p class="paddedb">Why not <a href="../../volunteer/">get involved and volunteer</a>?</p><h3 class="paddetandb" id="explainTable">
	What is this table? </h3>
'''
run.addRunners(text)
run.addVolunteers(text)

#http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=148
text = '''
<table class="sortable" id="results" align="center"><thead><tr><th class="pos">Pos</th><th style="width: 150px">parkrunner</th><th>Time</th><th>Age Cat</th><th>Age Grade</th><th/><th>Gender Pos</th><th style="width: 130px">Club</th><th>Note</th><th colspan="2">Total Runs</th></tr></thead><tbody><tr><td class="pos">1</td><td><a href="athletehistory?athleteNumber=3523674" target="_top">Dane VERWEY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>21.67 %</td><td>M</td><td>1</td><td><a href="../clubhistory?clubNum=18092">Frankston AC</a></td><td>First Timer!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">2</td><td><a href="athletehistory?athleteNumber=2602577" target="_top">Leigh STOKES</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>22.26 %</td><td>M</td><td>2</td><td><a href="../clubhistory?clubNum=18070">Athletics Nunawading</a></td><td>PB stays at 00:17:28</td><td>25</td><td style="min-width:72px"/></tr><tr><td class="pos">3</td><td><a href="athletehistory?athleteNumber=1967289" target="_top">Taylor KEARNEY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>21.51 %</td><td>M</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:17:37</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">4</td><td><a href="athletehistory?athleteNumber=2070175" target="_top">Philip CHAMPION</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>25.20 %</td><td>M</td><td>4</td><td><a href="../clubhistory?clubNum=18082">Diamond Valley AC</a></td><td>First Timer!</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">5</td><td><a href="athletehistory?athleteNumber=492200" target="_top">Ross PENTLAND</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>21.53 %</td><td>M</td><td>5</td><td><a href="../clubhistory?clubNum=20544">Crosbie Crew</a></td><td>PB stays at 00:20:45</td><td>174</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7221490"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">6</td><td><a href="athletehistory?athleteNumber=975787" target="_top">Graham COUTTS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>24.20 %</td><td>M</td><td>6</td><td><a href="../clubhistory?clubNum=22136">Yarra Ranges Athletics</a></td><td>PB stays at 00:20:07</td><td>132</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7322163"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">7</td><td><a href="athletehistory?athleteNumber=214623" target="_top">Simon JARVIS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>21.51 %</td><td>M</td><td>7</td><td><a href="../clubhistory?clubNum=1980">Almost Athletes</a></td><td>First Timer!</td><td>253</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 250 Club" src="https://images.parkrun.com/website/results/250_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7368417"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">8</td><td><a href="athletehistory?athleteNumber=2336872" target="_top">Tim STICKLEY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>24.20 %</td><td>M</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:19:40</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/148656"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">9</td><td><a href="athletehistory?athleteNumber=4271309" target="_top">John NANKERVIS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>21.51 %</td><td>M</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:10</td><td>7</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/738302"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">10</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">11</td><td><a href="athletehistory?athleteNumber=3447233" target="_top">Sarah WALL</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>25.84 %</td><td>F</td><td>1</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:14</td><td>32</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">12</td><td><a href="athletehistory?athleteNumber=4474708" target="_top">Martin FRY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>24.40 %</td><td>M</td><td>11</td><td><a href="../clubhistory?clubNum=19905">Melbourne Midday Milers</a></td><td>First Timer!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">13</td><td><a href="athletehistory?athleteNumber=4162732" target="_top">Graeme DUNNING</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>24.20 %</td><td>M</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:49</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">14</td><td><a href="athletehistory?athleteNumber=1198604" target="_top">Miso BELEVSKI</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>24.01 %</td><td>M</td><td>13</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>First Timer!</td><td>56</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">15</td><td><a href="athletehistory?athleteNumber=2899338" target="_top">Phillip DANE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>26.29 %</td><td>M</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:35</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">16</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">17</td><td><a href="athletehistory?athleteNumber=1353114" target="_top">Stefan LESKE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>22.92 %</td><td>M</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:57</td><td>28</td><td style="min-width:72px"/></tr><tr><td class="pos">18</td><td><a href="athletehistory?athleteNumber=1740015" target="_top">Ross GLIBATSAS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>22.76 %</td><td>M</td><td>17</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:25:09</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">19</td><td><a href="athletehistory?athleteNumber=1919748" target="_top">Barry WOODS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.98 %</td><td>M</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:38</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">20</td><td><a href="athletehistory?athleteNumber=1434738" target="_top">Zoe DOWLING</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW18-19">SW18-19</a></td><td>24.76 %</td><td>F</td><td>2</td><td><a href="../clubhistory?clubNum=18085">Eastern Suburbs AC</a></td><td>First Timer!</td><td>18</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">21</td><td><a href="athletehistory?athleteNumber=1787340" target="_top">Julie BOYD</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>31.45 %</td><td>F</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">22</td><td><a href="athletehistory?athleteNumber=1927958" target="_top">John KIARIE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>23.81 %</td><td>M</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:06</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">23</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">24</td><td><a href="athletehistory?athleteNumber=233228" target="_top">Carla BROWN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>24.67 %</td><td>F</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:01</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">25</td><td><a href="athletehistory?athleteNumber=4108573" target="_top">Khalil AHMAD</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>21.76 %</td><td>M</td><td>21</td><td><a href="../clubhistory?clubNum=26916">Slow Burners Running Group</a></td><td>PB stays at 00:23:16</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">26</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">27</td><td><a href="athletehistory?athleteNumber=1345648" target="_top">Chris MCNEILL</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>22.76 %</td><td>M</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:56</td><td>133</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">28</td><td><a href="athletehistory?athleteNumber=2221609" target="_top">Megan PEREIRA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>26.62 %</td><td>F</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:04</td><td>78</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">29</td><td><a href="athletehistory?athleteNumber=2694977" target="_top">Alison CHAPMAN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>28.29 %</td><td>F</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:51</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">30</td><td><a href="athletehistory?athleteNumber=3201895" target="_top">Marianne PALMER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>24.67 %</td><td>F</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:30</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/8511811"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">31</td><td><a href="athletehistory?athleteNumber=4210599" target="_top">Elliana SALTALAMACCHIA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>24.67 %</td><td>F</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:42</td><td>13</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/13341061"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">32</td><td><a href="athletehistory?athleteNumber=4396615" target="_top">Gary CASSIDY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>22.92 %</td><td>M</td><td>24</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:20:17</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">33</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">34</td><td><a href="athletehistory?athleteNumber=3205411" target="_top">Ian SKELTON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>23.45 %</td><td>M</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:07</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">35</td><td><a href="athletehistory?athleteNumber=4147617" target="_top">Rebekah POWIERSKI</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>25.84 %</td><td>F</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:00</td><td>10</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">36</td><td><a href="athletehistory?athleteNumber=3120027" target="_top">Alex MOCHALOV</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>21.87 %</td><td>M</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:58</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">37</td><td><a href="athletehistory?athleteNumber=2766367" target="_top">Ross ASHTON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>26.51 %</td><td>M</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">38</td><td><a href="athletehistory?athleteNumber=2382370" target="_top">Scott PORTBURY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>23.09 %</td><td>M</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:37</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/4319859"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">39</td><td><a href="athletehistory?athleteNumber=4501566" target="_top">Cecai ZHANG</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.78 %</td><td>M</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">40</td><td><a href="athletehistory?athleteNumber=2728709" target="_top">Alex STAMA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>21.53 %</td><td>M</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:17</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">41</td><td><a href="athletehistory?athleteNumber=2384866" target="_top">Russell SLUGGETT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>21.51 %</td><td>M</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:11</td><td>28</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23324528"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">42</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">43</td><td><a href="athletehistory?athleteNumber=753229" target="_top">Robyn FLETCHER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>27.01 %</td><td>F</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:35</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">44</td><td><a href="athletehistory?athleteNumber=3828690" target="_top">Ray NG</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>21.56 %</td><td>M</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:35</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">45</td><td><a href="athletehistory?athleteNumber=3908146" target="_top">Myai PHAM</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>25.23 %</td><td>F</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">46</td><td><a href="athletehistory?athleteNumber=2276777" target="_top">Adam KOOPS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>21.76 %</td><td>M</td><td>35</td><td><a href="../clubhistory?clubNum=23644">Achilles Running Club Melbourne</a></td><td>PB stays at 00:20:15</td><td>88</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">47</td><td><a href="athletehistory?athleteNumber=2927404" target="_top">Simon REYMERS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>22.12 %</td><td>M</td><td>36</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:19:19</td><td>40</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/18022867"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">48</td><td><a href="athletehistory?athleteNumber=1040241" target="_top">Amy BUCKERFIELD</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>24.67 %</td><td>F</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:00</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">49</td><td><a href="athletehistory?athleteNumber=1065439" target="_top">Michael GOGERLY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>22.59 %</td><td>M</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:49</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">50</td><td><a href="athletehistory?athleteNumber=2856359" target="_top">Paulo RAMOS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>23.28 %</td><td>M</td><td>38</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:56</td><td>17</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/914269"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">51</td><td><a href="athletehistory?athleteNumber=3671091" target="_top">Michael WAN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>22.92 %</td><td>M</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>22</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23772417"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">52</td><td><a href="athletehistory?athleteNumber=3447301" target="_top">Arthur WALL</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.78 %</td><td>M</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:59</td><td>44</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/29599152"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">53</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">54</td><td><a href="athletehistory?athleteNumber=4418401" target="_top">Nathan CHARRETT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM15-17">JM15-17</a></td><td>23.20 %</td><td>M</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:35</td><td>4</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/3190363"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">55</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">56</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">57</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">58</td><td><a href="athletehistory?athleteNumber=4370064" target="_top">Braden MUNS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>21.73 %</td><td>M</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:59</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">59</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">60</td><td><a href="athletehistory?athleteNumber=3056576" target="_top">Venkat DODDI</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.98 %</td><td>M</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:53</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">61</td><td><a href="athletehistory?athleteNumber=2827378" target="_top">Serena NG</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW18-19">SW18-19</a></td><td>24.95 %</td><td>F</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:09</td><td>4</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/17817663"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">62</td><td><a href="athletehistory?athleteNumber=1750540" target="_top">Jake NGU</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.98 %</td><td>M</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:10</td><td>123</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11301806"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">63</td><td><a href="athletehistory?athleteNumber=386694" target="_top">Felicia RUSS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>24.67 %</td><td>F</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">64</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">65</td><td><a href="athletehistory?athleteNumber=4181093" target="_top">Kai HINGMANN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>23.45 %</td><td>M</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:22</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">66</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">67</td><td><a href="athletehistory?athleteNumber=4371920" target="_top">Jeff KENNY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>26.06 %</td><td>M</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:43</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">68</td><td><a href="athletehistory?athleteNumber=2336000" target="_top">Peter YEOMAN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>22.12 %</td><td>M</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:19</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">69</td><td><a href="athletehistory?athleteNumber=4114817" target="_top">Achintha JAYAWANTHA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM15-17">JM15-17</a></td><td>23.20 %</td><td>M</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">70</td><td><a href="athletehistory?athleteNumber=4114804" target="_top">Nik JAYAWANTHA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>23.65 %</td><td>M</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:26</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">71</td><td><a href="athletehistory?athleteNumber=4316933" target="_top">Nathaniel PARSONS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>21.51 %</td><td>M</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:53</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">72</td><td><a href="athletehistory?athleteNumber=1133854" target="_top">Vaughan DE RIGGS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>28.76 %</td><td>M</td><td>58</td><td><a href="../clubhistory?clubNum=22580">Pakenham Road Runners</a></td><td>PB stays at 00:24:56</td><td>173</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7542528"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">73</td><td><a href="athletehistory?athleteNumber=3678564" target="_top">Danni RAMALINGAM</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>26.73 %</td><td>F</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:12</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">74</td><td><a href="athletehistory?athleteNumber=3233492" target="_top">Sudhir KUMAR</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>23.65 %</td><td>M</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:47</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">75</td><td><a href="athletehistory?athleteNumber=2378428" target="_top">Owen PEREIRA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>29.67 %</td><td>M</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:50</td><td>71</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">76</td><td><a href="athletehistory?athleteNumber=2223986" target="_top">Emily PEREIRA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>28.45 %</td><td>F</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">77</td><td><a href="athletehistory?athleteNumber=2135357" target="_top">Tania STEWART</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>27.95 %</td><td>F</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">78</td><td><a href="athletehistory?athleteNumber=3294027" target="_top">Abhay SURYA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>23.65 %</td><td>M</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:36</td><td>20</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">79</td><td><a href="athletehistory?athleteNumber=1748710" target="_top">Matthew RUSHTON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>22.92 %</td><td>M</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:08</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">80</td><td><a href="athletehistory?athleteNumber=4016990" target="_top">John WATTS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.59 %</td><td>M</td><td>63</td><td><a href="../clubhistory?clubNum=21915">Tri Alliance</a></td><td>PB stays at 00:27:37</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">81</td><td><a href="athletehistory?athleteNumber=4488709" target="_top">Shaurya SHUKLA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>26.31 %</td><td>M</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:56</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">82</td><td><a href="athletehistory?athleteNumber=2369510" target="_top">Jackie MITCHELL</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>29.40 %</td><td>F</td><td>18</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:26:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">83</td><td><a href="athletehistory?athleteNumber=3373087" target="_top">Nino MASCETTA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>26.73 %</td><td>M</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:44</td><td>44</td><td style="min-width:72px"/></tr><tr><td class="pos">84</td><td><a href="athletehistory?athleteNumber=2765339" target="_top">Duncan WHITE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.78 %</td><td>M</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">85</td><td><a href="athletehistory?athleteNumber=1737530" target="_top">Derek PARR</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>26.06 %</td><td>M</td><td>67</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:27:36</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">86</td><td><a href="athletehistory?athleteNumber=4412009" target="_top">Rohan CHARRETT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>24.01 %</td><td>M</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:15</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">87</td><td><a href="athletehistory?athleteNumber=4043596" target="_top">Suzanne EVANS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>34.84 %</td><td>F</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:58</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">88</td><td><a href="athletehistory?athleteNumber=1947254" target="_top">Ann-Maree BAKER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>26.04 %</td><td>F</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:01</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">89</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">90</td><td><a href="athletehistory?athleteNumber=4011261" target="_top">Roger HE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>23.65 %</td><td>M</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:20</td><td>18</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">91</td><td><a href="athletehistory?athleteNumber=4270234" target="_top">Eleassa PRESTON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>26.04 %</td><td>F</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:47</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">92</td><td><a href="athletehistory?athleteNumber=2187279" target="_top">Nanci MOORE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>31.90 %</td><td>F</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:59</td><td>51</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">93</td><td><a href="athletehistory?athleteNumber=2137630" target="_top">Mark STEWART</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>24.78 %</td><td>M</td><td>71</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:23</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">94</td><td><a href="athletehistory?athleteNumber=4290916" target="_top">Chin LIM</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>28.23 %</td><td>M</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:49</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/9773029"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">95</td><td><a href="athletehistory?athleteNumber=4261472" target="_top">Elaine TOR</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>24.67 %</td><td>F</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:40</td><td>7</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/26134133"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">96</td><td><a href="athletehistory?athleteNumber=3588740" target="_top">Josh DIEMAR</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>21.98 %</td><td>M</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:13</td><td>15</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/22594433"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">97</td><td><a href="athletehistory?athleteNumber=1882862" target="_top">Gregory MOORE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>23.65 %</td><td>M</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:22</td><td>69</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">98</td><td><a href="athletehistory?athleteNumber=4269724" target="_top">Jordan BROWN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>21.51 %</td><td>M</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:33</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">99</td><td><a href="athletehistory?athleteNumber=2428952" target="_top">Terry RANDALL</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>26.51 %</td><td>M</td><td>76</td><td><a href="../clubhistory?clubNum=22580">Pakenham Road Runners</a></td><td>PB stays at 00:27:50</td><td>91</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/15367956"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">100</td><td><a href="athletehistory?athleteNumber=1744665" target="_top">Maree DALZOTTO</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>28.65 %</td><td>F</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:21</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">101</td><td><a href="athletehistory?athleteNumber=4090905" target="_top">Natasha TRINKLE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>25.09 %</td><td>F</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:24</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">102</td><td><a href="athletehistory?athleteNumber=1736959" target="_top">Jock CRAVANA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM80-84">VM80-84</a></td><td>35.04 %</td><td>M</td><td>77</td><td><a href="../clubhistory?clubNum=18165">Victorian Masters Athletics</a></td><td>PB stays at 00:28:23</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">103</td><td><a href="athletehistory?athleteNumber=1822194" target="_top">Alexander ROBERTSON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>23.65 %</td><td>M</td><td>78</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:18</td><td>19</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">104</td><td><a href="athletehistory?athleteNumber=4526607" target="_top">Sam TYRER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>21.56 %</td><td>M</td><td>79</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">105</td><td><a href="athletehistory?athleteNumber=2777633" target="_top">Nathan SCOTT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>26.31 %</td><td>M</td><td>80</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:53</td><td>66</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">106</td><td><a href="athletehistory?athleteNumber=2777641" target="_top">Andrew SCOTT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>22.92 %</td><td>M</td><td>81</td><td><a href="../clubhistory?clubNum=21413">Derek Zoolander Centre for Kids Who Can't Run Good</a></td><td>PB stays at 00:24:49</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/199341"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">107</td><td><a href="athletehistory?athleteNumber=4368476" target="_top">Grant BROWN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>25.40 %</td><td>M</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:12</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">108</td><td><a href="athletehistory?athleteNumber=1928336" target="_top">Rebecca BUNYON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>26.23 %</td><td>F</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:46</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">109</td><td><a href="athletehistory?athleteNumber=3292314" target="_top">Laura DE RANGO</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>24.67 %</td><td>F</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:54</td><td>23</td><td style="min-width:72px"/></tr><tr><td class="pos">110</td><td><a href="athletehistory?athleteNumber=1745657" target="_top">Marianne SMITH</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>29.79 %</td><td>F</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:03</td><td>89</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">111</td><td><a href="athletehistory?athleteNumber=4396444" target="_top">Stephen John JEFFERY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>25.20 %</td><td>M</td><td>83</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:43</td><td>4</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/3405039"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">112</td><td><a href="athletehistory?athleteNumber=4521874" target="_top">Angel Lu JIAO</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>27.29 %</td><td>F</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">113</td><td><a href="athletehistory?athleteNumber=2345651" target="_top">Jackie DOWLING</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>30.18 %</td><td>F</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:36</td><td>43</td><td style="min-width:72px"/></tr><tr><td class="pos">114</td><td><a href="athletehistory?athleteNumber=2518028" target="_top">Jay WRIGHT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>31.45 %</td><td>F</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:55</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">115</td><td><a href="athletehistory?athleteNumber=2727180" target="_top">Pauline PAINE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>25.51 %</td><td>F</td><td>32</td><td><a href="../clubhistory?clubNum=23836">Human Performance Development</a></td><td>PB stays at 00:31:13</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23806054"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">116</td><td><a href="athletehistory?athleteNumber=3970628" target="_top">Marina LI</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>27.29 %</td><td>F</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:27</td><td>27</td><td style="min-width:72px"/></tr><tr><td class="pos">117</td><td><a href="athletehistory?athleteNumber=1928330" target="_top">Kaiya BUNYON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>27.73 %</td><td>F</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:59</td><td>11</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">118</td><td><a href="athletehistory?athleteNumber=1735852" target="_top">Josephine CARDACI</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>27.95 %</td><td>F</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:20</td><td>117</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">119</td><td><a href="athletehistory?athleteNumber=3284734" target="_top">Kate WILKINS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>31.01 %</td><td>F</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:23</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">120</td><td><a href="athletehistory?athleteNumber=3270886" target="_top">Roger SLADE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>30.15 %</td><td>M</td><td>84</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:04</td><td>28</td><td style="min-width:72px"/></tr><tr><td class="pos">121</td><td><a href="athletehistory?athleteNumber=3510394" target="_top">Donna KENNY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>28.65 %</td><td>F</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:29</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">122</td><td><a href="athletehistory?athleteNumber=2221582" target="_top">Rohini PEREIRA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>25.84 %</td><td>F</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:50</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">123</td><td><a href="athletehistory?athleteNumber=4529236" target="_top">Jonathan STONE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>21.51 %</td><td>M</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">124</td><td><a href="athletehistory?athleteNumber=2819837" target="_top">Toban WILD</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>21.51 %</td><td>M</td><td>86</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:00</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">125</td><td><a href="athletehistory?athleteNumber=2428953" target="_top">Janne RANDALL</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>31.90 %</td><td>F</td><td>39</td><td><a href="../clubhistory?clubNum=22580">Pakenham Road Runners</a></td><td>PB stays at 00:33:11</td><td>81</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">126</td><td><a href="athletehistory?athleteNumber=2083536" target="_top">Jenny JAMES</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>33.29 %</td><td>F</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:20</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">127</td><td><a href="athletehistory?athleteNumber=2135388" target="_top">Stephen WHITE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>25.62 %</td><td>M</td><td>87</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:29</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">128</td><td><a href="athletehistory?athleteNumber=3652092" target="_top">Leonora POWRIE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>33.79 %</td><td>F</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:25</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">129</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">130</td><td><a href="athletehistory?athleteNumber=1786065" target="_top">Tina TIAN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>26.73 %</td><td>F</td><td>42</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:27:48</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">131</td><td><a href="athletehistory?athleteNumber=1344650" target="_top">Clare BLACKSTOCK</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>26.73 %</td><td>F</td><td>43</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:31:10</td><td>97</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/7649007"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">132</td><td><a href="athletehistory?athleteNumber=1344947" target="_top">Shawn DUFF</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>22.92 %</td><td>M</td><td>89</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:42</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">133</td><td><a href="athletehistory?athleteNumber=1049384" target="_top">Nilmini JAYASURIYA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>27.29 %</td><td>F</td><td>44</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:34:29</td><td>47</td><td style="min-width:72px"/></tr><tr><td class="pos">134</td><td><a href="athletehistory?athleteNumber=4269919" target="_top">Cathy CANNON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>33.79 %</td><td>F</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:00</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">135</td><td><a href="athletehistory?athleteNumber=4516104" target="_top">Maryanne MCCONNELL</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>29.40 %</td><td>F</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">136</td><td><a href="athletehistory?athleteNumber=2397940" target="_top">Jean RUDD</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>33.79 %</td><td>F</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:21</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">137</td><td><a href="athletehistory?athleteNumber=2207985" target="_top">Andrew WEATHERHEAD</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>28.76 %</td><td>M</td><td>90</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:16</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">138</td><td><a href="athletehistory?athleteNumber=2769381" target="_top">Marina JORDAN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>28.65 %</td><td>F</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:34</td><td>77</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">139</td><td><a href="athletehistory?athleteNumber=1075619" target="_top">Jacquelene CROW</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>30.59 %</td><td>F</td><td>49</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:34:08</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">140</td><td><a href="athletehistory?athleteNumber=3212945" target="_top">Graham ROBERTSON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>30.59 %</td><td>M</td><td>91</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:06</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">141</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">142</td><td><a href="athletehistory?athleteNumber=3985226" target="_top">Joy YIN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>29.26 %</td><td>F</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:41</td><td>12</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">143</td><td><a href="athletehistory?athleteNumber=2777655" target="_top">Toby SCOTT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>28.37 %</td><td>M</td><td>93</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:22</td><td>68</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/25563106"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">144</td><td><a href="athletehistory?athleteNumber=3591013" target="_top">Robyn DIMOPOULOS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>24.84 %</td><td>F</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:13</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">145</td><td><a href="athletehistory?athleteNumber=2777601" target="_top">Christine KAUCNER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>27.29 %</td><td>F</td><td>52</td><td><a href="../clubhistory?clubNum=21832">Running Mums Australia</a></td><td>PB stays at 00:26:16</td><td>76</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/12533560"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">146</td><td><a href="athletehistory?athleteNumber=1717831" target="_top">Thomas SOM</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>25.40 %</td><td>M</td><td>94</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:17</td><td>95</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">147</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">148</td><td><a href="athletehistory?athleteNumber=4451023" target="_top">Kathir APPADURAI</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>23.81 %</td><td>M</td><td>96</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:41</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">149</td><td><a href="athletehistory?athleteNumber=4442556" target="_top">Bernie HEV</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>24.92 %</td><td>F</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">150</td><td><a href="athletehistory?athleteNumber=4453453" target="_top">Rochelle WHEATLEY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>25.09 %</td><td>F</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:50</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">151</td><td><a href="athletehistory?athleteNumber=1783047" target="_top">Helen TAYLOR</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>28.29 %</td><td>F</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:36</td><td>32</td><td style="min-width:72px"/></tr><tr><td class="pos">152</td><td><a href="athletehistory?athleteNumber=4366323" target="_top">Jessna ERUVASSERY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>25.09 %</td><td>F</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">153</td><td><a href="athletehistory?athleteNumber=3971595" target="_top">Sue BROWN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>27.95 %</td><td>F</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:38:32</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">154</td><td><a href="athletehistory?athleteNumber=1834605" target="_top">Marlene HANSFORD</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>29.79 %</td><td>F</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:24</td><td>71</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">155</td><td><a href="athletehistory?athleteNumber=3051236" target="_top">Jane TAYLOR</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>37.73 %</td><td>F</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:37:26</td><td>40</td><td style="min-width:72px"/></tr><tr><td class="pos">156</td><td><a href="athletehistory?athleteNumber=3803840" target="_top">Annie CHEN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>29.79 %</td><td>F</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:41</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">157</td><td><a href="athletehistory?athleteNumber=2207989" target="_top">Mary WEATHERHEAD</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>34.84 %</td><td>F</td><td>61</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">158</td><td><a href="athletehistory?athleteNumber=1745665" target="_top">Katie CROSS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>30.18 %</td><td>F</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:09</td><td>61</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">159</td><td><a href="athletehistory?athleteNumber=4363576" target="_top">Ann-Marie JEFFERY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>28.29 %</td><td>F</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:33</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">160</td><td><a href="athletehistory?athleteNumber=4358918" target="_top">Lais MATOS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>24.67 %</td><td>F</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:29</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">161</td><td><a href="athletehistory?athleteNumber=2202224" target="_top">Alex ROBINS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM75-79">VM75-79</a></td><td>32.15 %</td><td>M</td><td>97</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:27</td><td>100</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">162</td><td><a href="athletehistory?athleteNumber=3438449" target="_top">Allen PRATT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>29.06 %</td><td>M</td><td>98</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:52</td><td>34</td><td style="min-width:72px"/></tr><tr><td class="pos">163</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">164</td><td><a href="athletehistory?athleteNumber=4322666" target="_top">Christine PATERSON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>34.84 %</td><td>F</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:42:10</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">165</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">166</td><td><a href="athletehistory?athleteNumber=1792850" target="_top">Lisa BULLOCK</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>25.51 %</td><td>F</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">167</td><td><a href="athletehistory?athleteNumber=4225423" target="_top">Richard SLUGGETT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM60-64">VM60-64</a></td><td>27.45 %</td><td>M</td><td>101</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:42:22</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">168</td><td><a href="athletehistory?athleteNumber=4515767" target="_top">Kaila SWYER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>24.67 %</td><td>F</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">169</td><td><a href="athletehistory?athleteNumber=3314054" target="_top">Ken MATHERS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>30.15 %</td><td>M</td><td>102</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:43:42</td><td>21</td><td style="min-width:72px"/></tr><tr><td class="pos">170</td><td><a href="athletehistory?athleteNumber=4510967" target="_top">Shiva NEUPANE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>23.81 %</td><td>M</td><td>103</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">171</td><td><a href="athletehistory?athleteNumber=4272564" target="_top">Margaret HELLYER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>30.59 %</td><td>F</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:43:11</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">172</td><td><a href="athletehistory?athleteNumber=4510971" target="_top">Sharada NEUPANE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>26.73 %</td><td>F</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">173</td><td><a href="athletehistory?athleteNumber=4345698" target="_top">Peter DUDLEY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>25.84 %</td><td>M</td><td>104</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:46:19</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">174</td><td><a href="athletehistory?athleteNumber=2744351" target="_top">Fiona DAVIS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>27.01 %</td><td>F</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:43:10</td><td>79</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">175</td><td><a href="athletehistory?athleteNumber=4385697" target="_top">Janine OMALLEY</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>29.79 %</td><td>F</td><td>71</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">176</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">177</td><td><a href="athletehistory?athleteNumber=4106933" target="_top">Rosemary SLUGGETT</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>31.90 %</td><td>F</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:44:37</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">178</td><td><a href="athletehistory?athleteNumber=2068233" target="_top">Leanne OBERIN</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>26.48 %</td><td>F</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:45:24</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">179</td><td><a href="athletehistory?athleteNumber=4272066" target="_top">Danielle SALTALAMACCHIA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>27.29 %</td><td>F</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:25</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">180</td><td><a href="athletehistory?athleteNumber=3308365" target="_top">Pamela MATHERS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>37.73 %</td><td>F</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:45:41</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">181</td><td><a href="athletehistory?athleteNumber=4395237" target="_top">Carol NOBLET</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>34.32 %</td><td>F</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:48:29</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">182</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">183</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">184</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">185</td><td><a href="athletehistory?athleteNumber=4037850" target="_top">Shanaaz NAZIER-CUPIDO</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>25.09 %</td><td>F</td><td>77</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:48:37</td><td>23</td><td style="min-width:72px"/></tr><tr><td class="pos">186</td><td><a href="athletehistory?athleteNumber=4237030" target="_top">Michael RAMAGE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>28.48 %</td><td>M</td><td>109</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:38:00</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">187</td><td><a href="athletehistory?athleteNumber=3975800" target="_top">Ken JAMES</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>29.76 %</td><td>M</td><td>110</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:53</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">188</td><td><a href="athletehistory?athleteNumber=3704089" target="_top">Louise ROBERTSON</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>24.78 %</td><td>F</td><td>78</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:45:41</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">189</td><td><a href="athletehistory?athleteNumber=4053498" target="_top">Helen MANTEL</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>35.37 %</td><td>F</td><td>79</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:46:00</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">190</td><td><a href="athletehistory?athleteNumber=3179070" target="_top">Ting CAO</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>22.26 %</td><td>M</td><td>111</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:21</td><td>24</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5244204"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">191</td><td><a href="athletehistory?athleteNumber=1882866" target="_top">Elijah SUMMERS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>28.37 %</td><td>M</td><td>112</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:01</td><td>72</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">192</td><td><a href="athletehistory?athleteNumber=569294" target="_top">Lizzie SUMMERS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>25.34 %</td><td>F</td><td>80</td><td><a href="../clubhistory?clubNum=21990">UP Coaching</a></td><td>PB stays at 00:24:00</td><td>75</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">193</td><td><a href="athletehistory?athleteNumber=1245122" target="_top">Pauline EDER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>38.37 %</td><td>F</td><td>81</td><td><a href="../clubhistory?clubNum=23016">Heart Foundation Walking</a></td><td>PB stays at 00:46:19</td><td>69</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">194</td><td><a href="athletehistory?athleteNumber=1245151" target="_top">Herbert EDER</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM85-89">VM85-89</a></td><td>41.98 %</td><td>M</td><td>113</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:04</td><td>72</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">195</td><td><a href="athletehistory?athleteNumber=4477406" target="_top">Pinki SHUKLA</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>25.34 %</td><td>F</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:49:40</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">196</td><td><a href="athletehistory?athleteNumber=4237014" target="_top">Marie RAMAGE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>35.37 %</td><td>F</td><td>83</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:50</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">197</td><td><a href="athletehistory?athleteNumber=4037905" target="_top">Denzil CUPIDO</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>22.59 %</td><td>M</td><td>114</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:04</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">198</td><td><a href="athletehistory?athleteNumber=3004293" target="_top">Carolyn WHITE</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>27.29 %</td><td>F</td><td>84</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:44:47</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">199</td><td><a href="athletehistory?athleteNumber=1968284" target="_top">Eric NEGROS</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>25.62 %</td><td>M</td><td>115</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:52:13</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">200</td><td><a href="athletehistory?athleteNumber=1915242" target="_top">Brian LYNCH</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VM80-84">VM80-84</a></td><td>37.98 %</td><td>M</td><td>116</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:38:13</td><td>35</td><td style="min-width:72px"/></tr><tr><td class="pos">201</td><td><a href="athletehistory?athleteNumber=1716573" target="_top">Nina MUSUMECI</a></td><td>59:59</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>25.65 %</td><td>F</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:23</td><td>87</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr></tbody></table></p><div class="paddedt left"><h3 class="paddedtandb">Thanks to the volunteers</h3><p class="paddedb">We are very grateful to the volunteers who made this event happen:Michael BLEWETT, Naomi (Tullae) CROTTY, Kyra LAWRENCE, Mikayli MARLOWE, Gregory MOORE, Nina MUSUMECI, Evan ROBERTSON, Nicki WHITFIELD, Jing YE</p><p class="paddedb">Why not <a href="../../volunteer/">get involved and volunteer</a>?</p><h3 class="paddetandb" id="explainTable">
	What is this table? </h3>
'''
run.addRunners(text)
run.addVolunteers(text)

#http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=147
text = '''
<table class="sortable" id="results" align="center"><thead><tr><th class="pos">Pos</th><th style="width: 150px">parkrunner</th><th>Time</th><th>Age Cat</th><th>Age Grade</th><th/><th>Gender Pos</th><th style="width: 130px">Club</th><th>Note</th><th colspan="2">Total Runs</th></tr></thead><tbody><tr><td class="pos">1</td><td><a href="athletehistory?athleteNumber=1344427" target="_top">David VENOUR</a></td><td>17:32</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>83.46 %</td><td>M</td><td>1</td><td><a href="../clubhistory?clubNum=19905">Melbourne Midday Milers</a></td><td>PB stays at 00:16:53</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">2</td><td><a href="athletehistory?athleteNumber=3558536" target="_top">Long NGUYEN</a></td><td>17:53</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>72.13 %</td><td>M</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:17:42</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">3</td><td><a href="athletehistory?athleteNumber=1967289" target="_top">Taylor KEARNEY</a></td><td>18:02</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>71.53 %</td><td>M</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">4</td><td><a href="athletehistory?athleteNumber=3691268" target="_top">Callum MCDONALD</a></td><td>18:49</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>68.56 %</td><td>M</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>30</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23436720"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">5</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">6</td><td><a href="athletehistory?athleteNumber=3825324" target="_top">Josh FRANKLIN</a></td><td>20:14</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>63.76 %</td><td>M</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:19:49</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">7</td><td><a href="athletehistory?athleteNumber=4271309" target="_top">John NANKERVIS</a></td><td>20:23</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>63.29 %</td><td>M</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:10</td><td>7</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/738302"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">8</td><td><a href="athletehistory?athleteNumber=4434352" target="_top">Ben HEYS</a></td><td>20:25</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>64.57 %</td><td>M</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/17980032"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">9</td><td><a href="athletehistory?athleteNumber=1356829" target="_top">Evan ROBERTSON</a></td><td>20:35</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>70.53 %</td><td>M</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:19</td><td>64</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">10</td><td><a href="athletehistory?athleteNumber=3447233" target="_top">Sarah WALL</a></td><td>20:51</td><td><a href="../agecategorytable/?ageCat=JW15-17">JW15-17</a></td><td>74.34 %</td><td>F</td><td>1</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:14</td><td>32</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">11</td><td><a href="athletehistory?athleteNumber=2655215" target="_top">Dominic MASCETTA</a></td><td>20:59</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>61.80 %</td><td>M</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>60</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/26826765"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">12</td><td><a href="athletehistory?athleteNumber=2395752" target="_top">Michael COLLINS</a></td><td>21:42</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>69.05 %</td><td>M</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:19</td><td>46</td><td style="min-width:72px"/></tr><tr><td class="pos">13</td><td><a href="athletehistory?athleteNumber=2899338" target="_top">Phillip DANE</a></td><td>22:10</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>71.13 %</td><td>M</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>45</td><td style="min-width:72px"/></tr><tr><td class="pos">14</td><td><a href="athletehistory?athleteNumber=1779095" target="_top">Ray TERNES</a></td><td>22:41</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>57.83 %</td><td>M</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:04</td><td>47</td><td style="min-width:72px"/></tr><tr><td class="pos">15</td><td><a href="athletehistory?athleteNumber=2593832" target="_top">Wei-Han KUAN</a></td><td>22:44</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>62.39 %</td><td>M</td><td>14</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:22:00</td><td>49</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/20106243"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">16</td><td><a href="athletehistory?athleteNumber=4495527" target="_top">Jason SMITH</a></td><td>23:18</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>55.36 %</td><td>M</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">17</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">18</td><td><a href="athletehistory?athleteNumber=4449056" target="_top">Ben KEATING</a></td><td>23:31</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>56.41 %</td><td>M</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">19</td><td><a href="athletehistory?athleteNumber=1927958" target="_top">John KIARIE</a></td><td>23:32</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>60.69 %</td><td>M</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:06</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">20</td><td><a href="athletehistory?athleteNumber=4162217" target="_top">Daniel CHEN</a></td><td>23:33</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>64.97 %</td><td>M</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:21:25</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">21</td><td><a href="athletehistory?athleteNumber=2668532" target="_top">Tyler CARTLEDGE</a></td><td>23:36</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>72.10 %</td><td>M</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>30</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">22</td><td><a href="athletehistory?athleteNumber=4449070" target="_top">Tina MCCARTHY</a></td><td>23:40</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>62.96 %</td><td>F</td><td>2</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:28</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">23</td><td><a href="athletehistory?athleteNumber=4210599" target="_top">Elliana SALTALAMACCHIA</a></td><td>23:42</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>62.45 %</td><td>F</td><td>3</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>13</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/13341061"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">24</td><td><a href="athletehistory?athleteNumber=1765634" target="_top">Kyle BIBBY</a></td><td>23:54</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>59.34 %</td><td>M</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:05</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">25</td><td><a href="athletehistory?athleteNumber=4314582" target="_top">Hugh MCDONALD</a></td><td>23:55</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>61.18 %</td><td>M</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">26</td><td><a href="athletehistory?athleteNumber=1345648" target="_top">Chris MCNEILL</a></td><td>23:59</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>56.91 %</td><td>M</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:56</td><td>133</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">27</td><td><a href="athletehistory?athleteNumber=233228" target="_top">Carla BROWN</a></td><td>24:04</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>61.50 %</td><td>F</td><td>4</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:01</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">28</td><td><a href="athletehistory?athleteNumber=1928321" target="_top">Peter BUNYON</a></td><td>24:15</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>55.88 %</td><td>M</td><td>24</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:11</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">29</td><td><a href="athletehistory?athleteNumber=3120027" target="_top">Alex MOCHALOV</a></td><td>24:24</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>53.76 %</td><td>M</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:58</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">30</td><td><a href="athletehistory?athleteNumber=2443307" target="_top">Alan TAYLOR</a></td><td>24:27</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>57.53 %</td><td>M</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:17</td><td>58</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">31</td><td><a href="athletehistory?athleteNumber=2009396" target="_top">Andrew POMPHREY</a></td><td>24:32</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>59.17 %</td><td>M</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>44</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7127795"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">32</td><td><a href="athletehistory?athleteNumber=3205411" target="_top">Ian SKELTON</a></td><td>24:34</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>57.26 %</td><td>M</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:07</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">33</td><td><a href="athletehistory?athleteNumber=2382370" target="_top">Scott PORTBURY</a></td><td>24:40</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>56.15 %</td><td>M</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:37</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/4319859"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">34</td><td><a href="athletehistory?athleteNumber=3828690" target="_top">Ray NG</a></td><td>24:41</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>52.40 %</td><td>M</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:35</td><td>24</td><td style="min-width:72px"/></tr><tr><td class="pos">35</td><td><a href="athletehistory?athleteNumber=2221609" target="_top">Megan PEREIRA</a></td><td>24:45</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>64.51 %</td><td>F</td><td>5</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:04</td><td>78</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">36</td><td><a href="athletehistory?athleteNumber=1433534" target="_top">Declan DEMPSTER</a></td><td>25:02</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>51.53 %</td><td>M</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">37</td><td><a href="athletehistory?athleteNumber=2728709" target="_top">Alex STAMA</a></td><td>25:10</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>51.32 %</td><td>M</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:17</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">38</td><td><a href="athletehistory?athleteNumber=1915199" target="_top">Wayne JEWELL</a></td><td>25:25</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>60.46 %</td><td>M</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:39</td><td>76</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11496423"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">39</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">40</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">41</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">42</td><td><a href="athletehistory?athleteNumber=1858121" target="_top">Asher BUNYON</a></td><td>25:55</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>57.36 %</td><td>M</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:53</td><td>19</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">43</td><td><a href="athletehistory?athleteNumber=2363085" target="_top">Bill BROWN</a></td><td>25:58</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>65.21 %</td><td>M</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:13</td><td>33</td><td style="min-width:72px"/></tr><tr><td class="pos">44</td><td><a href="athletehistory?athleteNumber=4490875" target="_top">David IRVING</a></td><td>26:15</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>50.22 %</td><td>M</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>5</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23969193"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">45</td><td><a href="athletehistory?athleteNumber=4500454" target="_top">David COE</a></td><td>26:29</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>51.16 %</td><td>M</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">46</td><td><a href="athletehistory?athleteNumber=3447301" target="_top">Arthur WALL</a></td><td>26:31</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>56.07 %</td><td>M</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:59</td><td>44</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/29599152"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">47</td><td><a href="athletehistory?athleteNumber=1804566" target="_top">Chris SHANNON</a></td><td>26:36</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>55.45 %</td><td>M</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:12</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">48</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">49</td><td><a href="athletehistory?athleteNumber=3671091" target="_top">Michael WAN</a></td><td>26:51</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>51.21 %</td><td>M</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>22</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/23772417"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">50</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">51</td><td><a href="athletehistory?athleteNumber=3179070" target="_top">Ting CAO</a></td><td>27:15</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>48.99 %</td><td>M</td><td>46</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:26:21</td><td>24</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/5244204"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">52</td><td><a href="athletehistory?athleteNumber=2647705" target="_top">Michael BLEWETT</a></td><td>27:20</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>53.96 %</td><td>M</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:24</td><td>34</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/12444211"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">53</td><td><a href="athletehistory?athleteNumber=2336000" target="_top">Peter YEOMAN</a></td><td>27:21</td><td><a href="../agecategorytable/?ageCat=VM35-39">VM35-39</a></td><td>48.51 %</td><td>M</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">54</td><td><a href="athletehistory?athleteNumber=4054781" target="_top">Nick LI</a></td><td>27:27</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>51.67 %</td><td>M</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:04</td><td>11</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">55</td><td><a href="athletehistory?athleteNumber=4114804" target="_top">Nik JAYAWANTHA</a></td><td>27:29</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.61 %</td><td>M</td><td>50</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">56</td><td><a href="athletehistory?athleteNumber=4345657" target="_top">Camilla DUDLEY</a></td><td>27:31</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>66.69 %</td><td>F</td><td>6</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">57</td><td><a href="athletehistory?athleteNumber=1748710" target="_top">Matthew RUSHTON</a></td><td>27:32</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>49.94 %</td><td>M</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:08</td><td>106</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">58</td><td><a href="athletehistory?athleteNumber=3368539" target="_top">Courtney SPLATT</a></td><td>27:36</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>53.86 %</td><td>F</td><td>7</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:53</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">59</td><td><a href="athletehistory?athleteNumber=4316933" target="_top">Nathaniel PARSONS</a></td><td>27:42</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>46.57 %</td><td>M</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">60</td><td><a href="athletehistory?athleteNumber=4181093" target="_top">Kai HINGMANN</a></td><td>27:56</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>50.36 %</td><td>M</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:22</td><td>13</td><td style="min-width:72px"/></tr><tr><td class="pos">61</td><td><a href="athletehistory?athleteNumber=3960435" target="_top">B G</a></td><td>27:58</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.07 %</td><td>M</td><td>54</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:19</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">62</td><td><a href="athletehistory?athleteNumber=3313890" target="_top">Nicholas BALL</a></td><td>28:00</td><td><a href="../agecategorytable/?ageCat=SM20-24">SM20-24</a></td><td>46.07 %</td><td>M</td><td>55</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">63</td><td><a href="athletehistory?athleteNumber=3971716" target="_top">Kathy ZUCKER</a></td><td>28:03</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>67.26 %</td><td>F</td><td>8</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">64</td><td><a href="athletehistory?athleteNumber=3249513" target="_top">Kashyp BHATIA</a></td><td>28:06</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>51.66 %</td><td>M</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:59</td><td>39</td><td style="min-width:72px"/></tr><tr><td class="pos">65</td><td><a href="athletehistory?athleteNumber=3233492" target="_top">Sudhir KUMAR</a></td><td>28:09</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.38 %</td><td>M</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:23:47</td><td>30</td><td style="min-width:72px"/></tr><tr><td class="pos">66</td><td><a href="athletehistory?athleteNumber=2832517" target="_top">Bethan REES</a></td><td>28:25</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>52.08 %</td><td>F</td><td>9</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:20</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">67</td><td><a href="athletehistory?athleteNumber=2765339" target="_top">Duncan WHITE</a></td><td>28:27</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>52.26 %</td><td>M</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">68</td><td><a href="athletehistory?athleteNumber=1585499" target="_top">Trevor STAFFORD</a></td><td>28:30</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.94 %</td><td>M</td><td>59</td><td><a href="../clubhistory?clubNum=25510">Dude, Where's My Shoes</a></td><td>First Timer!</td><td>131</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/9714850"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">69</td><td><a href="athletehistory?athleteNumber=2135357" target="_top">Tania STEWART</a></td><td>28:38</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>58.56 %</td><td>F</td><td>10</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:05</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">70</td><td><a href="athletehistory?athleteNumber=4090888" target="_top">Hans TRINKLE</a></td><td>28:39</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>50.26 %</td><td>M</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">71</td><td><a href="athletehistory?athleteNumber=2634261" target="_top">Karen TAYLOR</a></td><td>28:39</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>55.44 %</td><td>F</td><td>11</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:01</td><td>15</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/3773986"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">72</td><td><a href="athletehistory?athleteNumber=2095309" target="_top">Rosemary WAGHORN</a></td><td>28:45</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>55.25 %</td><td>F</td><td>12</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:12</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">73</td><td><a href="athletehistory?athleteNumber=2285678" target="_top">Nicki WHITFIELD</a></td><td>28:52</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>58.78 %</td><td>F</td><td>13</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:06</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">74</td><td><a href="athletehistory?athleteNumber=3470293" target="_top">Claire SMALE</a></td><td>29:01</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>51.35 %</td><td>F</td><td>14</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">75</td><td><a href="athletehistory?athleteNumber=4016990" target="_top">John WATTS</a></td><td>29:10</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>50.57 %</td><td>M</td><td>61</td><td><a href="../clubhistory?clubNum=21915">Tri Alliance</a></td><td>PB stays at 00:27:37</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">76</td><td><a href="athletehistory?athleteNumber=2395670" target="_top">Wendy SHEN</a></td><td>29:16</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>57.29 %</td><td>F</td><td>15</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:29</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">77</td><td><a href="athletehistory?athleteNumber=1952769" target="_top">Roger DICKINSON</a></td><td>29:18</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>51.99 %</td><td>M</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>66</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">78</td><td><a href="athletehistory?athleteNumber=4270234" target="_top">Eleassa PRESTON</a></td><td>29:18</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>53.30 %</td><td>F</td><td>16</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>10</td><td style="min-width:72px"/></tr><tr><td class="pos">79</td><td><a href="athletehistory?athleteNumber=4145404" target="_top">Sam KAR</a></td><td>29:33</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>47.26 %</td><td>M</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">80</td><td><a href="athletehistory?athleteNumber=3294027" target="_top">Abhay SURYA</a></td><td>29:47</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>47.62 %</td><td>M</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:36</td><td>20</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">81</td><td><a href="athletehistory?athleteNumber=1785995" target="_top">Julie STAFFERTON</a></td><td>29:51</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>60.64 %</td><td>F</td><td>17</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:31</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">82</td><td><a href="athletehistory?athleteNumber=271155" target="_top">Kevin PARDY</a></td><td>29:52</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>50.17 %</td><td>M</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">83</td><td><a href="athletehistory?athleteNumber=3903734" target="_top">Andrew CHEN</a></td><td>30:06</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>49.39 %</td><td>M</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:22:52</td><td>15</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">84</td><td><a href="athletehistory?athleteNumber=4043596" target="_top">Suzanne EVANS</a></td><td>30:06</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>69.44 %</td><td>F</td><td>18</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:58</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">85</td><td><a href="athletehistory?athleteNumber=3270886" target="_top">Roger SLADE</a></td><td>30:17</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>59.71 %</td><td>M</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:04</td><td>28</td><td style="min-width:72px"/></tr><tr><td class="pos">86</td><td><a href="athletehistory?athleteNumber=3911794" target="_top">Daniel ZHANG</a></td><td>30:17</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>53.99 %</td><td>M</td><td>68</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:27:35</td><td>24</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">87</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">88</td><td><a href="athletehistory?athleteNumber=1345228" target="_top">Kara SMITH</a></td><td>30:22</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>52.58 %</td><td>F</td><td>19</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>5</td><td style="min-width:72px"/></tr><tr><td class="pos">89</td><td><a href="athletehistory?athleteNumber=1208556" target="_top">Colin SMITH</a></td><td>30:31</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>46.81 %</td><td>M</td><td>70</td><td><a href="../clubhistory?clubNum=23053">Ultrain</a></td><td>New PB!</td><td>29</td><td style="min-width:72px"/></tr><tr><td class="pos">90</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">91</td><td><a href="athletehistory?athleteNumber=3971846" target="_top">Rebecca POSTLETHWAITE</a></td><td>30:47</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>48.08 %</td><td>F</td><td>20</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">92</td><td><a href="athletehistory?athleteNumber=4290916" target="_top">Chin LIM</a></td><td>30:47</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>55.01 %</td><td>M</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>10</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/9773029"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">93</td><td><a href="athletehistory?athleteNumber=4488709" target="_top">Shaurya SHUKLA</a></td><td>30:56</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>51.02 %</td><td>M</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">94</td><td><a href="athletehistory?athleteNumber=3806290" target="_top">Rebecca WOOD</a></td><td>31:00</td><td><a href="../agecategorytable/?ageCat=SW20-24">SW20-24</a></td><td>47.74 %</td><td>F</td><td>21</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>14</td><td style="min-width:72px"/></tr><tr><td class="pos">95</td><td><a href="athletehistory?athleteNumber=4090905" target="_top">Natasha TRINKLE</a></td><td>31:07</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>48.37 %</td><td>F</td><td>22</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:24</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">96</td><td><a href="athletehistory?athleteNumber=2353796" target="_top">Warren PAGE</a></td><td>31:10</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>47.33 %</td><td>M</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:17</td><td>84</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/24529909"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">97</td><td><a href="athletehistory?athleteNumber=3904164" target="_top">Jonathan CHEN</a></td><td>31:10</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>50.64 %</td><td>M</td><td>75</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>18</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">98</td><td><a href="athletehistory?athleteNumber=4339838" target="_top">Joanna BELL</a></td><td>31:14</td><td><a href="../agecategorytable/?ageCat=SW18-19">SW18-19</a></td><td>47.55 %</td><td>F</td><td>23</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">99</td><td><a href="athletehistory?athleteNumber=1783568" target="_top">Andrew CARPENTER</a></td><td>31:16</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>50.43 %</td><td>M</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>120</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">100</td><td><a href="athletehistory?athleteNumber=1735935" target="_top">David GEORGE</a></td><td>31:30</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>41.27 %</td><td>M</td><td>77</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:24:19</td><td>39</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/19204898"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">101</td><td><a href="athletehistory?athleteNumber=2378428" target="_top">Owen PEREIRA</a></td><td>31:37</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>56.30 %</td><td>M</td><td>78</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:50</td><td>71</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">102</td><td><a href="athletehistory?athleteNumber=4376939" target="_top">Katie O'CALLAGHAN</a></td><td>31:38</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>49.74 %</td><td>F</td><td>24</td><td><a href="../clubhistory?clubNum=23079">GENERATION RUN</a></td><td>First Timer!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">103</td><td><a href="athletehistory?athleteNumber=3226546" target="_top">Lawrence YUEN</a></td><td>31:40</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>40.95 %</td><td>M</td><td>79</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:11</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">104</td><td><a href="athletehistory?athleteNumber=3911796" target="_top">Nathaniel ZHANG</a></td><td>31:41</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>59.02 %</td><td>M</td><td>80</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>New PB!</td><td>23</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">105</td><td><a href="athletehistory?athleteNumber=4217346" target="_top">Xiwu CHEN</a></td><td>31:43</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>46.87 %</td><td>M</td><td>81</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:22:59</td><td>16</td><td style="min-width:72px"/></tr><tr><td class="pos">106</td><td><a href="athletehistory?athleteNumber=4269724" target="_top">Jordan BROWN</a></td><td>31:46</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>40.61 %</td><td>M</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>8</td><td style="min-width:72px"/></tr><tr><td class="pos">107</td><td><a href="athletehistory?athleteNumber=1928330" target="_top">Kaiya BUNYON</a></td><td>31:59</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>52.01 %</td><td>F</td><td>25</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>11</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">108</td><td><a href="athletehistory?athleteNumber=3951953" target="_top">Jillian RIGG</a></td><td>32:01</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>50.60 %</td><td>F</td><td>26</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">109</td><td><a href="athletehistory?athleteNumber=2345651" target="_top">Jackie DOWLING</a></td><td>32:02</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>56.50 %</td><td>F</td><td>27</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:36</td><td>43</td><td style="min-width:72px"/></tr><tr><td class="pos">110</td><td><a href="athletehistory?athleteNumber=3985226" target="_top">Joy YIN</a></td><td>32:25</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>55.89 %</td><td>F</td><td>28</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">111</td><td><a href="athletehistory?athleteNumber=2848908" target="_top">Jenny PATERSON</a></td><td>32:28</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>48.46 %</td><td>F</td><td>29</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:55</td><td>65</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">112</td><td><a href="athletehistory?athleteNumber=2223986" target="_top">Emily PEREIRA</a></td><td>32:46</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>52.09 %</td><td>F</td><td>30</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:25:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">113</td><td><a href="athletehistory?athleteNumber=3286687" target="_top">Aaryan BHATIA</a></td><td>32:48</td><td><a href="../agecategorytable/?ageCat=JM15-17">JM15-17</a></td><td>42.43 %</td><td>M</td><td>83</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:30:23</td><td>32</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">114</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">115</td><td><a href="athletehistory?athleteNumber=4243723" target="_top">Om KAR</a></td><td>33:04</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>43.85 %</td><td>M</td><td>85</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:53</td><td>11</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">116</td><td><a href="athletehistory?athleteNumber=4388693" target="_top">Ling WU</a></td><td>33:08</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>50.00 %</td><td>F</td><td>31</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">117</td><td><a href="athletehistory?athleteNumber=4482563" target="_top">Susan SUN</a></td><td>33:16</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>49.20 %</td><td>F</td><td>32</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">118</td><td><a href="athletehistory?athleteNumber=4011261" target="_top">Roger HE</a></td><td>33:16</td><td><a href="../agecategorytable/?ageCat=JM11-14">JM11-14</a></td><td>42.64 %</td><td>M</td><td>86</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:20</td><td>18</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">119</td><td><a href="athletehistory?athleteNumber=4008975" target="_top">Cameron GODDARD</a></td><td>33:18</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>47.40 %</td><td>M</td><td>87</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:20:22</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">120</td><td><a href="athletehistory?athleteNumber=2374136" target="_top">Regina NEVILLE</a></td><td>33:22</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>46.80 %</td><td>F</td><td>33</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:26</td><td>62</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">121</td><td><a href="athletehistory?athleteNumber=4086945" target="_top">Jerry ZHANG</a></td><td>33:28</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>43.73 %</td><td>M</td><td>88</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:05</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">122</td><td><a href="athletehistory?athleteNumber=4193948" target="_top">Ashley COLEMAN-BOCK</a></td><td>33:40</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>43.96 %</td><td>F</td><td>34</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">123</td><td><a href="athletehistory?athleteNumber=4176258" target="_top">Jing YE</a></td><td>33:42</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>44.11 %</td><td>F</td><td>35</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:27</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">124</td><td><a href="athletehistory?athleteNumber=2135388" target="_top">Stephen WHITE</a></td><td>33:47</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>45.49 %</td><td>M</td><td>89</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:29</td><td>99</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">125</td><td><a href="athletehistory?athleteNumber=3951798" target="_top">Catherine LEIPOLD</a></td><td>33:55</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>46.83 %</td><td>F</td><td>36</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:54</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">126</td><td><a href="athletehistory?athleteNumber=3510394" target="_top">Donna KENNY</a></td><td>33:57</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>50.61 %</td><td>F</td><td>37</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:29</td><td>15</td><td style="min-width:72px"/></tr><tr><td class="pos">127</td><td><a href="athletehistory?athleteNumber=2221582" target="_top">Rohini PEREIRA</a></td><td>34:03</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>45.52 %</td><td>F</td><td>38</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:31:50</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">128</td><td><a href="athletehistory?athleteNumber=4498741" target="_top">Ivan DE SOUZA</a></td><td>34:28</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>43.47 %</td><td>M</td><td>90</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">129</td><td><a href="athletehistory?athleteNumber=2769381" target="_top">Marina JORDAN</a></td><td>34:35</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>49.69 %</td><td>F</td><td>39</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:34</td><td>77</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">130</td><td><a href="athletehistory?athleteNumber=2819837" target="_top">Toban WILD</a></td><td>34:36</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>37.28 %</td><td>M</td><td>91</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:00</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">131</td><td><a href="athletehistory?athleteNumber=2130247" target="_top">Rachel SCHRODER</a></td><td>34:37</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>42.85 %</td><td>F</td><td>40</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:16</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">132</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">133</td><td><a href="athletehistory?athleteNumber=4454515" target="_top">Tamaryn NURSE</a></td><td>34:59</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>42.50 %</td><td>F</td><td>41</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/7627900"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">134</td><td><a href="athletehistory?athleteNumber=2935289" target="_top">Suraj JOSE</a></td><td>35:26</td><td><a href="../agecategorytable/?ageCat=SM30-34">SM30-34</a></td><td>36.50 %</td><td>M</td><td>93</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">135</td><td><a href="athletehistory?athleteNumber=3226539" target="_top">Cameron SMITH</a></td><td>35:32</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>36.30 %</td><td>M</td><td>94</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:11</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">136</td><td><a href="athletehistory?athleteNumber=4501065" target="_top">Nick MULDER</a></td><td>35:33</td><td><a href="../agecategorytable/?ageCat=VM40-44">VM40-44</a></td><td>39.57 %</td><td>M</td><td>95</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">137</td><td><a href="athletehistory?athleteNumber=1717831" target="_top">Thomas SOM</a></td><td>35:34</td><td><a href="../agecategorytable/?ageCat=VM50-54">VM50-54</a></td><td>42.83 %</td><td>M</td><td>96</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:17</td><td>95</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">138</td><td><a href="athletehistory?athleteNumber=4501040" target="_top">Bec CURTIS</a></td><td>35:35</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>41.59 %</td><td>F</td><td>42</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">139</td><td><a href="athletehistory?athleteNumber=1783614" target="_top">Rachel SHEPPARD</a></td><td>35:36</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>44.62 %</td><td>F</td><td>43</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:54</td><td>80</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">140</td><td><a href="athletehistory?athleteNumber=2207985" target="_top">Andrew WEATHERHEAD</a></td><td>36:31</td><td><a href="../agecategorytable/?ageCat=VM65-69">VM65-69</a></td><td>47.24 %</td><td>M</td><td>97</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:16</td><td>54</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">141</td><td><a href="athletehistory?athleteNumber=3591013" target="_top">Robyn DIMOPOULOS</a></td><td>36:34</td><td><a href="../agecategorytable/?ageCat=SW30-34">SW30-34</a></td><td>40.75 %</td><td>F</td><td>44</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:27:13</td><td>49</td><td style="min-width:72px"/></tr><tr><td class="pos">142</td><td><a href="athletehistory?athleteNumber=1737530" target="_top">Derek PARR</a></td><td>36:37</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>42.69 %</td><td>M</td><td>98</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:27:36</td><td>63</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">143</td><td><a href="athletehistory?athleteNumber=4388798" target="_top">Natalie CHAN</a></td><td>36:42</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>42.82 %</td><td>F</td><td>45</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:37</td><td>4</td><td style="min-width:72px"/></tr><tr><td class="pos">144</td><td><a href="athletehistory?athleteNumber=4269919" target="_top">Cathy CANNON</a></td><td>36:52</td><td><a href="../agecategorytable/?ageCat=VW60-64">VW60-64</a></td><td>54.97 %</td><td>F</td><td>46</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:33:00</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">145</td><td><a href="athletehistory?athleteNumber=3093991" target="_top">Michelle TAY</a></td><td>36:54</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>40.79 %</td><td>F</td><td>47</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:02</td><td>48</td><td style="min-width:72px"/></tr><tr><td class="pos">146</td><td><a href="athletehistory?athleteNumber=4453453" target="_top">Rochelle WHEATLEY</a></td><td>37:16</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>40.38 %</td><td>F</td><td>48</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">147</td><td><a href="athletehistory?athleteNumber=2748671" target="_top">Robyn WHEATLEY</a></td><td>37:26</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>55.83 %</td><td>F</td><td>49</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">148</td><td><a href="athletehistory?athleteNumber=3212945" target="_top">Graham ROBERTSON</a></td><td>37:33</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>48.87 %</td><td>M</td><td>99</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:06</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">149</td><td><a href="athletehistory?athleteNumber=1737510" target="_top">Helen PARR</a></td><td>38:34</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>45.16 %</td><td>F</td><td>50</td><td><a href="../clubhistory?clubNum=21482">TXR Runners</a></td><td>PB stays at 00:29:37</td><td>70</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/11621051"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">150</td><td><a href="athletehistory?athleteNumber=4254332" target="_top">Ajantha KUMARI</a></td><td>38:46</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>41.36 %</td><td>F</td><td>51</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">151</td><td><a href="athletehistory?athleteNumber=3757996" target="_top">Elizabeth ALDA</a></td><td>39:38</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>43.94 %</td><td>F</td><td>52</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:37:58</td><td>19</td><td style="min-width:72px"/></tr><tr><td class="pos">152</td><td><a href="athletehistory?athleteNumber=3803840" target="_top">Annie CHEN</a></td><td>39:49</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>44.87 %</td><td>F</td><td>53</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>36</td><td style="min-width:72px"/></tr><tr><td class="pos">153</td><td><a href="athletehistory?athleteNumber=1786065" target="_top">Tina TIAN</a></td><td>39:52</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>40.22 %</td><td>F</td><td>54</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>PB stays at 00:27:48</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">154</td><td><a href="athletehistory?athleteNumber=3911799" target="_top">Sally ZHOU</a></td><td>39:53</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>39.82 %</td><td>F</td><td>55</td><td><a href="../clubhistory?clubNum=23243">Australian Chinese Running Group</a></td><td>PB stays at 00:39:01</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">155</td><td><a href="athletehistory?athleteNumber=3971595" target="_top">Sue BROWN</a></td><td>40:05</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>41.83 %</td><td>F</td><td>56</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:38:32</td><td>26</td><td style="min-width:72px"/></tr><tr><td class="pos">156</td><td><a href="athletehistory?athleteNumber=2207989" target="_top">Mary WEATHERHEAD</a></td><td>40:21</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>51.80 %</td><td>F</td><td>57</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:25</td><td>38</td><td style="min-width:72px"/></tr><tr><td class="pos">157</td><td><a href="athletehistory?athleteNumber=4363576" target="_top">Ann-Marie JEFFERY</a></td><td>41:03</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>41.33 %</td><td>F</td><td>58</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">158</td><td><a href="athletehistory?athleteNumber=2269520" target="_top">Melissa CHAN</a></td><td>41:31</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>43.60 %</td><td>F</td><td>59</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:23</td><td>11</td><td style="min-width:72px"/></tr><tr><td class="pos">159</td><td><a href="athletehistory?athleteNumber=3147151" target="_top">Nicola CICCARELLI</a></td><td>41:47</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>37.61 %</td><td>F</td><td>60</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:32:36</td><td>15</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">160</td><td><a href="athletehistory?athleteNumber=656410" target="_top">Naomi (Tullae) CROTTY</a></td><td>41:53</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>36.13 %</td><td>F</td><td>61</td><td><a href="../clubhistory?clubNum=20982">Wooters Runners</a></td><td>PB stays at 00:39:11</td><td>146</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun v25 Club" src="http://images.parkrun.com/website/results/25_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/25440878"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">161</td><td><a href="athletehistory?athleteNumber=2180623" target="_top">Jacqui CARTER</a></td><td>42:05</td><td><a href="../agecategorytable/?ageCat=VW65-69">VW65-69</a></td><td>49.66 %</td><td>F</td><td>62</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:17</td><td>61</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">162</td><td><a href="athletehistory?athleteNumber=3364517" target="_top">Patrick HAWKINS</a></td><td>42:06</td><td><a href="../agecategorytable/?ageCat=JM10">JM10</a></td><td>37.49 %</td><td>M</td><td>100</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:54</td><td>22</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 10 Club" src="https://images.parkrun.com/website/results/10_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">163</td><td><a href="athletehistory?athleteNumber=3364511" target="_top">Lisa SOMERVILLE</a></td><td>42:06</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>37.73 %</td><td>F</td><td>63</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:35:58</td><td>32</td><td style="min-width:72px"/></tr><tr><td class="pos">164</td><td><a href="athletehistory?athleteNumber=4491527" target="_top">Meaghan CHARRETT</a></td><td>42:16</td><td><a href="../agecategorytable/?ageCat=JW11-14">JW11-14</a></td><td>37.18 %</td><td>F</td><td>64</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">165</td><td><a href="athletehistory?athleteNumber=4412009" target="_top">Rohan CHARRETT</a></td><td>42:17</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>34.06 %</td><td>M</td><td>101</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:29:15</td><td>6</td><td style="min-width:72px"/></tr><tr><td class="pos">166</td><td><a href="athletehistory?athleteNumber=4488623" target="_top">Avantika AVANTIKA</a></td><td>42:47</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>35.37 %</td><td>F</td><td>65</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">167</td><td><a href="athletehistory?athleteNumber=2202224" target="_top">Alex ROBINS</a></td><td>42:58</td><td><a href="../agecategorytable/?ageCat=VM75-79">VM75-79</a></td><td>44.88 %</td><td>M</td><td>102</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:27</td><td>100</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 100 Club" src="https://images.parkrun.com/website/results/100_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">168</td><td><a href="athletehistory?athleteNumber=4427548" target="_top">Charlotte DAVIDSON</a></td><td>43:18</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>40.53 %</td><td>F</td><td>66</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>2</td><td style="min-width:72px"/></tr><tr><td class="pos">169</td><td><a href="athletehistory?athleteNumber=2334951" target="_top">Nathan WRIGHT</a></td><td>43:18</td><td><a href="../agecategorytable/?ageCat=SM25-29">SM25-29</a></td><td>29.79 %</td><td>M</td><td>103</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:26:14</td><td>53</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">170</td><td><a href="athletehistory?athleteNumber=770352" target="_top">Joan PERRY</a></td><td>43:35</td><td><a href="../agecategorytable/?ageCat=VW75-79">VW75-79</a></td><td>55.64 %</td><td>F</td><td>67</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:37:04</td><td>57</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">171</td><td><a href="athletehistory?athleteNumber=4427563" target="_top">Lily DAVIDSON</a></td><td>43:39</td><td><a href="../agecategorytable/?ageCat=JW10">JW10</a></td><td>41.50 %</td><td>F</td><td>68</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>1</td><td style="min-width:72px"/></tr><tr><td class="pos">172</td><td><a href="athletehistory?athleteNumber=2395087" target="_top">Ashley WILLIS</a></td><td>43:39</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>33.91 %</td><td>F</td><td>69</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:47</td><td>73</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a><a href="http://www.strava.com/athletes/16902477"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">173</td><td><a href="athletehistory?athleteNumber=2988047" target="_top">Anna DAVIDSON</a></td><td>43:39</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>34.67 %</td><td>F</td><td>70</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:28:39</td><td>31</td><td style="min-width:72px"/></tr><tr><td class="pos">174</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">175</td><td><a href="athletehistory?athleteNumber=2759962" target="_top">Sonny LIN</a></td><td>44:05</td><td><a href="../agecategorytable/?ageCat=VM45-49">VM45-49</a></td><td>32.17 %</td><td>M</td><td>105</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:39:37</td><td>2</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/529279"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">176</td><td><a href="athletehistory?athleteNumber=4037877" target="_top">Sadia NAZIER</a></td><td>44:49</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>34.14 %</td><td>F</td><td>71</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:41:29</td><td>20</td><td style="min-width:72px"/></tr><tr><td class="pos">177</td><td><a href="athletehistory?athleteNumber=4272564" target="_top">Margaret HELLYER</a></td><td>45:46</td><td><a href="../agecategorytable/?ageCat=VW55-59">VW55-59</a></td><td>40.09 %</td><td>F</td><td>72</td><td><a href="../clubhistory?clubNum=1187"/></td><td>New PB!</td><td>12</td><td style="min-width:72px"/></tr><tr><td class="pos">178</td><td><a href="athletehistory?athleteNumber=2068233" target="_top">Leanne OBERIN</a></td><td>47:02</td><td><a href="../agecategorytable/?ageCat=VW45-49">VW45-49</a></td><td>33.77 %</td><td>F</td><td>73</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:45:24</td><td>98</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">179</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">180</td><td><a href="athletehistory?athleteNumber=4037850" target="_top">Shanaaz NAZIER-CUPIDO</a></td><td>49:11</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>30.60 %</td><td>F</td><td>74</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:48:37</td><td>23</td><td style="min-width:72px"/></tr><tr><td class="pos">181</td><td><a href="athletehistory?athleteNumber=3975800" target="_top">Ken JAMES</a></td><td>49:30</td><td><a href="../agecategorytable/?ageCat=VM70-74">VM70-74</a></td><td>36.06 %</td><td>M</td><td>107</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:53</td><td>18</td><td style="min-width:72px"/></tr><tr><td class="pos">182</td><td><a href="athletehistory?athleteNumber=1245151" target="_top">Herbert EDER</a></td><td>51:01</td><td><a href="../agecategorytable/?ageCat=VM85-89">VM85-89</a></td><td>49.36 %</td><td>M</td><td>108</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:47:04</td><td>72</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">183</td><td><a href="athletehistory?athleteNumber=1245122" target="_top">Pauline EDER</a></td><td>51:06</td><td><a href="../agecategorytable/?ageCat=VW70-74">VW70-74</a></td><td>45.04 %</td><td>F</td><td>75</td><td><a href="../clubhistory?clubNum=23016">Heart Foundation Walking</a></td><td>PB stays at 00:46:19</td><td>69</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">184</td><td><a href="athletehistory?athleteNumber=4358918" target="_top">Lais MATOS</a></td><td>51:26</td><td><a href="../agecategorytable/?ageCat=SW25-29">SW25-29</a></td><td>28.78 %</td><td>F</td><td>76</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:29</td><td>9</td><td style="min-width:72px"/></tr><tr><td class="pos">185</td><td><a href="athletehistory?athleteNumber=2867022" target="_top">Viviane RAMOS</a></td><td>51:27</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>30.35 %</td><td>F</td><td>77</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:44:04</td><td>6</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/2075388"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">186</td><td><a href="athletehistory?athleteNumber=3804017" target="_top">Deanne CATMULL</a></td><td>52:05</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>29.06 %</td><td>F</td><td>78</td><td><a href="../clubhistory?clubNum=22935">Running in the Burbs</a></td><td>First Timer!</td><td>7</td><td style="min-width:72px"/></tr><tr><td class="pos">187</td><td><a href="athletehistory?athleteNumber=3135805" target="_top">Anthi CICCARELLI</a></td><td>54:36</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>27.47 %</td><td>F</td><td>79</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:34:48</td><td>34</td><td style="min-width:72px"><a href="http://www.strava.com/athletes/19094315"><img src="https://images.parkrun.com/website/partners/strava/AppIcon60x60.png" alt="Strava logo" style="padding-right: 4px; height: 20px; width: 20px"/></a></td></tr><tr><td class="pos">188</td><td><a href="athletehistory?athleteNumber=4477406" target="_top">Pinki SHUKLA</a></td><td>55:44</td><td><a href="../agecategorytable/?ageCat=VW35-39">VW35-39</a></td><td>27.27 %</td><td>F</td><td>80</td><td><a href="../clubhistory?clubNum=1187"/></td><td>First Timer!</td><td>3</td><td style="min-width:72px"/></tr><tr><td class="pos">189</td><td>Unknown</td><td/><td/><td/><td/><td/><td/><td/><td/><td/></tr><tr><td class="pos">190</td><td><a href="athletehistory?athleteNumber=1968284" target="_top">Eric NEGROS</a></td><td>1:03:12</td><td><a href="../agecategorytable/?ageCat=VM55-59">VM55-59</a></td><td>24.31 %</td><td>M</td><td>110</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:52:13</td><td>67</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr><tr><td class="pos">191</td><td><a href="athletehistory?athleteNumber=3286975" target="_top">Sonia BHATIA</a></td><td>1:03:33</td><td><a href="../agecategorytable/?ageCat=VW50-54">VW50-54</a></td><td>26.38 %</td><td>F</td><td>81</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:56:42</td><td>17</td><td style="min-width:72px"/></tr><tr><td class="pos">192</td><td><a href="athletehistory?athleteNumber=1716573" target="_top">Nina MUSUMECI</a></td><td>1:03:39</td><td><a href="../agecategorytable/?ageCat=VW40-44">VW40-44</a></td><td>24.17 %</td><td>F</td><td>82</td><td><a href="../clubhistory?clubNum=1187"/></td><td>PB stays at 00:36:23</td><td>87</td><td style="min-width:72px"><a href="http://www.parkrun.com/about/parkrunclubs"><img alt="Member of the parkrun 50 Club" src="https://images.parkrun.com/website/results/50_club_mini.jpg" style="padding-right: 4px; height: 20px; width: 20px"></a></td></tr></tbody></table></p><div class="paddedt left"><h3 class="paddedtandb">Thanks to the volunteers</h3><p class="paddedb">We are very grateful to the volunteers who made this event happen:Scott CARTLEDGE, Denzil CUPIDO, Jennifer HOGARTY, Braden MUNS, Nina MUSUMECI, Jake NGU, Caleb PAYNE, Allen PRATT, Ed WILSON</p><p class="paddedb">Why not <a href="../../volunteer/">get involved and volunteer</a>?</p><h3 class="paddetandb" id="explainTable">
	What is this table? </h3>
'''
run.addRunners(text)
run.addVolunteers(text)

In [7]:
html = run.createWeek(2)
print(html)
